In [1]:
import pandas as pd
import numpy as np

In [4]:
filenames = ['case note - unformatted',
            'Class and Testing History - unformatted',
            'Client Community Activity - unformatted',
            'Client information (demographics and assessment) - unformatted',
            'DDA Case info - unformatted','Employer contact history - unformatted',
            'Employer placement hours - unformatted',
            'Employer placement record - unformatted',
            'Funding - unformatted',
            'guardianship information - unformatted',
            'individual work schedule (eval) - unformatted',
            'Intake Wizard (intake application) - unformatted','Mentor availability - unformatted',
            'outcome activity (case note) - unformatted',
            'outcome area (case notes) - unformatted',
            'outcome goal (goals) - unformatted',
            'people - unformatted',
            'profiles - unformatted',
            'profiles - users - unformatted',
            'Relationships (contacts) - unformatted',
            'contacts - unformatted']

In [13]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [159]:
pd.reset_option('display.max_columns', None)
pd.reset_option('display.max_rows', None)

people is the highest source of truth for participants the ID associated with people is the id used in the individual

people also holds contacts, the individual is the type for participants

### This is the start of merging the individual people information into one big spreadsheet

In [278]:
file1 = pd.read_excel(r'S:\Rehab\Apricot\Migration\people - unformatted.xlsx')
file1 = file1.add_prefix('people_')
file2 = pd.read_excel(r'S:\Rehab\Apricot\Migration\Client information (demographics and assessment) - unformatted.xlsx')
file2 = file2.add_prefix('demo_')

In [279]:
file1.columns

Index(['people_ID', 'people_First Name', 'people_Full Name', 'people_Title',
       'people_Forms Dropdowns', 'people_Created', 'people_Created By',
       'people_Content Type', 'people_E-Mail', 'people_E-mail 2',
       'people_Phone', 'people_Mobile Number', 'people_Professional Type',
       'people_Comments', 'people_Filter', 'people_Folder Child Count',
       'people_Item Child Count', 'people_Last Name', 'people_Modified',
       'people_Modified By', 'people_Item Type', 'people_Path'],
      dtype='object')

In [280]:
merge1 = file1.merge(file2,how='left',left_on='people_Full Name',right_on='demo_Individual')
merge2 = merge1[merge1['people_Forms Dropdowns']=='Individual']

In [281]:
# merge2.to_excel(r'S:\Rehab\Apricot\Migration\people merge.xlsx')

In [282]:
file3 = pd.read_excel(r'S:\Rehab\Apricot\Migration\profiles - unformatted.xlsx')
file3 = file3.add_prefix('prof_')
file4 = pd.read_excel(r'S:\Rehab\Apricot\Migration\Intake Wizard (intake application) - unformatted.xlsx')
file4 = file4.add_prefix('intake_')

In [283]:
merge3 = merge2.merge(file3,how='left',left_on='people_Full Name',right_on='prof_Person')
# merge3 = merge3.drop(columns=['ID_x','Title','Forms Dropdowns','Created By','Content Type_x','ID_y','Content Type_y','View_x','View_y','Item Type_x','Item Type_y','Path','Path_x','Path_y','Item Type','Created_x','Created By_x','E-Mail_x'])
# merge3.head(2)

In [284]:
# merge3.columns

In [285]:
merge4 = merge3.merge(file4,how='left',left_on='people_Full Name',right_on='intake_Full Name')
merge4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1532 entries, 0 to 1531
Columns: 257 entries, people_ID to intake_Path
dtypes: datetime64[ns](19), float64(47), int64(3), object(188)
memory usage: 3.0+ MB


In [286]:
# add in dda information to the df

ddacase = pd.read_excel(r'S:\Rehab\Apricot\Migration\DDA Case info - unformatted.xlsx')
ddacase = ddacase.add_prefix('dda_')

merge5 = merge4.merge(ddacase,how='left',left_on='intake_Client',right_on='dda_Individual')

In [287]:
# remove duplicates

merge5 = merge5.drop_duplicates(subset=['people_First Name','people_Last Name'])

In [288]:
# remove any text (within parethesis) to standardize names

merge5['people_First Name'] = merge5['people_First Name'].replace(r'\(.*?\)','',regex=True).str.strip()
merge5['people_Last Name'] = merge5['people_Last Name'].replace(r'\(.*?\)','',regex=True).str.strip()

merge5['people_First Name'] = merge5['people_First Name'].replace(r'\".*?\"','',regex=True).str.strip()
merge5['people_Last Name'] = merge5['people_Last Name'].replace(r'\".*?\"','',regex=True).str.strip()

In [305]:
merge6 = merge5[~merge5['people_Full Name'].str.contains('test', case=False, na=False)].copy()

In [306]:
merge6 = merge6[merge6['people_Modified'] >= datetime(2023,1,1)].copy()
merge6

,people_ID,people_First Name,people_Full Name,people_Title,people_Forms Dropdowns,people_Created,people_Created By,people_Content Type,people_E-Mail,people_E-mail 2,people_Phone,people_Mobile Number,people_Professional Type,people_Comments,people_Filter,people_Folder Child Count,people_Item Child Count,people_Last Name,people_Modified,people_Modified By,people_Item Type,people_Path,demo_ID,demo_Individual,demo_Preferred Names,demo_Date of Birth,demo_Gender,demo_Disability - Primary,demo_Disabilities - Additional,demo_Disability / Barrier Notes:,demo_Date of Last Employment,demo_Smoker?,demo_Veteran - Discharge Date and Disposition,"demo_If homeless, for how long?","demo_If homeless, last city of residence",demo_Emergency Contact Name,demo_Emergency Contact Number,demo_Emergency Contact Relationship,demo_Household Size,demo_Household Type,demo_Health Insurance,demo_Aboriginal Status,demo_Agency Exit Date,demo_Agency Intake Date,demo_Allergies,demo_Alt. Telephone Number,demo_Benefits,demo_Bladder & Bowel Functions,demo_City,demo_Clothing (Last Seen Wearing),demo_Communication Level,demo_Communication Support,demo_Content Type,demo_Coordinator,demo_Coordinator's Cell Number,demo_Country,demo_Created,demo_Created By,demo_Criminal History,demo_Daily Living Support,demo_DDA Case File Number (if applicable),demo_Diet/Nutrition,demo_Education Level,demo_E-mail Address,demo_Ethnicity,demo_Eye Colour,demo_Folder Child Count,demo_Hair Colour,demo_Head Of Household,demo_Health Care Plan Number,demo_Health Concerns,demo_Height,demo_Household Income,"demo_If clean and sober housing, how long?",demo_Immigrant and Refugee Status,demo_Item Child Count,demo_Language Primary,demo_Language Spoken,demo_Living Situation,demo_Marital Status,demo_Medications and Conditions,demo_MISSING PERSON POSTER,demo_Mobility Aids,demo_Modified,demo_Modified By,demo_Number of Dependents,demo_Office Number,demo_Orion ID Number,demo_Other I.D. Numbers,demo_Personal Hygiene,demo_Pertinent Information,demo_Photo,demo_Postal Code,demo_Province,demo_Safety Requirement,demo_Sleep Patterns,demo_Social Insurance Number,demo_Special Characteristics,demo_Special Education History,demo_Street Address,demo_Telephone Number,demo_Transportation,demo_Veteran Status,demo_View,demo_Visual/Hearing Aids,demo_Weight,demo_Work Restrictions,demo_Work Restrictions Details,demo_Item Type,demo_Path,prof_ID,prof_Person,prof_Content Type,prof_Programs,prof_Organization,prof_State,prof_City,prof_ZIP Code,prof_Active,prof_Address,prof_Anonymous,prof_Became Donor On,prof_Board Position,prof_Case Worker,prof_Charitable Interest,prof_Checkbox Column 1,prof_Checkbox Column 2,prof_Choice Column 1,prof_Choice Column 2,prof_Choice Column 3,prof_Choice Column 4,prof_Choice Column 5,prof_Comments,prof_Comments Column 1,prof_Comments Column 2,prof_Country,prof_Created,prof_Created By,prof_Date Column 1,prof_Date Column 2,prof_Donor Motivation,prof_E-Mail,prof_Email Lists,prof_Folder Child Count,prof_Item Child Count,prof_Job Title,prof_Link Column 1,prof_Link Column 2,prof_Mailing Address,prof_Mailing City,prof_Mailing Country,prof_Mailing Lists,prof_Mailing State,prof_Mailing ZIP Code,prof_Membership Type,prof_Mobile Phone,prof_Modified,prof_Modified By,prof_Number Column 1,prof_Number Column 2,prof_Permissions Set,prof_Personal Interest,prof_Phone,prof_Phone 2,prof_Photo,prof_Professional Type,prof_Residence,prof_Set Show Programs and Residences - disabled,prof_Show Programs,prof_Show Residence,prof_Text Column 1,prof_Text Column 3,prof_Text Column 4,prof_Text Column 5,prof_View,prof_Volunteer Position,prof_Web Page,prof_Item Type,prof_Path,intake_Client,intake_Full Name,intake_Nickname,intake_Address,intake_Phone,intake_City,intake_Orion Location,intake_Start Date,intake_Referral Details: Service,intake_Complete Client Intake Process,intake_IntakeWizardCompleted,intake_Processing details: Completed Winning Traits,intake_Processing details: Funding and Project Eligibility,intake_Status of Intake,inta

In [307]:
merge6.to_excel(r'S:\Rehab\Apricot\Migration\exports\real people_merge2.xlsx')

#### Merge6 was changed to add in additional people

In [2]:
# added missing people to the merge6

merge6 = pd.read_excel(r'S:\Rehab\Apricot\Migration\exports\real people_merge2.xlsx')

#### Individual profile start

In [388]:
merge6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 528 entries, 0 to 527
Columns: 276 entries, people_ID to dda_Path
dtypes: bool(1), datetime64[ns](22), float64(98), int64(6), object(149)
memory usage: 1.1+ MB


In [389]:
# create a pseudo individual profiles import sheet

individual = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'intake_Nickname',
    'intake_Phone',
    'intake_Email',
    'intake_Address',
    'intake_City',
    'intake_State',
    'intake_Zip Code:County',
    'intake_Zip Code',
    'intake_Guardian:',
    'intake_Guardian: email',
    'intake_Guardian: First Name',
    'intake_Guardian: Last Name',
    'intake_Guardian: Mobile Phone',
    'intake_Guardian: Phone',
    'intake_Guardian: Status',
    'dda_DDA Case File Number',
    'dda_Funding Code',
    'dda_Acuity Level',
    'intake_Orion Location'
]]

In [390]:
individual.head(2)

,people_First Name,people_Last Name,demo_Date of Birth,intake_Nickname,intake_Phone,intake_Email,intake_Address,intake_City,intake_State,intake_Zip Code:County,intake_Zip Code,intake_Guardian:,intake_Guardian: email,intake_Guardian: First Name,intake_Guardian: Last Name,intake_Guardian: Mobile Phone,intake_Guardian: Phone,intake_Guardian: Status,dda_DDA Case File Number,dda_Funding Code,dda_Acuity Level,intake_Orion Location
0,Keith,Shuman,1970-05-22 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Melissa,Waltjen,1977-10-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [391]:
# format the individual profile 
individual = individual.copy()
individual['Guardian info'] = individual['intake_Guardian: email']+' '+individual['intake_Guardian: First Name']+' '+individual['intake_Guardian: Last Name']+' '+individual['intake_Guardian: Phone']+' '+individual['intake_Guardian: Mobile Phone']+' '+individual['intake_Guardian: Status']

individual2 = individual.drop(columns=['intake_Guardian: email','intake_Guardian: First Name','intake_Guardian: Last Name','intake_Guardian: Phone','intake_Guardian: Mobile Phone','intake_Guardian: Status']).copy()
individual2['intake_Orion Location'] = individual2['intake_Orion Location'].replace('Auburn',1174)
individual2['intake_Orion Location'] = individual2['intake_Orion Location'].replace('Mukilteo',2870)

individual2['migrationID'] = range(1,len(individual) + 1)
individual2.head(2)

,people_First Name,people_Last Name,demo_Date of Birth,intake_Nickname,intake_Phone,intake_Email,intake_Address,intake_City,intake_State,intake_Zip Code:County,intake_Zip Code,intake_Guardian:,dda_DDA Case File Number,dda_Funding Code,dda_Acuity Level,intake_Orion Location,Guardian info,migrationID
0,Keith,Shuman,1970-05-22 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,Melissa,Waltjen,1977-10-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [392]:
# from datetime import datetime

individual2['demo_Date of Birth'] = individual2['demo_Date of Birth'].fillna('1/1/1800')
individual2.head(2)

,people_First Name,people_Last Name,demo_Date of Birth,intake_Nickname,intake_Phone,intake_Email,intake_Address,intake_City,intake_State,intake_Zip Code:County,intake_Zip Code,intake_Guardian:,dda_DDA Case File Number,dda_Funding Code,dda_Acuity Level,intake_Orion Location,Guardian info,migrationID
0,Keith,Shuman,1970-05-22 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,Melissa,Waltjen,1977-10-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [393]:
individual2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 528 entries, 0 to 527
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   people_First Name         528 non-null    object 
 1   people_Last Name          528 non-null    object 
 2   demo_Date of Birth        528 non-null    object 
 3   intake_Nickname           75 non-null     object 
 4   intake_Phone              438 non-null    object 
 5   intake_Email              389 non-null    object 
 6   intake_Address            438 non-null    object 
 7   intake_City               444 non-null    object 
 8   intake_State              456 non-null    object 
 9   intake_Zip Code:County    417 non-null    object 
 10  intake_Zip Code           428 non-null    float64
 11  intake_Guardian:          456 non-null    float64
 12  dda_DDA Case File Number  34 non-null     float64
 13  dda_Funding Code          32 non-null     object 
 14  dda_Acuity

In [394]:
individual2.to_excel(r'S:\Rehab\Apricot\Migration\exports\real individual profile2.xlsx')

#### individual done

#### demographics start

In [395]:
demo = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'prof_Created',
    'demo_Gender',
    'demo_Ethnicity',
    'demo_Language Primary',
    'demo_Language Spoken',
    'demo_Marital Status',
    'demo_Veteran Status',
    'demo_Head Of Household',
    'demo_Living Situation',
    'demo_Immigrant and Refugee Status',
    'intake_Orion Location'
]].copy()

demo['intake_Orion Location'] = demo['intake_Orion Location'].replace('Auburn',1174)
demo['intake_Orion Location'] = demo['intake_Orion Location'].replace('Mukilteo',2870)

In [396]:
demo['demo_Gender'].value_counts()

demo_Gender
Male       335
Female     171
Trans        6
Other        2
Unknown      1
Name: count, dtype: int64

In [397]:
demo.loc[:,'demo_Gender'] = demo['demo_Gender'].str.replace('Trans','Transgender')

In [398]:
demo.loc[:,'demo_Ethnicity'] = demo['demo_Ethnicity'].replace(r';#\d{2}','|',regex=True)
demo.loc[:,'demo_Ethnicity'] = demo['demo_Ethnicity'].str.rstrip('|')
demo.loc[:,'demo_Ethnicity'] = demo['demo_Ethnicity'].str.replace(';#','')

In [399]:
demo.loc[:,'demo_Ethnicity'] = demo['demo_Ethnicity'].str.replace('Hawaiin','Hawaiian')
demo.loc[:,'demo_Ethnicity'] = demo['demo_Ethnicity'].str.replace('African American / Black','Black / African American')
demo.loc[:,'demo_Ethnicity'] = demo['demo_Ethnicity'].str.replace('Multi-Racial','Other')

In [400]:
demo['demo_Language Primary'] = demo['demo_Language Primary'].str.replace('Other (please specify)','Other')

In [401]:
mar_rep = {'Divorced':'Single','Separated':'Single','Widowed':'Single'}

demo['demo_Marital Status'] = demo['demo_Marital Status'].replace(mar_rep)

In [402]:
vet_rep = {
    'Current Active Duty, Guard, Reserves or Veteran':'Currently AD, Guard, Reserves or Veteran',
    'Veteran - Present or Past ':'Currently AD, Guard, Reserves or Veteran',
    'Partner or Spouse of Active Service Member':'Partner or Spouse of Active Service Member/Veteran',
    'Minor Dependent of Active Service Member':'None'
}

demo['demo_Veteran Status'] = demo['demo_Veteran Status'].replace(vet_rep)
demo['demo_Veteran Status'] = demo['demo_Veteran Status'].fillna('None')

In [403]:
head_rep = {
    False:'No',
    True:'Yes'
}

demo['demo_Head Of Household'] = demo['demo_Head Of Household'].replace(head_rep)
demo['demo_Head Of Household'] = demo['demo_Head Of Household'].fillna('Unknown')

In [404]:
im_rep = {
    'Immigrant':'Immigrant status',
    'Refugee':'Refugee Status',
    'Unknown;#None':'Neither'
}

demo['demo_Immigrant and Refugee Status'] = demo['demo_Immigrant and Refugee Status'].replace(im_rep)
demo['demo_Immigrant and Refugee Status'] = demo['demo_Immigrant and Refugee Status'].fillna('Neither')

In [405]:
liverep = {
    'Homeless':'Homeless / Unhoused',
    'Independently / with Spouse or Partner':'Alone or with Spouse / Partner',
    'Other permanent housing (CSBG data)':'Other',
    'Own (CSBG data)':'Alone or with Spouse / Partner',
    'Rent (CSBG data)':'Alone or with Spouse / Partner',
    'Transitional / Temporary Housing':'Temporary/Transitional Housing',
    'With Parents':'With Parents or other/extended family',
    'With Extended Family':'With Parents or other/extended family'
}

demo['demo_Living Situation'] = demo['demo_Living Situation'].replace(liverep)
demo['demo_Living Situation'] = demo['demo_Living Situation'].fillna('Unknown')

In [406]:
demo['demo_Head Of Household'].value_counts()

demo_Head Of Household
0.0        479
1.0         39
Unknown     10
Name: count, dtype: int64

In [407]:
headrep = {
    0:'No',
    1:'Yes'
}

demo['demo_Head Of Household'] = demo['demo_Head Of Household'].replace(headrep)
demo.head(2)

,people_First Name,people_Last Name,demo_Date of Birth,prof_Created,demo_Gender,demo_Ethnicity,demo_Language Primary,demo_Language Spoken,demo_Marital Status,demo_Veteran Status,demo_Head Of Household,demo_Living Situation,demo_Immigrant and Refugee Status,intake_Orion Location
0,Keith,Shuman,1970-05-22 00:00:00,2021-04-08 12:09:20,Male,Caucasian / White,English,English,Single,None,No,With Roommates,Neither,NaN
1,Melissa,Waltjen,1977-10-06 00:00:00,2023-01-05 09:18:44,Female,Caucasian / White,English,English,Single,None,No,Other,Neither,NaN


In [408]:
demo['migrationID'] = range(1,len(demo) + 1)

In [409]:
demo.to_excel(r'S:\Rehab\Apricot\Migration\exports\real demographics2.xlsx')

#### Demographics done

In [145]:
for col in merge5.columns:
    print(col)

people_ID
people_Last Name
people_First Name
people_Full Name
people_Title
people_Forms Dropdowns
people_Created
people_Created By
people_Content Type
people_E-Mail
people_E-mail 2
people_Phone
people_Mobile Number
people_Professional Type
people_Item Type
people_Path
demo_ID
demo_Individual
demo_Preferred Names
demo_Date of Birth
demo_Gender
demo_Disability - Primary
demo_Disabilities - Additional
demo_Disability / Barrier Notes:
demo_Date of Last Employment
demo_Smoker?
demo_Veteran - Discharge Date and Disposition
demo_If homeless, for how long?
demo_If homeless, last city of residence
demo_Emergency Contact Name
demo_Emergency Contact Number
demo_Emergency Contact Relationship
demo_Household Size
demo_Household Type
demo_Health Insurance
demo_Aboriginal Status
demo_Agency Exit Date
demo_Agency Intake Date
demo_Allergies
demo_Alt. Telephone Number
demo_Benefits
demo_Bladder & Bowel Functions
demo_City
demo_Clothing (Last Seen Wearing)
demo_Communication Level
demo_Communication Supp

#### Intake Assessment start

In [532]:
for row in merge6:
    print(row)

people_ID
people_First Name
people_Full Name
people_Title
people_Forms Dropdowns
people_Created
people_Created By
people_Content Type
people_E-Mail
people_E-mail 2
people_Phone
people_Mobile Number
people_Professional Type
people_Comments
people_Filter
people_Folder Child Count
people_Item Child Count
people_Last Name
people_Modified
people_Modified By
people_Item Type
people_Path
demo_ID
demo_Individual
demo_Preferred Names
demo_Date of Birth
demo_Gender
demo_Disability - Primary
demo_Disabilities - Additional
demo_Disability / Barrier Notes:
demo_Date of Last Employment
demo_Smoker?
demo_Veteran - Discharge Date and Disposition
demo_If homeless, for how long?
demo_If homeless, last city of residence
demo_Emergency Contact Name
demo_Emergency Contact Number
demo_Emergency Contact Relationship
demo_Household Size
demo_Household Type
demo_Health Insurance
demo_Aboriginal Status
demo_Agency Exit Date
demo_Agency Intake Date
demo_Allergies
demo_Alt. Telephone Number
demo_Benefits
demo_Bla

In [24]:
intassess = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'demo_Education Level',
    'demo_Special Education History',
    'demo_Transportation',
    'demo_Allergies',
    'demo_Disability - Primary',
    'demo_Disabilities - Additional',
    'demo_Medications and Conditions',
    'demo_Criminal History',
    'demo_Smoker?',
    'demo_Household Income',
    'demo_Benefits',
    'demo_HealthInsurance',
    'intake_Orion Location'
]].copy()

intassess['intake_Orion Location'] = intassess['intake_Orion Location'].replace('Auburn',1174)
intassess['intake_Orion Location'] = intassess['intake_Orion Location'].replace('Mukilteo',2870)

steps to do:

- yes/no for allergies
- yes/no for each mental disability
- yes/no for criminal history
- yes/no for household income

In [25]:
# create a column of disabilities and combine them together

intassess['disabilities'] = intassess['demo_Disability - Primary'] +' '+ intassess['demo_Disabilities - Additional']
intassess['disabilities'] = intassess['disabilities'].astype(str)

In [26]:
# add in a column for yes/no if they have a disability

mentallist = ['Depression','Anxiety','PTSD','ADHD','Substance Abuse','Learning Disability']

for i in mentallist:
    intassess[i] = intassess['disabilities'].apply(lambda x: 'Yes' if i in x else 'No')

In [27]:
edurep = {
    'No High School Diploma / GED':'No HS Diploma - Under 12th grade education',
    'In High School':'HS Diploma',
    'High School Diploma':'HS Diploma',
    'Graduated - In High School':'HS Diploma',
    'Some College / Technical School (no certificate / diploma)':'Some post high school',
    'Masters (or beyond)':'Masters and above',
    'Certificate':'Certificate - 2 years or less'
}

intassess['demo_Education Level'] = intassess['demo_Education Level'].replace(edurep)

In [28]:
intassess['demo_Transportation'].value_counts()

demo_Transportation
Car               605
Public Transit    406
Para-transit       68
Ride               50
Other              36
School Bus         16
Unknown            10
Bike                9
Name: count, dtype: int64

In [29]:
transrep = {
    'Car':'Driving Myself',
    'Public Transit':'Public Transportation',
    'Para-transit':'Paratransit',
    'Unknown':'Other'
}

intassess['demo_Transportation'] = intassess['demo_Transportation'].replace(transrep)

#### Intake Assessment done

### Start of people index 

In [2]:
peeps = pd.read_csv(r'S:\Rehab\Apricot\Migration\for import\IndividualProfile import1 reviewed.csv')
peeps = peeps.drop(index=0)
peeps2 = peeps[['field_3436_first','field_3436_middle','field_3436_last','field_3195']].copy()
sep=','
peeps2['Individual'] = peeps['field_3436_last'].astype(str)+ sep + peeps['field_3436_first'].astype(str)
peeps2

,field_3436_first,field_3436_middle,field_3436_last,field_3195,Individual
1,Besharo,NaN,Abduwahab,8/30/2000,"Abduwahab,Besharo"
2,Noah,NaN,Abraham,4/2/1972,"Abraham,Noah"
3,Rebecca,NaN,Acosta,5/1/1979,"Acosta,Rebecca"
4,Jnean,NaN,Acteson,5/22/1980,"Acteson,Jnean"
5,Chris,NaN,Adair,11/26/1995,"Adair,Chris"
...,...,...,...,...,...
1365,NaN,NaN,NaN,NaN,"nan,nan"
1366,NaN,NaN,NaN,NaN,"nan,nan"
1367,NaN,NaN,NaN,NaN,"nan,nan"
1368,NaN,NaN,NaN,NaN,"nan,nan"


In [3]:
peeps3 = peeps2.drop_duplicates(subset=['field_3436_first','field_3436_middle','field_3436_last','field_3195'])
peeps3

,field_3436_first,field_3436_middle,field_3436_last,field_3195,Individual
1,Besharo,NaN,Abduwahab,8/30/2000,"Abduwahab,Besharo"
2,Noah,NaN,Abraham,4/2/1972,"Abraham,Noah"
3,Rebecca,NaN,Acosta,5/1/1979,"Acosta,Rebecca"
4,Jnean,NaN,Acteson,5/22/1980,"Acteson,Jnean"
5,Chris,NaN,Adair,11/26/1995,"Adair,Chris"
...,...,...,...,...,...
1358,Tevin,NaN,Zeno,2/21/1991,"Zeno,Tevin"
1359,Manuel,NaN,Zepeda,9/6/1980,"Zepeda,Manuel"
1360,Edwin,NaN,Zhang,10/30/1998,"Zhang,Edwin"
1361,Yuqing,NaN,Zhang,2/12/1978,"Zhang,Yuqing"


### Evaluation Merge

In [189]:
eval_list = ['eval - clampblock - unformatted', 
             'eval - custodial - unformatted', 
             'eval - finish - unformatted',  
             'eval - front desk - unformatted',
             'eval - grommets - unformatted',
             'eval - HBU - unformatted',
             'eval - machining cell - unformatted',
             'eval - Manufacturing office - unformatted',
             'eval - mechanical assembly - unformatted',
             'eval - mukilteo assembly - unformatted',
             'eval - mukilteo shipping and recieving - unformatted',
             'eval - mukilteo wires - unformatted',
             # 'eval - office skills - unformatted',
             'eval - QA - unformatted',
             'eval - sheet metal- unformatted',
             'eval - shipping and recieving - unformatted'#,
             # 'eval - soft skills - unformatted'
]


In [188]:
for i in eval_list:
    # establish the file path and file name
    path = 'S:/Rehab/Apricot/Migration/'+i+'.xlsx'
    
    # set the dataframe to a list to access later
    df = pd.read_excel(path)

    if 'EvaluationSoftSkillsID' in df.columns:
        print(i+' has ID')
    else:
        print(i+' no ID')

eval - clampblock - unformatted has ID
eval - custodial - unformatted has ID
eval - finish - unformatted has ID
eval - front desk - unformatted has ID
eval - grommets - unformatted has ID
eval - HBU - unformatted has ID
eval - machining cell - unformatted has ID
eval - Manufacturing office - unformatted has ID
eval - mechanical assembly - unformatted has ID
eval - mukilteo assembly - unformatted has ID
eval - mukilteo shipping and recieving - unformatted has ID
eval - mukilteo wires - unformatted has ID
eval - office skills - unformatted no ID
eval - QA - unformatted has ID
eval - sheet metal- unformatted has ID
eval - shipping and recieving - unformatted has ID


In [205]:
# set all evals to a list to iterat through later

eval_df = pd.DataFrame()

softskills = pd.read_excel('S:/Rehab/Apricot/Migration/eval - soft skills - unformatted.xlsx')
softskills = softskills.add_suffix('_soft')

for i in eval_list:
    
    # establish the file path and file name
    path = 'S:/Rehab/Apricot/Migration/'+i+'.xlsx'
    
    # set the dataframe to a list to access later
    df = pd.read_excel(path)

    # remove repetitive columns
    df = df.drop(columns=['Title','Item Type','Path','Folder Child Count','Created By','Content Type',])
    
    # remove the (###) from the individual name
    df.loc[:,'Individual'] = df['Individual'].replace(r'\b\d{1,4};#','',regex=True)

    # remove test case rows
    df = df[~df['Individual'].str.contains('test', case=False, na=False)].copy()

    # remove any text (within parethesis) to standardize names
    df.loc[:,'Individual'] = df['Individual'].replace(r'\(.*?\)','',regex=True).str.strip()

    df = df.merge(softskills,how='left',left_on=['EvaluationSoftSkillsID','Individual','Venue','Mentor'],right_on=['ID_soft','Individual_soft','Venue_soft','Mentor_soft'])
    
    eval_df = pd.concat([eval_df,df], ignore_index=True)

office = pd.read_excel('S:/Rehab/Apricot/Migration/eval - office skills - unformatted.xlsx')
eval_df = pd.concat([eval_df,office])

eval_df

,Venue,Mentor,Individual,Work Cell,Work Center,Grommets:,Modified,Packaging & Part Mark:,Assembly:,Assembly: Accurately faces the blocks the same way in the box,...,Workplace Appearance: Comments,Workplace Appearance: Hygiene and Grooming,Workplace Performance: Assignments are Completed in their Entirety,Workplace Performance: Comments,Workplace Performance: Daily Assignments are Accurate,Workplace Performance: Follows Directions and Rules of the Classroom/Orion,Workplace Performance: Progress on Tutorials,"Workplace Performance: Weekly Logs are Turned In - Neat, Completed and Detailed",Item Type,Path
0,Auburn - Manufacturing Program,Greg Ford,"Reed, Daniel",Clamp Block,Packaging,NaN,2018-09-04 23:21:30,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Auburn - Manufacturing Program,Layne Norris,"Smyth, Rachel",Clamp Block,Assembly,NaN,2018-10-08 00:00:54,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Auburn - Manufacturing Program,Layne Norris,"Smyth, Rachel",Clamp Block,Assembly,NaN,2018-10-15 00:00:43,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Auburn - Manufacturing Program,Jeff Wold,"Cawley, Jimmy",Clamp Block,Assembly,NaN,2018-10-29 00:01:20,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Auburn - Manufacturing Program,Jeff Wold,"Cawley, Jimmy",Clamp Block,Assembly,NaN,2018-11-05 00:01:04,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,NaN,Jeri Lucas Audit,"1760;#Lewis, Marian",NaN,NaN,NaN,2024-06-17 16:30:31,NaN,NaN,NaN,...,NaN,Proficient,Yes,Marian let CI know when she left late for brea...,Yes,Yes,Average,Yes,Item,Lists/EvaluationOfficeSkills
422,NaN,Jeri Lucas Audit,"1749;#Ferguson, Sylvie",NaN,NaN,NaN,2024-06-21 14:51:27,NaN,NaN,NaN,...,NaN,Proficient,Yes,•\tDuring the second week of the evaluation pe...,Yes,Yes,Slower than Average,Yes,Item,Lists/EvaluationOfficeSkills
423,NaN,Jeri Lucas Audit,"1773;#Sam, Kathy",NaN,NaN,NaN,2024-06-21 15:25:02,NaN,NaN,NaN,...,Kathy always presents well.,Proficient,Yes,Kathy sometimes stays after to work. She is co...,Yes,Yes,Faster than Average,Yes,Item,Lists/EvaluationOfficeSkills
424,NaN,Jeri Lucas Audit,"1752;#Reasor, Stephanie",NaN,NaN,NaN,2024-06-25 13:05:47,NaN,NaN,NaN,...,NaN,Exemplary,Yes,•\t6/7 – Front was complete and had excellent ...,Yes,Yes,Average,Yes,Item,Lists/EvaluationOfficeSkills


In [213]:
eval_df2 = eval_df.copy()
eval_df2 = eval_df2.reset_index()

In [216]:
# remove space on the individual to allow for merging
eval_df2.loc[:,'Individual'] = eval_df2['Individual'].str.replace(', ',',')

# remove test case rows
eval_df2 = eval_df2[~eval_df2['Individual'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
eval_df2['Individual'] = eval_df2['Individual'].replace(r'\(.*?\)','',regex=True).str.strip()

eval_df2['Individual'] = eval_df2['Individual'].replace(r'\".*?\"','',regex=True).str.strip()

eval_df2['Final Due Date'] = eval_df2['Due Date_soft'].combine_first(eval_df2['Evaluation Due Date'])
eval_df2 = eval_df2.drop(columns=['Evaluation Due Date','Due Date_soft'])

In [217]:
# change evaluation text to standardized text
eval_df2 = eval_df2.replace('Exemplary','Proficient',regex=True)
eval_df2 = eval_df2.replace('Unsatisfactory','Not Competitive',regex=True)
eval_df2 = eval_df2.replace('Inconsistent','Not Competitive',regex=True)
eval_df2 = eval_df2.replace('Did Not Train N/A','N/A',regex=True)
eval_df2 = eval_df2.replace('Did Not Train (N/A)','N/A',regex=False)
eval_df2

,index,Venue,Mentor,Individual,Work Cell,Work Center,Grommets:,Modified,Packaging & Part Mark:,Assembly:,...,Workplace Appearance: Hygiene and Grooming,Workplace Performance: Assignments are Completed in their Entirety,Workplace Performance: Comments,Workplace Performance: Daily Assignments are Accurate,Workplace Performance: Follows Directions and Rules of the Classroom/Orion,Workplace Performance: Progress on Tutorials,"Workplace Performance: Weekly Logs are Turned In - Neat, Completed and Detailed",Item Type,Path,Final Due Date
0,0,Auburn - Manufacturing Program,Greg Ford,"Reed,Daniel",Clamp Block,Packaging,NaN,2018-09-04 23:21:30,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,1,Auburn - Manufacturing Program,Layne Norris,"Smyth,Rachel",Clamp Block,Assembly,NaN,2018-10-08 00:00:54,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,2,Auburn - Manufacturing Program,Layne Norris,"Smyth,Rachel",Clamp Block,Assembly,NaN,2018-10-15 00:00:43,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,3,Auburn - Manufacturing Program,Jeff Wold,"Cawley,Jimmy",Clamp Block,Assembly,NaN,2018-10-29 00:01:20,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,4,Auburn - Manufacturing Program,Jeff Wold,"Cawley,Jimmy",Clamp Block,Assembly,NaN,2018-11-05 00:01:04,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12181,421,NaN,Jeri Lucas Audit,"1760;#Lewis,Marian",NaN,NaN,NaN,2024-06-17 16:30:31,NaN,NaN,...,Proficient,Yes,Marian let CI know when she left late for brea...,Yes,Yes,Average,Yes,Item,Lists/EvaluationOfficeSkills,2024-06-20
12182,422,NaN,Jeri Lucas Audit,"1749;#Ferguson,Sylvie",NaN,NaN,NaN,2024-06-21 14:51:27,NaN,NaN,...,Proficient,Yes,•\tDuring the second week of the evaluation pe...,Yes,Yes,Slower than Average,Yes,Item,Lists/EvaluationOfficeSkills,2024-06-25
12183,423,NaN,Jeri Lucas Audit,"1773;#Sam,Kathy",NaN,NaN,NaN,2024-06-21 15:25:02,NaN,NaN,...,Proficient,Yes,Kathy sometimes stays after to work. She is co...,Yes,Yes,Faster than Average,Yes,Item,Lists/EvaluationOfficeSkills,2024-06-25
12184,424,NaN,Jeri Lucas Audit,"1752;#Reasor,Stephanie",NaN,NaN,NaN,2024-06-25 13:05:47,NaN,NaN,...,Proficient,Yes,•\t6/7 – Front was complete and had excellent ...,Yes,Yes,Average,Yes,Item,Lists/EvaluationOfficeSkills,2024-06-25


In [218]:
peepsmerge = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'people_Full Name'
]].copy()

peepsmerge['people_Full Name'] = peepsmerge['people_Full Name'].str.replace(', ',',')
peepsmerge['demo_Date of Birth'] = peepsmerge['demo_Date of Birth'].fillna('1/1/1800')

In [219]:
eval_df2 = eval_df2.merge(peepsmerge, how='right',left_on='Individual',right_on='people_Full Name')
eval_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5639 entries, 0 to 5638
Columns: 1014 entries, index to people_Full Name
dtypes: datetime64[ns](9), float64(25), object(980)
memory usage: 43.6+ MB


In [220]:
eval_df2.to_excel(r'S:\Rehab\Apricot\Migration\exports\eval check1.xlsx')

In [221]:
eval_df2['location code'] = np.where(eval_df2['Venue'].str.contains('Auburn'),1174,2870)
eval_df2

,index,Venue,Mentor,Individual,Work Cell,Work Center,Grommets:,Modified,Packaging & Part Mark:,Assembly:,...,Workplace Performance: Progress on Tutorials,"Workplace Performance: Weekly Logs are Turned In - Neat, Completed and Detailed",Item Type,Path,Final Due Date,people_First Name,people_Last Name,demo_Date of Birth,people_Full Name,location code
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,Keith,Shuman,1970-05-22 00:00:00,"Shuman,Keith",1174
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,Melissa,Waltjen,1977-10-06 00:00:00,"Waltjen,Melissa",1174
2,2364.0,Auburn - Manufacturing Program,Courtney Aubert,"Lockhart,Rebekah",Clamp Block,Assembly,0.0,2023-01-17 07:41:47,True,True,...,NaN,NaN,NaN,NaN,2023-01-19,Rebekah,Lockhart,2002-03-26 00:00:00,"Lockhart,Rebekah",1174
3,2372.0,Auburn - Manufacturing Program,Courtney Aubert,"Lockhart,Rebekah",Clamp Block,Assembly,0.0,2023-01-24 09:28:16,False,True,...,NaN,NaN,NaN,NaN,2023-01-26,Rebekah,Lockhart,2002-03-26 00:00:00,"Lockhart,Rebekah",1174
4,2382.0,Auburn - Manufacturing Program,Courtney Aubert,"Lockhart,Rebekah",Clamp Block,Assembly,0.0,2023-01-31 13:12:04,False,True,...,NaN,NaN,NaN,NaN,2023-02-02,Rebekah,Lockhart,2002-03-26 00:00:00,"Lockhart,Rebekah",1174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5634,9275.0,Mukilteo - Manufacturing Program,Anako Lison,"Johnston,George",Wire,Assembly,NaN,2023-09-06 13:55:13,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,George,Johnston,1975-08-14 00:00:00,"Johnston,George",2870
5635,9276.0,Mukilteo - Manufacturing Program,Anako Lison,"Johnston,George",Wire,Assembly,NaN,2023-09-18 14:34:33,NaN,NaN,...,NaN,NaN,NaN,NaN,2023-09-21,George,Johnston,1975-08-14 00:00:00,"Johnston,George",2870
5636,9280.0,Mukilteo - Manufacturing Program,Anako Lison,"Johnston,George",Wire,Assembly,NaN,2023-09-26 06:52:34,NaN,NaN,...,NaN,NaN,NaN,NaN,2023-09-28,George,Johnston,1975-08-14 00:00:00,"Johnston,George",2870
5637,9296.0,Mukilteo - Manufacturing Program,Anako Lison,"Johnston,George",Wire,Assembly,NaN,2023-10-09 13:58:51,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,George,Johnston,1975-08-14 00:00:00,"Johnston,George",2870


In [222]:
eval_df3 = eval_df2.copy()

In [223]:
eval_df3['Work Cell'] = np.where(eval_df3['Venue'].str.contains('Auburn'),'Auburn '+eval_df3['Work Cell'],'Mukilteo '+eval_df3['Work Cell'])

In [224]:
# change work cell names to match apricot names

eval_df3['Work Cell'] = eval_df3['Work Cell'].str.replace('Auburn Clamp Block','Auburn Clamp Block Issuing')
eval_df3['Work Cell'] = eval_df3['Work Cell'].str.replace('Auburn Finish','Auburn Racking / Unracking; Paint Prep')
### eval_df3['Work Cell'] = eval_df3['Work Cell'].str.replace('Auburn Sheet Metal','')
### eval_df3['Work Cell'] = eval_df3['Work Cell'].str.replace('Auburn Shipping and Recieving','Auburn Shipping')
eval_df3['Work Cell'] = eval_df3['Work Cell'].str.replace('Mukilteo Wire','Mukilteo Wire Assembly')
### eval_df3['Work Cell'] = eval_df3['Work Cell'].str.replace('Auburn Machining','')
### eval_df3['Work Cell'] = eval_df3['Work Cell'].str.replace('Auburn Office (Manufacturing)','')
### eval_df3['Work Cell'] = eval_df3['Work Cell'].str.replace('Mukilteo Assembly','')
### eval_df3['Work Cell'] = eval_df3['Work Cell'].str.replace('Auburn Contact Center','Auburn Clerical Customer Service')
eval_df3['Work Cell'] = eval_df3['Work Cell'].str.replace('Mukilteo Shipping and Receiving','Mukilteo Shipping & Receiving')
eval_df3['Work Cell'] = eval_df3['Work Cell'].str.replace('Auburn Quality Assurance','Auburn QA')
### eval_df3['Work Cell'] = eval_df3['Work Cell'].str.replace('Mukilteo Machining','')

eval_df3['Work Cell'].value_counts()

Work Cell
Auburn Clamp Block Issuing                1423
Auburn Sheet Metal                         673
Mukilteo Wire Assembly                     575
Auburn Racking / Unracking; Paint Prep     563
Auburn Mills                               475
Auburn Mechanical Assembly                 396
Auburn Shipping and Receiving              247
Auburn Machining                           211
Mukilteo Assembly                          204
Mukilteo Shipping & Receiving              135
Auburn Office (Manufacturing)              122
Mukilteo Mechanical Assembly               116
Auburn HBU                                 108
Auburn Custodial                            65
Auburn Contact Center                       56
Auburn QA                                   29
Mukilteo Machining                           8
Auburn Assembly                              2
Name: count, dtype: int64

In [226]:
eval_df3.to_excel(r'S:\Rehab\Apricot\Migration\exports\eval merge 5.xlsx')

In [229]:
for i in eval_df3.columns:
    print(i)

index
Venue
Mentor
Individual
Work Cell
Work Center
Grommets:
Modified
Packaging & Part Mark:
Assembly:
Assembly: Accurately faces the blocks the same way in the box
Assembly: Accurately measures screws and spacers using calipers
Assembly: Accurately verifies parts with paperwork
Assembly: Comments for Basic Manufacturing Task List SectionFilter
Assembly: Comments on Advanced Manufacturing Tasks List SectionFilter
Assembly: Demonstrates ability to locate correct part when pulling standards
Assembly: Demonstrates ability to place jobs on correct spot of warehouse shelf
Assembly: Demonstrating the ability to properly restock inventory to correct area
Assembly: Follows posted assembly instructions accurately
Assembly: Independent ability to accurately read the details of a blueprint
Assembly: Independent ability to build a clamp block correctly
Assembly: Independent ability to locate blueprint for the job
Assembly: Properly lays out the blocks for building
Assembly: Replacing paperwork in

In [228]:
eval_df3 = eval_df3.replace(';#','|',regex=True)

In [232]:
evaldf = eval_df3[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'Mentor',
    'Work Cell',
    'For the Week of:_soft',
    'Final Due Date',
    'Attendance (Please note number of full days missed. If no days were missed, fill in 0)_soft',
    'Punctuality_soft',
    'Demeanor & Attitude_soft',
    'Workplace Appearance & Attire_soft',
    'Grooming & Hygiene_soft',
    'Cooperation & Teamwork_soft',
    'Interpersonal Communication Style_soft',
    'Response to Feedback / Instruction_soft',
    'Organization_soft',
    'Work Effort & Initiative_soft',
    'Work Tolerance & Stamina_soft',
    'Learning Style_soft',
    'Follows all Safety Rules & Policies?_soft',
    'Note Any Inappropriate Behaviors Observed during Evaluation Period (if no inappropriate behaviors observed, mark that choice)_soft',
    "Are Person's Soft Skills / General Employment Traits Competitive?_soft",
    'Soft Skills Comments_soft',
    # general employment
    'General Employment: Keeps a clean & organized workspace in order to reduce FOD',
    # Utilizes pictures/blueprint to produce clamp block assembly
    'Assembly: Independent ability to accurately read the details of a blueprint',
    # Lays out blocks for building correctly
    'Assembly: Properly lays out the blocks for building',
    # Places jobs on correct location on shelves|Returns excess pieces to inventory
    'Assembly: Demonstrates ability to place jobs on correct spot of warehouse shelf',
    #Sets up for part mark, including part mark location and selecting proper size for part mark
    'Packaging & Part Mark: Proper set up for part mark',
    #Assesses part mark for quality and redo as needed|Applies overcoat
    'Packaging & Part Mark: Applies overcoat precisely, with desired thickness',
    #Judges size of bag to package parts|Packages multiple parts that will meet inspection standards|Operates bag sealer
    'Packaging & Part Mark: Correct operation of bag sealer',
    #Removes labels from boxes before use; Creates labels for job boxes
    'Clamp Block Inventory: Creates labels for job boxes',
    #When pulling parts from inventory - Identifies and verifies part #'s and lot #'s using the Job Material Queue
    'Clamp Block Inventory: For each part pulled from inventory, records correct lot # and qty to work order',
    #Issues parts with computer using Job Material Queue and Process by ID
    'Clamp Block Inventory: Issues parts with computer using Inventory Transaction in VMFG',
    #Operates the chop saw
    'CHAKU: Operation of saw',
    #Operates the punch press
    'CHAKU: Operation of punch press',
    #Operates the countersink machine
    'CHAKU: Operation of countersink',
    #Operates the flat finisher (deburr)
    'CHAKU: Operation of deburr',
    #Operates the CNC Mill
    'CHAKU: Operation of CNC Mill',
    #Utilizes work order to differentiate between grommet shape: (ex. F material from J or H material)
    'Grommets: Utilizes work order to differentiate between grommet shape',
    #Kiss cuts washers
    'Grommets: Demonstrates ability to kiss cut washers',
    #Wraps prepped grommet onto centers; loads into jig
    'Grommets: Independent ability to accurately load a jig',
    #Assembles a grommet|Identifies holes in grommet|Repairs holes in grommet|Builds at average rate of 10-15 parts per hour
    'Grommets: Ability to identify holes in grommet',
    #Cleans offices, corridors, meeting rooms, and common spaces|Dusts|Cleans restrooms|Uses auto scrubber to clean floor|Vacuums carpeted floors|Washes windows and doors|Takes out trash and recycling
    'Custodial: Follows proper restroom cleaning sequence that meets quality standards',
    #Finds SDS information|Works with chemicals and mixes cleaning solutions
    'Custodial: Independent ability to properly mix cleaning solutions',
    #Checks appliances and light fixtures, changes light bulbs as needed
    'Custodial: Independently changes light bulbs as needed',
    #Identifies each job and work order number|Identifies the process for each part/job|Creates identification tags for jobs travelling through processes
    'Racking / Unracking; Paint Prep: Independent ability to obtain process information from work order traveler',
    #Masks parts using tape and/or plugs|Removes masking tape and/or plugs|Lays out parts on paint screen
    'Racking / Unracking; Paint Prep: Able to properly layout parts on paint screen',
    #Pre-cleans parts by removing ink or excessive oils
    'Racking / Unracking; Paint Prep: Pre-cleans parts by removing ink or excessive oils',
    #Enters part mark data on the A200 or A320i part mark machine|Places part mark in correct location
    'Part Mark: Independently utilize the A320i vendor options for data entry of part mark',
    #Uploads part mark onto a dauber sheet|Applies part mark using ink dauber
    'Part Mark: Application of part marks by using ink dauber',
    #Ability to correctly judge what size bag to package parts in
    'Packaging: Ability to correctly judge what size bag to package parts in',
    #Greets visitors and assists with purpose of visit|Answers telephone and direct calls|Handles customer questions or issues requiring follow up
    'General Admin Clerk: Provides excellent customer service',
    #Checks daily mail and delivers
    'General Admin Clerk: Independent with the mail run (grabs mail from mailbox, sorts and delivers to appropriate parties)',
    #Scans documents and saves files to the appropriate location on computer|Uses photocopy machine|Creates paperwork packets
    'General Admin Clerk: Saving documents/files in the appropriate location on the computer',
    #Completes over coat operations
    'HBU: Independently completes over coating operations',
    #Completes job shadow of an assembly technician
    'HBU: Completion of job shadow of an assembly technician',
    #Identifies call outs, flags notes and locations for parts on blueprints
    'HBU - assembly: Recognizes views and orientation of parts on blueprints',
    #Builds parts while being shadowed by mentor/technician
    'HBU - assembly: Accurately builds parts while being shadowed by mentor/technician',
    #Receives certifications (note in comments which certifications)
    'HBU - assembly: Receives certifications (note in comments which certifications)',
    #Loads and unloads tooling from holders and spindle
    'Mills: Independent ability to load and unload tools from holders and spindle',
    #Identifies burrs|Deburrs parts
    'Mills: Independent ability to deburr parts correctly',
    #Understands and explains concept of X, Y, Z coordinate system
    'Mills: Able to understand and explain concept of X, Y, Z coordinate system',
    #Sets up tools|Breaks down returned tools
    'Mills Tool Crib: Accurately sets up tools',
    #Identifies burrs, damage and flaws
    'Vibraburr: Independent ability to identify a burr',
    #Runs Vibrodyne machine|Runs Roto machine|Runs Burr King machine|Runs Cement Mixer machine
    'Vibraburr: Independent ability to run Vibrodyne machine',
    #Identifies when machine cleaning is needed (ex. change water)
    'Vibraburr: Independent ability to clean machine as needed (ex. change water)',
    #Identifies and stamps military export controlled work orders|Identifies work orders with FAI (First Article Inspection) operation|Sorts, staples, and confirms documents are in order
    'Commercial Work Order Processing: IDs and stamps military export controlled work orders',
    #Verifies that revision levels listed on work order are current (sources of reference may include Boeing EID and customer purchase order notes)
    'Commercial Work Order Processing: Verifies revision levels listed on work order are current',
    #Opens estimating tab in Epicor and inputs data into correct fields|Opens and reviews estimating quote log in Epicor|Logs onto Exostar/Boeing portal|Opens EID (Engineering Information Delivery) page, selects & downloads correct drawings on Boeing Portal
    'Estimating Training Evaluation: Correctly opens EID page, selects & downloads correct drawings on Boeing Portal',
    #Identifies drill bits and countersink size
    'Mechanical Assembly: Independent, accurate usage of precision measuring equipment',
    #Operates the countersink machine|Sets up the countersink machine
    'Mechanical Assembly: Independent operation of countersink',
    #Operates the riveter/squeeze machine|Sets up the riveter/squeeze machine
    'Mechanical Assembly: Independent operation of riveter / squeeze',
    #Operates the drill press|Sets up the drill press
    'Mechanical Assembly: Independent operation of drill press',
    #Utilizes procedure for bonding tasks
    'Mechanical Assembly : Utilizes proper procedure for bonding tasks',
    #Assembles wiggle plates
    'Mechanical Assembly : Assembly of wiggle plates',
    #Sets up minor jobs (nut plate, wiggle plate, countersink machine)
    'Mechanical Assembly : Independent set-up of minor jobs (nut plate, wiggle plate, countersink machine)',
    #Determines drawings required
    'QA: Ability to determine drawings required and current revisions',
    #Operates the blanking press
    'Blanking Press: Acceptable operation of blanking press',
    #Locates tooling for each job|Sets up and takes down tooling|Puts tool away
    'Blanking Press: Ability to locate proper tooling for each job',
    #Identifies burrs|Deburrs parts
    'Blanking Press: Correctly deburrs parts',
    #Lubricates tools
    'Blanking Press: Proper lubrication of material',
    #Operates the Amada
    'Amada: Proper operation of Amada',
    #Operates brake form in Mylar cutting
    'Brake Form: Operation of brake form in Mylar cutting',
    #Sets up and centers Mylar cutting tool
    'Brake Form: Independent ability to set up and center Mylar cutting tool',
    #Identifies burrs|Uses "Timesaver"|Deburrs parts
    'Brake Form: Correctly deburrs parts',
    #Identifies and locates correct material for each job|Loads and unloads material|Performs computer steps to start machine
    'Routers: Independent ability to identify and locate correct material for each job',
    #Identifies and locates tooling for each job|Sets up and changes tooling plates
    'Routers: Independent ability to setup and change tooling plates',
    #Identifies burrs|Deburrs parts
    'Routers: Correctly deburrs parts',
    #Differentiates between Boeing and Commercial work orders|Locates tools and templates used for Boeing work orders|Locates, prints, and verifies certification (cert)
    'Shears: Independent ability to differentiate between Boeing and Commercial work orders',
    #Sets up the machine
    'Shears: Independent ability to set up machine',
    #Operates the shear
    'Shears: Accurate operation of Shear',
    #Understands machine operation and computer inputs|Differentiates between Automatic Start and Manual Start procedure and when each would be used
    'Waterjet: Understands and demonstrates proper machine operation and computer inputs',
    #Operates the water jet|Sets up the water jet
    'Waterjet: Independent setup and machine operation of Water Jet',
    #Operates the grind wheel|Uses deburr hand tools including scotch brite or sand paper|Cleans parts thoroughly with alcohol
    'Deburr: Cleans parts thoroughly with alcohol',
    #Counts and verifies part mark and paperwork prior to packaging outgoing shipment
    'Commercial Shipping: Sort, segregate and prioritize verified work order by due date and shipping date',
    #Visually inspects incoming product for damage, re-package product according to Orion guidelines prior to moving on to next area
    'Receiving: Independence with receiving materials and performing tasks in Receiving work area',
    #Operates the pallet jack
    'Receiving: Independent utilization of pallet jack',
    #Visually inspects incoming product for damage, re-packages product per Orion guidelines prior to moving into inventory
    "Inventory: for parts pulled out of inventory, verifies correct part #'s on work order",
    #Retrieves product from inventory; moves to next location based on work order or pull ticket
    'Inventory: Independently finds correct location when picking parts from inventory',
    #Counts product in each specified bin location during cycle count activity
    'Inventory: Ensures that counts in inventory match what is posted for cycle counts',
    #Applies bond and adheres waffle or strips to orifice|Places weights on waffle orifices
    'Bond: Application of bond and adherence of waffle or strips to orifice',
    #Calculates cure time for bond
    'Bond: Independent, accurate calculation of cure time for bond',
    #Creates internal work order
    'Shipping & Receiving: Independent creation of internal work order',
    #Enters information into Boeing Portal
    'Shipping & Receiving: Accurate entering of information into Boeing Portal',
    #Utilizes and navigates computer applications
    'Manufacturing Offices: Independent ability to utilize and navigate computer applications',
    #Creates Purchase Orders|Creates quotes/estimates
    'Manufacturing Offices: Independently creates Purchase Orders',
    #Gathers data from internal lists and makes determinations based on information
    'Manufacturing Offices: Gathers correct data from internal lists and makes correct decisions based on this informaton',
    #Visually inspects and measures parts
    'Wire Assembly: Independent visual inspection and measurement of each part',
    #Ensures that counts per bag match what is posted on front of boxes|Properly runs 2 bag system
    'Inventory: Proper run of 2 bag system',
    #Inspects and verifies wire part marks/lengths|Completes job (operations closed, paperwork stamped, etc.)
    'Wire QA: Accurate inspection and verification of wire lengths',
    #Loads and threads wire through de-reeler, unracks wire spool and returns to its correct location
    'Laser Marking: Independently and accurately loads and threads wire through de-reeler',
    #Follows procedure to access laser marking information on flash drive/computer
    'Laser Marking: Follows proper procedure to access laser marking information on flash drive/computer',
    'location code'
]]

In [245]:
evalreplist = ['Keeps a clean & organized workspace in order to reduce FOD',
'General Employment: Keeps a clean & organized workspace in order to reduce FOD',
'Utilizes pictures/blueprint to produce clamp block assembly',
'Assembly: Independent ability to accurately read the details of a blueprint',
'Lays out blocks for building correctly',
'Assembly: Properly lays out the blocks for building',
'Places jobs on correct location on shelves|Returns excess pieces to inventory',
'Assembly: Demonstrates ability to place jobs on correct spot of warehouse shelf',
'Sets up for part mark, including part mark location and selecting proper size for part mark',
'Packaging & Part Mark: Proper set up for part mark',
'Assesses part mark for quality and redo as needed|Applies overcoat',
'Packaging & Part Mark: Applies overcoat precisely, with desired thickness',
'Judges size of bag to package parts|Packages multiple parts that will meet inspection standards|Operates bag sealer',
'Packaging & Part Mark: Correct operation of bag sealer',
'Removes labels from boxes before use; Creates labels for job boxes',
'Clamp Block Inventory: Creates labels for job boxes',
"When pulling parts from inventory - Identifies and verifies part #'s and lot #'s using the Job Material Queue",
'Clamp Block Inventory: For each part pulled from inventory, records correct lot # and qty to work order',
'Issues parts with computer using Job Material Queue and Process by ID',
'Clamp Block Inventory: Issues parts with computer using Inventory Transaction in VMFG',
'Operates the chop saw',
'CHAKU: Operation of saw',
'Operates the punch press',
'CHAKU: Operation of punch press',
'Operates the countersink machine',
'CHAKU: Operation of countersink',
'Operates the flat finisher (deburr)',
'CHAKU: Operation of deburr',
'Operates the CNC Mill',
'CHAKU: Operation of CNC Mill',
'Utilizes work order to differentiate between grommet shape: (ex. F material from J or H material)',
'Grommets: Utilizes work order to differentiate between grommet shape',
'Kiss cuts washers',
'Grommets: Demonstrates ability to kiss cut washers',
'Wraps prepped grommet onto centers; loads into jig',
'Grommets: Independent ability to accurately load a jig',
'Assembles a grommet|Identifies holes in grommet|Repairs holes in grommet|Builds at average rate of 10-15 parts per hour',
'Grommets: Ability to identify holes in grommet',
'Cleans offices, corridors, meeting rooms, and common spaces|Dusts|Cleans restrooms|Uses auto scrubber to clean floor|Vacuums carpeted floors|Washes windows and doors|Takes out trash and recycling',
'Custodial: Follows proper restroom cleaning sequence that meets quality standards',
'Finds SDS information|Works with chemicals and mixes cleaning solutions',
'Custodial: Independent ability to properly mix cleaning solutions',
'Checks appliances and light fixtures, changes light bulbs as needed',
'Custodial: Independently changes light bulbs as needed',
'Identifies each job and work order number|Identifies the process for each part/job|Creates identification tags for jobs travelling through processes',
'Racking / Unracking; Paint Prep: Independent ability to obtain process information from work order traveler',
'Masks parts using tape and/or plugs|Removes masking tape and/or plugs|Lays out parts on paint screen',
'Racking / Unracking; Paint Prep: Able to properly layout parts on paint screen',
'Pre-cleans parts by removing ink or excessive oils',
'Racking / Unracking; Paint Prep: Pre-cleans parts by removing ink or excessive oils',
'Enters part mark data on the A200 or A320i part mark machine|Places part mark in correct location',
'Part Mark: Independently utilize the A320i vendor options for data entry of part mark',
'Uploads part mark onto a dauber sheet|Applies part mark using ink dauber',
'Part Mark: Application of part marks by using ink dauber',
'Ability to correctly judge what size bag to package parts in',
'Packaging: Ability to correctly judge what size bag to package parts in',
'Greets visitors and assists with purpose of visit|Answers telephone and direct calls|Handles customer questions or issues requiring follow up',
'General Admin Clerk: Provides excellent customer service',
'Checks daily mail and delivers',
'General Admin Clerk: Independent with the mail run (grabs mail from mailbox, sorts and delivers to appropriate parties)',
'Scans documents and saves files to the appropriate location on computer|Uses photocopy machine|Creates paperwork packets',
'General Admin Clerk: Saving documents/files in the appropriate location on the computer',
'Completes over coat operations',
'HBU: Independently completes over coating operations',
'Completes job shadow of an assembly technician',
'HBU: Completion of job shadow of an assembly technician',
'Identifies call outs, flags notes and locations for parts on blueprints',
'HBU - assembly: Recognizes views and orientation of parts on blueprints',
'Builds parts while being shadowed by mentor/technician',
'HBU - assembly: Accurately builds parts while being shadowed by mentor/technician',
'Receives certifications (note in comments which certifications)',
'HBU - assembly: Receives certifications (note in comments which certifications)',
'Loads and unloads tooling from holders and spindle',
'Mills: Independent ability to load and unload tools from holders and spindle',
'Identifies burrs|Deburrs parts',
'Mills: Independent ability to deburr parts correctly',
'Understands and explains concept of X, Y, Z coordinate system',
'Mills: Able to understand and explain concept of X, Y, Z coordinate system',
'Sets up tools|Breaks down returned tools',
'Mills Tool Crib: Accurately sets up tools',
'Identifies burrs, damage and flaws',
'Vibraburr: Independent ability to identify a burr',
'Runs Vibrodyne machine|Runs Roto machine|Runs Burr King machine|Runs Cement Mixer machine',
'Vibraburr: Independent ability to run Vibrodyne machine',
'Identifies when machine cleaning is needed (ex. change water)',
'Vibraburr: Independent ability to clean machine as needed (ex. change water)',
'Identifies and stamps military export controlled work orders|Identifies work orders with FAI (First Article Inspection) operation|Sorts, staples, and confirms documents are in order',
'Commercial Work Order Processing: IDs and stamps military export controlled work orders',
'Verifies that revision levels listed on work order are current (sources of reference may include Boeing EID and customer purchase order notes)',
'Commercial Work Order Processing: Verifies revision levels listed on work order are current',
'Opens estimating tab in Epicor and inputs data into correct fields|Opens and reviews estimating quote log in Epicor|Logs onto Exostar/Boeing portal|Opens EID (Engineering Information Delivery) page, selects & downloads correct drawings on Boeing Portal',
'Estimating Training Evaluation: Correctly opens EID page, selects & downloads correct drawings on Boeing Portal',
'Identifies drill bits and countersink size',
'Mechanical Assembly: Independent, accurate usage of precision measuring equipment',
'Operates the countersink machine|Sets up the countersink machine',
'Mechanical Assembly: Independent operation of countersink',
'Operates the riveter/squeeze machine|Sets up the riveter/squeeze machine',
'Mechanical Assembly: Independent operation of riveter / squeeze',
'Operates the drill press|Sets up the drill press',
'Mechanical Assembly: Independent operation of drill press',
'Utilizes procedure for bonding tasks',
'Mechanical Assembly : Utilizes proper procedure for bonding tasks',
'Assembles wiggle plates',
'Mechanical Assembly : Assembly of wiggle plates',
'Sets up minor jobs (nut plate, wiggle plate, countersink machine)',
'Mechanical Assembly : Independent set-up of minor jobs (nut plate, wiggle plate, countersink machine)',
'Determines drawings required',
'QA: Ability to determine drawings required and current revisions',
'Operates the blanking press',
'Blanking Press: Acceptable operation of blanking press',
'Locates tooling for each job|Sets up and takes down tooling|Puts tool away',
'Blanking Press: Ability to locate proper tooling for each job',
'Identifies burrs|Deburrs parts',
'Blanking Press: Correctly deburrs parts',
'Lubricates tools',
'Blanking Press: Proper lubrication of material',
'Operates the Amada',
'Amada: Proper operation of Amada',
'Operates brake form in Mylar cutting',
'Brake Form: Operation of brake form in Mylar cutting',
'Sets up and centers Mylar cutting tool',
'Brake Form: Independent ability to set up and center Mylar cutting tool',
'Identifies burrs|Uses "Timesaver"|Deburrs parts',
'Brake Form: Correctly deburrs parts',
'Identifies and locates correct material for each job|Loads and unloads material|Performs computer steps to start machine',
'Routers: Independent ability to identify and locate correct material for each job',
'Identifies and locates tooling for each job|Sets up and changes tooling plates',
'Routers: Independent ability to setup and change tooling plates',
'Identifies burrs|Deburrs parts',
'Routers: Correctly deburrs parts',
'Differentiates between Boeing and Commercial work orders|Locates tools and templates used for Boeing work orders|Locates, prints, and verifies certification (cert)',
'Shears: Independent ability to differentiate between Boeing and Commercial work orders',
'Sets up the machine',
'Shears: Independent ability to set up machine',
'Operates the shear',
'Shears: Accurate operation of Shear',
'Understands machine operation and computer inputs|Differentiates between Automatic Start and Manual Start procedure and when each would be used',
'Waterjet: Understands and demonstrates proper machine operation and computer inputs',
'Operates the water jet|Sets up the water jet',
'Waterjet: Independent setup and machine operation of Water Jet',
'Operates the grind wheel|Uses deburr hand tools including scotch brite or sand paper|Cleans parts thoroughly with alcohol',
'Deburr: Cleans parts thoroughly with alcohol',
'Counts and verifies part mark and paperwork prior to packaging outgoing shipment',
'Commercial Shipping: Sort, segregate and prioritize verified work order by due date and shipping date',
'Visually inspects incoming product for damage, re-package product according to Orion guidelines prior to moving on to next area',
'Receiving: Independence with receiving materials and performing tasks in Receiving work area',
'Operates the pallet jack',
'Receiving: Independent utilization of pallet jack',
'Visually inspects incoming product for damage, re-packages product per Orion guidelines prior to moving into inventory',
"Inventory: for parts pulled out of inventory, verifies correct part #'s on work order",
'Retrieves product from inventory; moves to next location based on work order or pull ticket',
'Inventory: Independently finds correct location when picking parts from inventory',
'Counts product in each specified bin location during cycle count activity',
'Inventory: Ensures that counts in inventory match what is posted for cycle counts',
'Applies bond and adheres waffle or strips to orifice|Places weights on waffle orifices',
'Bond: Application of bond and adherence of waffle or strips to orifice',
'Calculates cure time for bond',
'Bond: Independent, accurate calculation of cure time for bond',
'Creates internal work order',
'Shipping & Receiving: Independent creation of internal work order',
'Enters information into Boeing Portal',
'Shipping & Receiving: Accurate entering of information into Boeing Portal',
'Utilizes and navigates computer applications',
'Manufacturing Offices: Independent ability to utilize and navigate computer applications',
'Creates Purchase Orders|Creates quotes/estimates',
'Manufacturing Offices: Independently creates Purchase Orders',
'Gathers data from internal lists and makes determinations based on information',
'Manufacturing Offices: Gathers correct data from internal lists and makes correct decisions based on this informaton',
'Visually inspects and measures parts',
'Wire Assembly: Independent visual inspection and measurement of each part',
'Ensures that counts per bag match what is posted on front of boxes|Properly runs 2 bag system',
'Inventory: Proper run of 2 bag system',
'Inspects and verifies wire part marks/lengths|Completes job (operations closed, paperwork stamped, etc.)',
'Wire QA: Accurate inspection and verification of wire lengths',
'Loads and threads wire through de-reeler, unracks wire spool and returns to its correct location',
'Laser Marking: Independently and accurately loads and threads wire through de-reeler',
'Follows procedure to access laser marking information on flash drive/computer',
'Laser Marking: Follows proper procedure to access laser marking information on flash drive/computer']

In [249]:
evaldf2 = evaldf.copy()

In [252]:
for i,v in enumerate(evalreplist):
    i1 = i+1
    try:
        v1 = evalreplist[i1]
    except:
        print('done')
    if i%2 == 0:
        evaldf2[v] = np.where(evaldf2[v1].notna(),v,'')
    else:
        continue

done


In [254]:
evaldf2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5639 entries, 0 to 5638
Columns: 200 entries, people_First Name to Follows procedure to access laser marking information on flash drive/computer
dtypes: datetime64[ns](2), float64(1), int32(1), object(196)
memory usage: 8.6+ MB


In [256]:
evaldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5639 entries, 0 to 5638
Columns: 114 entries, people_First Name to Keeps a clean & organized workspace in order to reduce FOD
dtypes: datetime64[ns](2), float64(1), int32(1), object(110)
memory usage: 4.9+ MB


In [257]:
evaldf4 = evaldf2[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'Mentor',
    'Work Cell',
    'For the Week of:_soft',
    'Final Due Date',
    'Attendance (Please note number of full days missed. If no days were missed, fill in 0)_soft',
    'Punctuality_soft',
    'Demeanor & Attitude_soft',
    'Workplace Appearance & Attire_soft',
    'Grooming & Hygiene_soft',
    'Cooperation & Teamwork_soft',
    'Interpersonal Communication Style_soft',
    'Response to Feedback / Instruction_soft',
    'Organization_soft',
    'Work Effort & Initiative_soft',
    'Work Tolerance & Stamina_soft',
    'Learning Style_soft',
    'Follows all Safety Rules & Policies?_soft',
    'Note Any Inappropriate Behaviors Observed during Evaluation Period (if no inappropriate behaviors observed, mark that choice)_soft',
    "Are Person's Soft Skills / General Employment Traits Competitive?_soft",
    'Soft Skills Comments_soft',
    'Keeps a clean & organized workspace in order to reduce FOD',
    'General Employment: Keeps a clean & organized workspace in order to reduce FOD',
    'Utilizes pictures/blueprint to produce clamp block assembly',
    'Assembly: Independent ability to accurately read the details of a blueprint',
    'Lays out blocks for building correctly',
    'Assembly: Properly lays out the blocks for building',
    'Places jobs on correct location on shelves|Returns excess pieces to inventory',
    'Assembly: Demonstrates ability to place jobs on correct spot of warehouse shelf',
    'Sets up for part mark, including part mark location and selecting proper size for part mark',
    'Packaging & Part Mark: Proper set up for part mark',
    'Assesses part mark for quality and redo as needed|Applies overcoat',
    'Packaging & Part Mark: Applies overcoat precisely, with desired thickness',
    'Judges size of bag to package parts|Packages multiple parts that will meet inspection standards|Operates bag sealer',
    'Packaging & Part Mark: Correct operation of bag sealer',
    'Removes labels from boxes before use; Creates labels for job boxes',
    'Clamp Block Inventory: Creates labels for job boxes',
    "When pulling parts from inventory - Identifies and verifies part #'s and lot #'s using the Job Material Queue",
    'Clamp Block Inventory: For each part pulled from inventory, records correct lot # and qty to work order',
    'Issues parts with computer using Job Material Queue and Process by ID',
    'Clamp Block Inventory: Issues parts with computer using Inventory Transaction in VMFG',
    'Operates the chop saw',
    'CHAKU: Operation of saw',
    'Operates the punch press',
    'CHAKU: Operation of punch press',
    'Operates the countersink machine',
    'CHAKU: Operation of countersink',
    'Operates the flat finisher (deburr)',
    'CHAKU: Operation of deburr',
    'Operates the CNC Mill',
    'CHAKU: Operation of CNC Mill',
    'Utilizes work order to differentiate between grommet shape: (ex. F material from J or H material)',
    'Grommets: Utilizes work order to differentiate between grommet shape',
    'Kiss cuts washers',
    'Grommets: Demonstrates ability to kiss cut washers',
    'Wraps prepped grommet onto centers; loads into jig',
    'Grommets: Independent ability to accurately load a jig',
    'Assembles a grommet|Identifies holes in grommet|Repairs holes in grommet|Builds at average rate of 10-15 parts per hour',
    'Grommets: Ability to identify holes in grommet',
    'Cleans offices, corridors, meeting rooms, and common spaces|Dusts|Cleans restrooms|Uses auto scrubber to clean floor|Vacuums carpeted floors|Washes windows and doors|Takes out trash and recycling',
    'Custodial: Follows proper restroom cleaning sequence that meets quality standards',
    'Finds SDS information|Works with chemicals and mixes cleaning solutions',
    'Custodial: Independent ability to properly mix cleaning solutions',
    'Checks appliances and light fixtures, changes light bulbs as needed',
    'Custodial: Independently changes light bulbs as needed',
    'Identifies each job and work order number|Identifies the process for each part/job|Creates identification tags for jobs travelling through processes',
    'Racking / Unracking; Paint Prep: Independent ability to obtain process information from work order traveler',
    'Masks parts using tape and/or plugs|Removes masking tape and/or plugs|Lays out parts on paint screen',
    'Racking / Unracking; Paint Prep: Able to properly layout parts on paint screen',
    'Pre-cleans parts by removing ink or excessive oils',
    'Racking / Unracking; Paint Prep: Pre-cleans parts by removing ink or excessive oils',
    'Enters part mark data on the A200 or A320i part mark machine|Places part mark in correct location',
    'Part Mark: Independently utilize the A320i vendor options for data entry of part mark',
    'Uploads part mark onto a dauber sheet|Applies part mark using ink dauber',
    'Part Mark: Application of part marks by using ink dauber',
    'Ability to correctly judge what size bag to package parts in',
    'Packaging: Ability to correctly judge what size bag to package parts in',
    'Greets visitors and assists with purpose of visit|Answers telephone and direct calls|Handles customer questions or issues requiring follow up',
    'General Admin Clerk: Provides excellent customer service',
    'Checks daily mail and delivers',
    'General Admin Clerk: Independent with the mail run (grabs mail from mailbox, sorts and delivers to appropriate parties)',
    'Scans documents and saves files to the appropriate location on computer|Uses photocopy machine|Creates paperwork packets',
    'General Admin Clerk: Saving documents/files in the appropriate location on the computer',
    'Completes over coat operations',
    'HBU: Independently completes over coating operations',
    'Completes job shadow of an assembly technician',
    'HBU: Completion of job shadow of an assembly technician',
    'Identifies call outs, flags notes and locations for parts on blueprints',
    'HBU - assembly: Recognizes views and orientation of parts on blueprints',
    'Builds parts while being shadowed by mentor/technician',
    'HBU - assembly: Accurately builds parts while being shadowed by mentor/technician',
    'Receives certifications (note in comments which certifications)',
    'HBU - assembly: Receives certifications (note in comments which certifications)',
    'Loads and unloads tooling from holders and spindle',
    'Mills: Independent ability to load and unload tools from holders and spindle',
    'Identifies burrs|Deburrs parts',
    'Mills: Independent ability to deburr parts correctly',
    'Understands and explains concept of X, Y, Z coordinate system',
    'Mills: Able to understand and explain concept of X, Y, Z coordinate system',
    'Sets up tools|Breaks down returned tools',
    'Mills Tool Crib: Accurately sets up tools',
    'Identifies burrs, damage and flaws',
    'Vibraburr: Independent ability to identify a burr',
    'Runs Vibrodyne machine|Runs Roto machine|Runs Burr King machine|Runs Cement Mixer machine',
    'Vibraburr: Independent ability to run Vibrodyne machine',
    'Identifies when machine cleaning is needed (ex. change water)',
    'Vibraburr: Independent ability to clean machine as needed (ex. change water)',
    'Identifies and stamps military export controlled work orders|Identifies work orders with FAI (First Article Inspection) operation|Sorts, staples, and confirms documents are in order',
    'Commercial Work Order Processing: IDs and stamps military export controlled work orders',
    'Verifies that revision levels listed on work order are current (sources of reference may include Boeing EID and customer purchase order notes)',
    'Commercial Work Order Processing: Verifies revision levels listed on work order are current',
    'Opens estimating tab in Epicor and inputs data into correct fields|Opens and reviews estimating quote log in Epicor|Logs onto Exostar/Boeing portal|Opens EID (Engineering Information Delivery) page, selects & downloads correct drawings on Boeing Portal',
    'Estimating Training Evaluation: Correctly opens EID page, selects & downloads correct drawings on Boeing Portal',
    'Identifies drill bits and countersink size',
    'Mechanical Assembly: Independent, accurate usage of precision measuring equipment',
    'Operates the countersink machine|Sets up the countersink machine',
    'Mechanical Assembly: Independent operation of countersink',
    'Operates the riveter/squeeze machine|Sets up the riveter/squeeze machine',
    'Mechanical Assembly: Independent operation of riveter / squeeze',
    'Operates the drill press|Sets up the drill press',
    'Mechanical Assembly: Independent operation of drill press',
    'Utilizes procedure for bonding tasks',
    'Mechanical Assembly : Utilizes proper procedure for bonding tasks',
    'Assembles wiggle plates',
    'Mechanical Assembly : Assembly of wiggle plates',
    'Sets up minor jobs (nut plate, wiggle plate, countersink machine)',
    'Mechanical Assembly : Independent set-up of minor jobs (nut plate, wiggle plate, countersink machine)',
    'Determines drawings required',
    'QA: Ability to determine drawings required and current revisions',
    'Operates the blanking press',
    'Blanking Press: Acceptable operation of blanking press',
    'Locates tooling for each job|Sets up and takes down tooling|Puts tool away',
    'Blanking Press: Ability to locate proper tooling for each job',
    'Identifies burrs|Deburrs parts',
    'Blanking Press: Correctly deburrs parts',
    'Lubricates tools',
    'Blanking Press: Proper lubrication of material',
    'Operates the Amada',
    'Amada: Proper operation of Amada',
    'Operates brake form in Mylar cutting',
    'Brake Form: Operation of brake form in Mylar cutting',
    'Sets up and centers Mylar cutting tool',
    'Brake Form: Independent ability to set up and center Mylar cutting tool',
    'Identifies burrs|Uses "Timesaver"|Deburrs parts',
    'Brake Form: Correctly deburrs parts',
    'Identifies and locates correct material for each job|Loads and unloads material|Performs computer steps to start machine',
    'Routers: Independent ability to identify and locate correct material for each job',
    'Identifies and locates tooling for each job|Sets up and changes tooling plates',
    'Routers: Independent ability to setup and change tooling plates',
    'Identifies burrs|Deburrs parts',
    'Routers: Correctly deburrs parts',
    'Differentiates between Boeing and Commercial work orders|Locates tools and templates used for Boeing work orders|Locates, prints, and verifies certification (cert)',
    'Shears: Independent ability to differentiate between Boeing and Commercial work orders',
    'Sets up the machine',
    'Shears: Independent ability to set up machine',
    'Operates the shear',
    'Shears: Accurate operation of Shear',
    'Understands machine operation and computer inputs|Differentiates between Automatic Start and Manual Start procedure and when each would be used',
    'Waterjet: Understands and demonstrates proper machine operation and computer inputs',
    'Operates the water jet|Sets up the water jet',
    'Waterjet: Independent setup and machine operation of Water Jet',
    'Operates the grind wheel|Uses deburr hand tools including scotch brite or sand paper|Cleans parts thoroughly with alcohol',
    'Deburr: Cleans parts thoroughly with alcohol',
    'Counts and verifies part mark and paperwork prior to packaging outgoing shipment',
    'Commercial Shipping: Sort, segregate and prioritize verified work order by due date and shipping date',
    'Visually inspects incoming product for damage, re-package product according to Orion guidelines prior to moving on to next area',
    'Receiving: Independence with receiving materials and performing tasks in Receiving work area',
    'Operates the pallet jack',
    'Receiving: Independent utilization of pallet jack',
    'Visually inspects incoming product for damage, re-packages product per Orion guidelines prior to moving into inventory',
    "Inventory: for parts pulled out of inventory, verifies correct part #'s on work order",
    'Retrieves product from inventory; moves to next location based on work order or pull ticket',
    'Inventory: Independently finds correct location when picking parts from inventory',
    'Counts product in each specified bin location during cycle count activity',
    'Inventory: Ensures that counts in inventory match what is posted for cycle counts',
    'Applies bond and adheres waffle or strips to orifice|Places weights on waffle orifices',
    'Bond: Application of bond and adherence of waffle or strips to orifice',
    'Calculates cure time for bond',
    'Bond: Independent, accurate calculation of cure time for bond',
    'Creates internal work order',
    'Shipping & Receiving: Independent creation of internal work order',
    'Enters information into Boeing Portal',
    'Shipping & Receiving: Accurate entering of information into Boeing Portal',
    'Utilizes and navigates computer applications',
    'Manufacturing Offices: Independent ability to utilize and navigate computer applications',
    'Creates Purchase Orders|Creates quotes/estimates',
    'Manufacturing Offices: Independently creates Purchase Orders',
    'Gathers data from internal lists and makes determinations based on information',
    'Manufacturing Offices: Gathers correct data from internal lists and makes correct decisions based on this informaton',
    'Visually inspects and measures parts',
    'Wire Assembly: Independent visual inspection and measurement of each part',
    'Ensures that counts per bag match what is posted on front of boxes|Properly runs 2 bag system',
    'Inventory: Proper run of 2 bag system',
    'Inspects and verifies wire part marks/lengths|Completes job (operations closed, paperwork stamped, etc.)',
    'Wire QA: Accurate inspection and verification of wire lengths',
    'Loads and threads wire through de-reeler, unracks wire spool and returns to its correct location',
    'Laser Marking: Independently and accurately loads and threads wire through de-reeler',
    'Follows procedure to access laser marking information on flash drive/computer',
    'Laser Marking: Follows proper procedure to access laser marking information on flash drive/computer',
    'location code']].copy()

In [258]:
evaldf4.to_excel(r'S:\Rehab\Apricot\Migration\exports\eval merge 6.xlsx')

### evaluation 2

In [8]:
# val = pd.read_excel(r'S:\Rehab\Apricot\Migration\exports\eval merge 3.xlsx')

# # drop first fill row
# val = val.drop(index=0)

# # drop index column
# val = val.drop(columns='Unnamed: 0')

In [225]:
val = eval_df3.copy()

In [184]:
soft = val[val['Mentor Evaluation Status'].notna().copy()]
hard = val[val['Mentor Evaluation Status'].isna().copy()]
soft.info(),hard.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5045 entries, 23 to 10967
Columns: 997 entries, Venue to people_Full Name_y
dtypes: datetime64[ns](8), float64(20), int32(1), object(968)
memory usage: 38.4+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 5924 entries, 0 to 10968
Columns: 997 entries, Venue to people_Full Name_y
dtypes: datetime64[ns](8), float64(20), int32(1), object(968)
memory usage: 45.1+ MB


(None, None)

In [177]:
# soft.to_excel(r'S:\Rehab\Apricot\Migration\exports\evalsoft.xlsx')
# hard.to_excel(r'S:\Rehab\Apricot\Migration\exports\evalhard.xlsx')

In [22]:
soft = soft.drop(soft.columns[9:959],axis=1)
hard = hard.drop(hard.columns[959:986],axis=1)
soft.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10827 entries, 12360 to 23186
Data columns (total 40 columns):
 #   Column                                                                                                                         Non-Null Count  Dtype 
---  ------                                                                                                                         --------------  ----- 
 0   Venue                                                                                                                          10824 non-null  object
 1   Mentor                                                                                                                         10827 non-null  object
 2   Individual                                                                                                                     10827 non-null  object
 3   Work Cell                                                                                                       

### Demographics Merge

In [116]:
# peeps3 is the base merge form that contains first last and birthday to prevent duplicates

# peeps3

In [134]:
demo1 = pd.read_excel(r'S:\Rehab\Apricot\Migration\Client information2 (demographics and assessment) - unformatted.xlsx')

demomerge = peeps3.merge(demo1,how='left',on='Individual')
demomerge = demomerge.dropna(how='all',axis=1)
demomerge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1428 entries, 0 to 1427
Data columns (total 53 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   field_3436_first                          1428 non-null   object        
 1   field_3436_last                           1428 non-null   object        
 2   field_3195                                1237 non-null   object        
 3   Individual                                1428 non-null   object        
 4   ID                                        1284 non-null   float64       
 5   Preferred Names                           625 non-null    object        
 6   Date of Birth                             1172 non-null   object        
 7   Gender                                    1284 non-null   object        
 8   Disability - Primary                      1004 non-null   object        
 9   Disabilities - Additional     

In [178]:
demomerge2 = demomerge.drop_duplicates(subset=['field_3436_first','field_3436_last','field_3195'])

In [186]:
import re


demomerge2.loc[:,'Ethnicity'] = demomerge2['Ethnicity'].replace(r';#\d{2}','|',regex=True)
demomerge2.loc[:,'Ethnicity'] = demomerge2['Ethnicity'].str.rstrip('|')
demomerge2.loc[:,'Ethnicity'] = demomerge2['Ethnicity'].str.replace(';#','')
demomerge2['Ethnicity'].head(50)

0                              African American / Black
1                                                 Other
2                                     Caucasian / White
3                                     Caucasian / White
4                                     Caucasian / White
5                                     Caucasian / White
6                                                   NaN
7                                     Caucasian / White
8                                     Caucasian / White
9                                     Caucasian / White
10                                    Caucasian / White
11                                    Caucasian / White
12                             African American / Black
13                                             Hispanic
14    African American / Black|Native Hawaiin / Othe...
15                                    Caucasian / White
16                                    Caucasian / White
17              Native Hawaiin / Other Pacific I

In [188]:
demomerge2.loc[:,'Ethnicity'] = demomerge2['Ethnicity'].str.replace('Hawaiin','Hawaiian')
demomerge2.loc[:,'Ethnicity'] = demomerge2['Ethnicity'].str.replace('African American / Black','Black / African American')
demomerge2.loc[:,'Ethnicity'] = demomerge2['Ethnicity'].str.replace('Multi-Racial','Other')

In [190]:
demomerge2['Ethnicity'].value_counts()

Ethnicity
Caucasian / White                                                                                    728
Black / African American                                                                             149
Asian                                                                                                 92
Other                                                                                                 33
Hispanic                                                                                              29
Native Hawaiian / Other Pacific Islander                                                              25
Unknown                                                                                               17
American Indian / Alaskan Native                                                                      15
Caucasian / White|American Indian / Alaskan Native                                                    10
Black / African American|Caucasian / White   

In [198]:
demomerge2.to_excel(r'S:\Rehab\Apricot\Migration\exports\demographics import2.xlsx')

### Intake Assessment

In [135]:
demo1 = pd.read_excel(r'S:\Rehab\Apricot\Migration\Client information (demographics and assessment) - unformatted.xlsx')

In [136]:
# sort the individual column values alphabetically
demo1 = demo1.sort_values('Individual',ascending=True)

# the key value had a space in it, so needed to remove that space
demo1['Individual'] = demo1['Individual'].str.replace(', ',',')

# remove test case rows
demo1 = demo1[~demo1['Individual'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
demo1['Individual'] = demo1['Individual'].replace(r'\(.*?\)','',regex=True).str.strip()

demo1['Individual'] = demo1['Individual'].replace(r'\".*?\"','',regex=True).str.strip()

In [137]:
peepsmerge = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'people_Full Name'
]].copy()

peepsmerge['people_Full Name'] = peepsmerge['people_Full Name'].str.replace(', ',',')
peepsmerge['demo_Date of Birth'] = peepsmerge['demo_Date of Birth'].fillna('1/1/1800')

In [138]:
assess = demo1.merge(peepsmerge,how='right',left_on='Individual',right_on='people_Full Name')
assess.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535 entries, 0 to 534
Data columns (total 92 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   ID                                        520 non-null    float64       
 1   Individual                                520 non-null    object        
 2   Preferred Names                           144 non-null    object        
 3   Date of Birth                             486 non-null    object        
 4   Gender                                    512 non-null    object        
 5   Disability - Primary                      414 non-null    object        
 6   Disabilities - Additional                 317 non-null    object        
 7   Disability / Barrier Notes:               357 non-null    object        
 8   Date of Last Employment                   362 non-null    datetime64[ns]
 9   Smoker?                         

In [139]:
assess['disabilities'] = assess['Disability - Primary'] +' '+ assess['Disabilities - Additional'] + ' '+ assess['Disability / Barrier Notes:']
assess['disabilities'] = assess['disabilities'].astype(str)

In [140]:
assess['allergy'] = np.where(assess['Allergies'].notna(),'Yes','No')

In [141]:
mentallist = ['Depression','Anxiety','PTSD','ADHD','Substance Abuse','Learning Disability','Hearing','asthma','vision','memory','seizure']

for i in mentallist:
    assess[i] = assess['disabilities'].apply(lambda x: 'Yes' if i in x else 'No')
assess

,ID,Individual,Preferred Names,Date of Birth,Gender,Disability - Primary,Disabilities - Additional,Disability / Barrier Notes:,Date of Last Employment,Smoker?,...,Anxiety,PTSD,ADHD,Substance Abuse,Learning Disability,Hearing,asthma,vision,memory,seizure
0,775.0,"Shuman,Keith",NaN,1970-05-22 00:00:00,Male,Dev - Autism,Dev - Intellectual;#6,NaN,NaT,No,...,No,No,No,No,No,No,No,No,No,No
1,1059.0,"Waltjen,Melissa",Missy,1977-10-06 00:00:00,Female,Dev - Intellectual,Hearing - Hard of Hearing;#11,Missy wears hearing aids and when being spoken...,NaT,No,...,No,No,No,No,No,Yes,No,No,No,No
2,1061.0,"Lockhart,Rebekah",Bekah,2002-03-26 00:00:00,Female,Ortho - Leg / Foot,Psych - Anxiety;#43;#Psych - Depression;#46,Hx of knee injury. Did rehab post surgery. Rep...,2022-11-21,No,...,Yes,No,No,No,No,No,No,No,No,No
3,1060.0,"Gainey,Jerry",NaN,1985-02-16 00:00:00,Male,Substance Abuse,Psych - Depression;#46;#Psych - Anxiety;#43,hx of addiction to opiates and alcohol; clean ...,2022-03-02,No,...,Yes,No,No,Yes,No,No,No,No,No,No
4,1062.0,"Oster,Dez'Rae",NaN,1995-03-28 00:00:00,Female,Psych - PTSD,Psych - Depression;#46;#Psych - Anxiety;#43;#M...,"Has never has worked, needs to get driver's li...",NaT,NaN,...,Yes,Yes,No,No,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,1057.0,"Ghansah,Maxine",NaN,1989-09-02 00:00:00,Female,Psych - Schizoaffective,Psych - Bi-Polar;#29,NaN,NaT,No,...,No,No,No,No,No,No,No,No,No,No
531,1058.0,"Mason,Steven",NaN,1966-04-19 00:00:00,Male,Med - Other,NaN,NaN,2020-03-01,Yes,...,No,No,No,No,No,No,No,No,No,No
532,1083.0,"Phillips,Savannah",Savannah,1997-05-19 00:00:00,Female,Neuro - Learning Disability,Psych - Depression;#46,Hx of learning (Triple X syndrome) and depress...,2019-01-01,No,...,No,No,No,No,Yes,No,No,No,No,No
533,1158.0,"Johnston,George",NaN,1975-08-14 00:00:00,Male,Med - Coronary,Psych - Depression;#46,Shortness of breath - pacemaker \nCHP - left b...,2019-07-01,No,...,No,No,No,No,No,No,No,No,No,No


In [142]:
assess['Transportation'].value_counts()

Transportation
Car               206
Public Transit    170
Para-transit       38
Ride               28
Other              15
School Bus         10
Unknown             4
Bike                3
Name: count, dtype: int64

In [143]:
tranrep = {
    'Car':'Driving Myself',
    'Public Transit':'Public Transportation',
    'Para-transit':'Paratransit',
    'Ride':'Rides/Carpool',
    'Unknown':'Other'
}
assess['Transportation'] = assess['Transportation'].replace(tranrep)

In [144]:
assess['Work Restrictions'] =assess['Work Restrictions'].astype(str)

physlist = ['Standing','Sitting','Lift over 10','Lift over 25','Bending / Stooping']

for i in physlist:
    assess[i] = assess['Work Restrictions'].apply(lambda x: 'No' if i in x else 'Yes')

In [145]:
mentlist = ['Reading','English Language Proficiency',]

for i in mentlist:
    assess[i] = assess['Work Restrictions'].apply(lambda x: 'Yes' if i in x else 'No')

In [146]:
assess['Education Level'].value_counts()

Education Level
High School Diploma                                           215
Some College / Technical School (no certificate / diploma)     67
No High School Diploma / GED                                   58
GED                                                            57
Graduated - In High School                                     29
Bachelors                                                      25
Associates                                                     21
In High School                                                 12
Certificate                                                     6
Other                                                           4
Masters (or beyond)                                             3
Name: count, dtype: int64

In [147]:
edurep = {
    'High School Diploma':'HS Diploma',
    'Some College / Technical School (no certificate / diploma)':'Some post high school',
    'No High School Diploma / GED':'No HS Diploma - Under 12th grade education',
    'Graduated - In High School':'HS Diploma',
    'Masters (or beyond)':'Masters and above',
    'In High School':'No HS Diploma - Under 12th grade education',
    'Certificate':'Certificate - 2 years or less'
}
assess['Education Level'] = assess['Education Level'].replace(edurep)

In [148]:
assess['Benefits'] = assess['Benefits'].str.replace(';#',',',regex=True)
assess['Benefits'] = assess['Benefits'].str.replace('Food Stamps','SNAP',regex=True)
assess['Benefits'] = assess['Benefits'].str.replace('None','',regex=True)

In [149]:
assess['Insurance'] = np.where(assess['Benefits'].str.contains('Medical Benefits \(State / Fed\)'),'State Health Insurance for Adults','')

assess['non cash'] = np.where(assess['Benefits'].str.contains('Childcare Voucher'),'Childcare Voucher',
                            np.where(assess['Benefits'].str.contains('Housing Choice'),'Housing Choice Voucher',
                            np.where(assess['Benefits'].str.contains('Public Housing'),'Public Housing',
                            np.where(assess['Benefits'].str.contains('Permanent Supportive'),'Permanent Supportive Housing',
                            ''))))

In [150]:
benrep = {
    'L&I - Time Loss':'L&I Time Loss',
    'SNAP':'SNAP / EBT / Food Stamps',
    'Unemployment Insurance':'Unemployment',
    'Veterans Benefits':'VA Service-Connectid Disability Compensation',
    'WIC (CSBG)':'WIC',
    'Medical Benefits \(State / Fed\)':'',
    'Housing Assistance':'',
    'Childcare Voucher \(CSBG\)':'',
    'Housing Choice Voucher \(CSBG\)':'',
    'Public Housing \(CSBG\)':'',
    'Permanent Supportive Housing \(CSBG\)':''
}

assess['Benefits'] = assess['Benefits'].replace(benrep,regex=True)

In [151]:
assess['medication'] = np.where(assess['Medications and Conditions'].notna(),'Yes','No')

In [152]:
assess['Benefits']

0                                ,,SSI
1                                 SSDI
2                                     
3            ,SNAP / EBT / Food Stamps
4             SNAP / EBT / Food Stamps
                    ...               
530      SNAP / EBT / Food Stamps,,SSI
531           SNAP / EBT / Food Stamps
532       SNAP / EBT / Food Stamps,SSI
533    ,,SNAP / EBT / Food Stamps,SSDI
534          ,SNAP / EBT / Food Stamps
Name: Benefits, Length: 535, dtype: object

In [153]:
assess['Benefits'] = assess['Benefits'].str.replace(',,',',',regex=True)
assess['Benefits'] = assess['Benefits'].str.strip(',')

In [156]:
assess['Benefits'] = assess['Benefits'].str.replace(',','|',regex=True)
assess['Benefits']

0                                SSI
1                               SSDI
2                                   
3           SNAP / EBT / Food Stamps
4           SNAP / EBT / Food Stamps
                   ...              
530     SNAP / EBT / Food Stamps|SSI
531         SNAP / EBT / Food Stamps
532     SNAP / EBT / Food Stamps|SSI
533    SNAP / EBT / Food Stamps|SSDI
534         SNAP / EBT / Food Stamps
Name: Benefits, Length: 535, dtype: object

In [157]:
assess['Child'] = np.where((assess['Number of Dependents'] == 0) | (assess['Number of Dependents'].isna()),'No','Yes')

In [162]:
assess2 = assess[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'Created',
    'Standing',
    'Sitting',
    'Bending / Stooping',
    'Lift over 10',
    'Lift over 25',
    'disabilities',
    'Education Level',
    'Reading',
    'English Language Proficiency',
    'Special Education History',
    'Hearing',
    'vision',
    'memory',
    'seizure',
    'asthma',
    'allergy',
    'Allergies',
    'Depression',
    'Anxiety',
    'PTSD',
    'disabilities',
    'ADHD',
    'medication',
    'Medications and Conditions',
    'Transportation',
    'Child',
    'Household Size',
    'Number of Dependents',
    'Substance Abuse',
    'Smoker?',
    'Benefits',
    'Insurance',
    'Health Insurance',
    'non cash'
]]

In [163]:
assess2.to_excel(r'S:\Rehab\Apricot\Migration\exports\real assess merge3.xlsx')

### Case note merge

In [4]:
case1 = pd.read_excel(r'S:\Rehab\Apricot\Migration\case note - unformatted.xlsx')
case1 = case1.add_suffix('_case')
case2 = pd.read_excel(r'S:\Rehab\Apricot\Migration\outcome activity (case note) - unformatted.xlsx')
case2 = case2.add_suffix('_activity')
area = pd.read_excel(r'S:\Rehab\Apricot\Migration\outcome area (case notes) - unformatted.xlsx')
area = area.add_suffix('_area')
goal = pd.read_excel(r'S:\Rehab\Apricot\Migration\outcome goal (goals) - unformatted.xlsx')
goal = goal.add_suffix('_goal')

In [5]:
outcome1 = goal.merge(area,how='left',left_on='Outcome Area ID_goal',right_on='ID_area')
outcome1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5885 entries, 0 to 5884
Data columns (total 44 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   Start Date_goal                               5865 non-null   datetime64[ns]
 1   Goal_goal                                     5882 non-null   object        
 2   Action Steps_goal                             0 non-null      float64       
 3   Goal Summary_goal                             1830 non-null   object        
 4   Created_goal                                  5885 non-null   datetime64[ns]
 5   Created By_goal                               5885 non-null   object        
 6   Modified_goal                                 5885 non-null   datetime64[ns]
 7   Modified By_goal                              5885 non-null   object        
 8   Client_goal                                   5885 non-null   object

In [7]:
outcome2 = case2.merge(outcome1,how='left',left_on='GoalID_activity',right_on='ID_goal')

In [14]:
outcome2.columns

Index(['Date_activity', 'Client_activity',
       'Time Spent on this Activity_activity', 'Client Hours_activity',
       'Hours Type_activity', 'Staff_activity', 'Contact Type_activity',
       'DDA Phase_activity', 'Modified_activity', 'Activity_activity',
       'Administrative Time_activity', 'Progress_activity',
       'TotalHours_activity', 'Travel Time_activity', 'CaseNoteID_activity',
       'Content Type_activity', 'Created_activity', 'Created By_activity',
       'Folder Child Count_activity', 'GoalID_activity', 'ID_activity',
       'Item Child Count_activity', 'Modified By_activity', 'Title_activity',
       'Item Type_activity', 'Path_activity', 'Start Date_goal', 'Goal_goal',
       'Action Steps_goal', 'Goal Summary_goal', 'Created_goal',
       'Created By_goal', 'Modified_goal', 'Modified By_goal', 'Client_goal',
       'Completion Date_goal', 'Discontinuation Date_goal',
       'Content Type_goal', 'Folder Child Count_goal',
       'How do you know this goal has been 

In [39]:
outcome3 = outcome2[[
    'Client_activity',
    'Date_activity',
    'Contact Type_activity',
    'Time Spent on this Activity_activity',
    'Activity_activity',
    'DDA Phase_activity',
    'Administrative Time_activity',
    'Travel Time_activity',
    'Goal_goal',
    'Service_area',
    'Funding Source_area'
]]

In [40]:
case1.columns

Index(['ID_case', 'Date_case', 'Client_case', 'Venue_case', 'Service_case',
       'DDA Phase_case', 'Contact Type_case', 'Created_case',
       'Created By_case', 'Content Type_case',
       'Hours Spent (not on defined goals)_case', 'Other Comments_case',
       'Administrative Time_case', 'Folder Child Count_case',
       'Funding Source_case', 'Item Child Count_case', 'Modified_case',
       'Modified By_case', 'Staff_case', 'Title_case',
       'Total Hours Spent (not on defined goals)_case', 'Travel Time_case',
       'Item Type_case', 'Path_case'],
      dtype='object')

In [41]:
case11 = case1[[
    'Client_case',
    'Date_case',
    'Contact Type_case',
    'Hours Spent (not on defined goals)_case',
    'Other Comments_case',
    'DDA Phase_case',
    'Administrative Time_case',
    'Travel Time_case',
    'Service_case',
    'Funding Source_case'
]]

In [42]:
casecol1 = [
    'Client',
    'Date',
    'Contact Type',
    'Time Spent',
    'Case Note',
    'DDA Phase',
    'Admin Time',
    'Travel Time',
    'Service',
    'Funding'
]

casecol2 = [
    'Client',
    'Date',
    'Contact Type',
    'Time Spent',
    'Case Note',
    'DDA Phase',
    'Admin Time',
    'Travel Time',
    'Goal',
    'Service',
    'Funding'
]

case11.columns = casecol1
outcome3.columns = casecol2

In [43]:
casenote = pd.concat([outcome3,case11],axis=0)
casenote.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81884 entries, 0 to 47771
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Client        81884 non-null  object        
 1   Date          81884 non-null  datetime64[ns]
 2   Contact Type  72346 non-null  object        
 3   Time Spent    66937 non-null  float64       
 4   Case Note     72315 non-null  object        
 5   DDA Phase     11666 non-null  object        
 6   Admin Time    2 non-null      float64       
 7   Travel Time   6 non-null      float64       
 8   Goal          34022 non-null  object        
 9   Service       75150 non-null  object        
 10  Funding       68276 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(7)
memory usage: 7.5+ MB


In [44]:
casenote['Goal Reviewed?'] = np.where(casenote['Goal'].notna(),'Yes','No')

In [45]:
casenote[['Client','Individual']] = casenote['Client'].str.split(';#', expand=True)

# sort the individual column values alphabetically
casenote = casenote.sort_values('Individual',ascending=True)

# the key value had a space in it, so needed to remove that space
casenote['Individual'] = casenote['Individual'].str.replace(', ',',')

# remove test case rows
casenote = casenote[~casenote['Individual'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
casenote['Individual'] = casenote['Individual'].replace(r'\(.*?\)','',regex=True).str.strip()

casenote['Individual'] = casenote['Individual'].replace(r'\".*?\"','',regex=True).str.strip()

In [46]:
peepsmerge = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'people_Full Name'
]].copy()

peepsmerge['people_Full Name'] = peepsmerge['people_Full Name'].str.replace(', ',',')
peepsmerge['demo_Date of Birth'] = peepsmerge['demo_Date of Birth'].fillna('1/1/1800')

In [47]:
casenote2 = casenote.merge(peepsmerge,how='right',left_on='Individual',right_on='people_Full Name')
casenote2 = casenote2.drop(columns=['Admin Time','Travel Time'])
casenote2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50996 entries, 0 to 50995
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Client              50990 non-null  object        
 1   Date                50990 non-null  datetime64[ns]
 2   Contact Type        43753 non-null  object        
 3   Time Spent          41640 non-null  float64       
 4   Case Note           43783 non-null  object        
 5   DDA Phase           10306 non-null  object        
 6   Goal                19298 non-null  object        
 7   Service             48021 non-null  object        
 8   Funding             46442 non-null  object        
 9   Goal Reviewed?      50990 non-null  object        
 10  Individual          50990 non-null  object        
 11  people_First Name   50996 non-null  object        
 12  people_Last Name    50996 non-null  object        
 13  demo_Date of Birth  50996 non-null  object    

In [35]:
casenote2['Time Spent'].value_counts()

Time Spent
0.25    19339
0.50     7105
1.00     4178
1.50     2566
2.00     2357
0.75     1958
1.25     1394
1.75      849
2.50      664
3.00      391
2.25      260
2.75      136
0.00       91
3.50       67
4.50       52
3.25       45
4.00       40
4.25       34
4.75       26
5.00       23
3.75       16
5.50       12
7.00        6
6.50        6
8.00        5
5.25        5
6.00        4
6.25        3
5.75        3
7.50        3
7.25        2
Name: count, dtype: int64

In [48]:
contactrep = {
    'Face to Face':'In Person',
    'General Info':'General',
    'Instructor / Set-Up':'Instructor',
    'DDA - Record Keeping':'DDA Record Keeping',
    'Referral Agency':'Referral',
    'Time Off Request':'Mentor Communication',
    'Closure Note (Include any return requirements or recommendations)':'General',
    '(Select)':'General'
}
casenote2['Contact Type'] = casenote2['Contact Type'].replace(contactrep)

In [49]:
casenote2['Who Participated'] = 'Client'

In [50]:
casenote2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50996 entries, 0 to 50995
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Client              50990 non-null  object        
 1   Date                50990 non-null  datetime64[ns]
 2   Contact Type        43753 non-null  object        
 3   Time Spent          41640 non-null  float64       
 4   Case Note           43783 non-null  object        
 5   DDA Phase           10306 non-null  object        
 6   Goal                19298 non-null  object        
 7   Service             48021 non-null  object        
 8   Funding             46442 non-null  object        
 9   Goal Reviewed?      50990 non-null  object        
 10  Individual          50990 non-null  object        
 11  people_First Name   50996 non-null  object        
 12  people_Last Name    50996 non-null  object        
 13  demo_Date of Birth  50996 non-null  object    

In [51]:
casenote2['Referrals Made'] = np.where(casenote2['Contact Type'] == 'Referral','Yes','No')

In [54]:
casenote2['Service'].value_counts()

Service
DDA - Individual Employment Services    9320
                                        8619
Assessment                              8229
Job Development                         5053
Community Based Assessment              4843
Case Management                         2617
BFET - SL (Life Skills)                 2395
DDA - Community Inclusion               2092
Job Retention                           1496
Training                                1246
DDA - School to Work                     842
BFET - JT (Job Search Training)          454
Intensive Training Services              263
BFET - JS (Supervised Job Search)        219
BFET - BR (Job Retention)                147
DDA - Job Foundations                     81
Internship - Other                        52
Internship - CNC Mills                    30
IL Services                               23
Name: count, dtype: int64

In [55]:
import re
casenote2.loc[:,'Service'] = casenote2['Service'].replace(r'\d{1,2};#','',regex=True)
casenote2.loc[:,'Funding'] = casenote2['Funding'].replace(r'\d{1,2};#','',regex=True)

In [75]:
casenote2['Import Service'] = casenote2['Service']
casenote2['Import Funding'] = casenote2['Funding']

In [77]:
casenote3 = casenote2[casenote2['Case Note'].notna()].copy()
casenote3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43783 entries, 0 to 50995
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Client              43783 non-null  object        
 1   Date                43783 non-null  datetime64[ns]
 2   Contact Type        43548 non-null  object        
 3   Time Spent          41521 non-null  float64       
 4   Case Note           43783 non-null  object        
 5   DDA Phase           10186 non-null  object        
 6   Goal                19153 non-null  object        
 7   Service             40826 non-null  object        
 8   Funding             39259 non-null  object        
 9   Goal Reviewed?      43783 non-null  object        
 10  Individual          43783 non-null  object        
 11  people_First Name   43783 non-null  object        
 12  people_Last Name    43783 non-null  object        
 13  demo_Date of Birth  43783 non-null  object        


In [60]:
query = pd.read_excel(r'S:\Rehab\Apricot\Migration\exports\appricot subservice funder export.xlsx')

In [62]:
query = query.drop_duplicates(subset=['First','Last','Status','Location','Date Intake Started','Date Services Starting','Service','Sub-Service Category','Funding Source'])
query.info()

<class 'pandas.core.frame.DataFrame'>
Index: 587 entries, 0 to 640
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   First                   587 non-null    object 
 1   Middle                  0 non-null      float64
 2   Last                    587 non-null    object 
 3   Date of Birth           587 non-null    object 
 4   Status                  587 non-null    object 
 5   Location                587 non-null    object 
 6   Date Intake Started     585 non-null    object 
 7   Date Services Starting  587 non-null    object 
 8   Service                 587 non-null    object 
 9   Sub-Service Category    587 non-null    object 
 10  Funding Source          587 non-null    object 
 11  Start Date              587 non-null    object 
 12  associated service      587 non-null    object 
 13  sub service             587 non-null    object 
dtypes: float64(1), object(13)
memory usage: 68.8+ K

In [78]:
casenote3['Apricot Service'] = np.where(casenote3['Import Service'].str.contains('Assessment|Trial Work Experience|Life Skills'),'Assessment',
                                    np.where(casenote3['Import Service'].str.contains('DDA'),'DDA Services',
                                    np.where(casenote3['Import Service'].str.contains('Job Retention|Intensive Training Services'),'Job Retention',
                                    np.where(casenote3['Import Service'].str.contains('Job Development|Supervised Job Search|Job Search Training'),'Job Development',
                                    np.where(casenote3['Import Service'].str.contains('Case Management'),'Case Management',
                                    np.where(casenote3['Import Service'].str.contains('IL Services'),'Independent Living',
                                    np.where(casenote3['Import Service'].str.contains('Internship|Training'),'Training',
                                    '')))))))

In [87]:
casenote3[['Apricot Service','Apricot Funding']].value_counts()

Apricot Service     Apricot Funding                               
DDA Services        DDA                                               11802
Assessment          DVR                                                5433
                    BFET                                               2611
                    King County Jobs & Housing                         2502
Case Management     Snohomish County - ERSS                            2451
Job Development                                                        2378
                    DVR                                                1655
                                                                       1639
Assessment                                                             1396
                    City Funding - Human Services (Auburn, Renton)     1162
Job Retention                                                           887
                    DVR                                                 763
Assessment          C

In [80]:
casenote3['Import Funding'] = casenote3['Import Funding'].str.replace('1','',regex=True)

In [81]:
fundrep = {
    'Olsen & Associates':'Labor & Industries',
    'Strategic Consulting Services':'Labor & Industries',
    'Pioneer Industries':'Labor & Industries',
    'Aerospace Joint Apprenticeship Committee (AJAC)':'Labor & Industries',
    'Workforce Training and Education Coordinating Board':'WRT',
    'WRT - State Board for Community and Technical Colleges':'WRT',
    'City of Renton':'City Funding - Human Services (Auburn, Renton)',
    'City of Kent':'City Funding - Human Services (Auburn, Renton)',
    'City of Auburn':'City Funding - Human Services (Auburn, Renton)',
    'City of Federal Way':'City Funding - CDBG (Federal Way)',
    'Snohomish County Human Services | Housing and Community Services':'Snohomish County - ERSS',
    'DDA-King County':'DDA',
    'DDA-Snohomish':'DDA',
    'Veterans Administration-Seattle':'Veterans Administration',
    'King County Jobs and Housing':'King County Jobs & Housing',
    'SJI (Seattle Jobs Initiative)':'Labor & Industries',
    'DSB-Tacoma':'DVR'
}

casenote3['Apricot Funding'] = casenote3['Import Funding'].replace(fundrep)

In [84]:
casenote3['Apricot Funding'] = casenote3['Apricot Funding'].str.replace(r'^DVR*','DVR',regex=True)

In [86]:
casenote3['Apricot Funding'].value_counts()

Apricot Funding
DDA                                               11963
DVR                                                8345
                                                   6546
BFET                                               3569
King County Jobs & Housing                         3051
Snohomish County - ERSS                            2726
City Funding - Human Services (Auburn, Renton)     1417
City Funding - CDBG (Federal Way)                   981
WRT                                                 535
Labor & Industries                                  119
Fusion Family Center                                  6
Orion Industries                                      1
Name: count, dtype: int64

In [89]:
casenote4 = casenote3.merge(query,how='left',left_on=['people_First Name','people_Last Name','Apricot Service','Apricot Funding'],right_on=['First','Last','Service','Funding Source'])

In [91]:
casenote4['CI Phase'] = np.where(casenote4['Import Service'].str.contains('Community Inclusion'),'Record Keeping','')

In [92]:
casenote4.to_excel(r'S:\Rehab\Apricot\Migration\exports\real casenote2.xlsx')

### Outcome Area/goal/activity

In [8]:
area = pd.read_excel(r'S:\Rehab\Apricot\Migration\outcome area (case notes) - unformatted.xlsx')
area = area.add_suffix('_area')
goal = pd.read_excel(r'S:\Rehab\Apricot\Migration\outcome goal (goals) - unformatted.xlsx')
goal = goal.add_suffix('_goal')
activity = pd.read_excel(r'S:\Rehab\Apricot\Migration\outcome activity (case note) - unformatted.xlsx')
activity = activity.add_suffix('_activity')

In [9]:
outcome1 = goal.merge(area,how='left',left_on='Outcome Area ID_goal',right_on='ID_area')

outcome1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5885 entries, 0 to 5884
Data columns (total 44 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   Start Date_goal                               5865 non-null   datetime64[ns]
 1   Goal_goal                                     5882 non-null   object        
 2   Action Steps_goal                             0 non-null      float64       
 3   Goal Summary_goal                             1830 non-null   object        
 4   Created_goal                                  5885 non-null   datetime64[ns]
 5   Created By_goal                               5885 non-null   object        
 6   Modified_goal                                 5885 non-null   datetime64[ns]
 7   Modified By_goal                              5885 non-null   object        
 8   Client_goal                                   5885 non-null   object

In [10]:
outcome2 = activity.merge(outcome1,how='left',left_on='GoalID_activity',right_on='ID_goal')

In [11]:
outcome2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34112 entries, 0 to 34111
Data columns (total 70 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   Date_activity                                 34112 non-null  datetime64[ns]
 1   Client_activity                               34112 non-null  object        
 2   Time Spent on this Activity_activity          32017 non-null  float64       
 3   Client Hours_activity                         32 non-null     float64       
 4   Hours Type_activity                           37 non-null     object        
 5   Staff_activity                                34109 non-null  object        
 6   Contact Type_activity                         33742 non-null  object        
 7   DDA Phase_activity                            6112 non-null   object        
 8   Modified_activity                             34112 non-null  date

In [12]:
outcome2['Client_activity']

0        9;#Anderson, Jimmy-Test
1        9;#Anderson, Jimmy-Test
2        9;#Anderson, Jimmy-Test
3        9;#Anderson, Jimmy-Test
4        9;#Anderson, Jimmy-Test
                  ...           
34107      1221;#McCaughan, Shay
34108     1386;#Waltjen, Melissa
34109            442;#Yao, Shaun
34110            442;#Yao, Shaun
34111             1489;#Wy, Evan
Name: Client_activity, Length: 34112, dtype: object

In [538]:
# split out the contents of the client column on the ;# and add the split values to the newly created individual column
outcome2[['Client_activity','Individual']] = outcome2['Client_activity'].str.split(';#', expand=True)

# sort the individual column values alphabetically
outcome2 = outcome2.sort_values('Individual',ascending=True)

# the key value had a space in it, so needed to remove that space
outcome2['Individual'] = outcome2['Individual'].str.replace(', ',',')

# remove test case rows
outcome2 = outcome2[~outcome2['Individual'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
outcome2['Individual'] = outcome2['Individual'].replace(r'\(.*?\)','',regex=True).str.strip()

outcome2['Individual'] = outcome2['Individual'].replace(r'\".*?\"','',regex=True).str.strip()

In [539]:
peepsmerge = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'people_Full Name'
]].copy()

peepsmerge['people_Full Name'] = peepsmerge['people_Full Name'].str.replace(', ',',')
peepsmerge['demo_Date of Birth'] = peepsmerge['demo_Date of Birth'].fillna('1/1/1800')

In [540]:
outcome3 = outcome2.merge(peepsmerge,how='right',left_on='Individual',right_on='people_Full Name')
outcome3

,Date_activity,Client_activity,Time Spent on this Activity_activity,Client Hours_activity,Hours Type_activity,Staff_activity,Contact Type_activity,DDA Phase_activity,Modified_activity,Activity_activity,Administrative Time_activity,Progress_activity,TotalHours_activity,Travel Time_activity,CaseNoteID_activity,Content Type_activity,Created_activity,Created By_activity,Folder Child Count_activity,GoalID_activity,ID_activity,Item Child Count_activity,Modified By_activity,Title_activity,Item Type_activity,Path_activity,Start Date_goal,Goal_goal,Action Steps_goal,Goal Summary_goal,Created_goal,Created By_goal,Modified_goal,Modified By_goal,Client_goal,Completion Date_goal,Discontinuation Date_goal,Content Type_goal,Folder Child Count_goal,How do you know this goal has been met?_goal,ID_goal,Item Child Count_goal,Outcome Area ID_goal,Projected Completion Date_goal,Reason for Discontinuation_goal,Review Schedule_goal,Title_goal,Item Type_goal,Path_goal,Date_area,Client_area,Service_area,Objective - DO NOT USE_area,End Date_area,Outcome Successful?_area,Funding Source_area,Goal Area_area,Venue_area,Comments_area,Created_area,Created By_area,Content Type_area,Folder Child Count_area,ID_area,Item Child Count_area,Modified_area,Modified By_area,Title_area,Item Type_area,Path_area,Individual,people_First Name,people_Last Name,demo_Date of Birth,people_Full Name
0,2022-11-07,1105,1.75,NaN,NaN,Ron Lagman,Face to Face,Phase 1 - Discovery / Assessment / Job Prep,2022-11-07 12:24:16,Client and EC volunteered at Bella's Voice. Ke...,NaN,Satisfactory Progress,1.75,NaN,23863,Activity,2022-11-07 12:24:16,Ron Lagman,0.0,2533.0,19676.0,0.0,Ron Lagman,View,Item,Lists/OutcomeActivity,2021-06-01,Engage with individuals in the community while...,NaN,NaN,2021-08-30 14:25:04,Traci Dosch,2023-04-19 11:29:51,Megan Baggett,"1105;#Shuman, Keith",2023-04-17,2023-04-17,Goal,0.0,Look into volunteer positions throughout South...,2533.0,0.0,2487.0,2021-08-31,Chose other goal,2021-06-01 00:00:00,View,Item,Lists/OutcomeGoal,2021-06-01,"1105;#Shuman, Keith",24;#DDA - Community Inclusion,NaN,NaT,NaN,84;#DDA-King County,Community Involvement - Engage in my Community,21;#Mukilteo - Community Based Services,Keith would like to volunteer at a thrift stor...,2021-08-30 14:23:19,Traci Dosch,Outcome Area,0.0,2487.0,0.0,2021-08-30 14:23:19,Traci Dosch,View,Item,Lists/OutcomeArea,"Shuman,Keith",Keith,Shuman,1970-05-22 00:00:00,"Shuman,Keith"
1,2024-01-08,1105,1.75,NaN,NaN,Megan Welch,Face to Face,Phase 1 - Discovery / Assessment / Job Prep,2024-01-16 13:11:00,Keith left in the middle of class to use the b...,NaN,Satisfactory Progress,1.75,NaN,40331,Activity,2024-01-16 13:11:00,Megan Welch,0.0,4100.0,30066.0,0.0,Megan Welch,View,Item,Lists/OutcomeActivity,2023-04-17,I WILL USE THE BATHROOM BEFORE MY EC PICKS ME ...,NaN,NaN,2023-04-19 11:28:44,Megan Baggett,2024-02-27 09:35:28,Megan Welch,"1105;#Shuman, Keith",NaT,2024-01-15,Goal,0.0,NaN,4100.0,0.0,2487.0,NaT,Chose other goal,2023-04-17 00:00:00,View,Item,Lists/OutcomeGoal,2021-06-01,"1105;#Shuman, Keith",24;#DDA - Community Inclusion,NaN,NaT,NaN,84;#DDA-King County,Community Involvement - Engage in my Community,21;#Mukilteo - Community Based Services,Keith would like to volunteer at a thrift stor...,2021-08-30 14:23:19,Traci Dosch,Outcome Area,0.0,2487.0,0.0,2021-08-30 14:23:19,Traci Dosch,View,Item,Lists/OutcomeArea,"Shuman,Keith",Keith,Shuman,1970-05-22 00:00:00,"Shuman,Keith"
2,2022-04-29,1105,2.25,NaN,NaN,Ron Lagman,Face to Face,Phase 1 - Discovery / Assessment / Job Prep,2022-04-29 10:08:44,"Keith arrived at Bella's voice on time, and cl...",NaN,Needs Attention,2.25,NaN,18629,Activity,2022-04-29 10:08:44,Ron Lagman,0.0,2533.0,15559.0,0.0,Ron Lagman,View,Item,Lists/OutcomeActivity,2021-06-01,Engage with individuals in the community while...,NaN,NaN,2021-08-30 14:25:04,Traci Dosch,2023-04-19 11:29:51,Megan Baggett,"1105;#Shuman, Keith",2023-04-17,2023-04-17,Goal,0.0,Look into volunteer positions throughout South...,2533.0,0.0

In [541]:
for i in outcome3.columns:
    print(i)

Date_activity
Client_activity
Time Spent on this Activity_activity
Client Hours_activity
Hours Type_activity
Staff_activity
Contact Type_activity
DDA Phase_activity
Modified_activity
Activity_activity
Administrative Time_activity
Progress_activity
TotalHours_activity
Travel Time_activity
CaseNoteID_activity
Content Type_activity
Created_activity
Created By_activity
Folder Child Count_activity
GoalID_activity
ID_activity
Item Child Count_activity
Modified By_activity
Title_activity
Item Type_activity
Path_activity
Start Date_goal
Goal_goal
Action Steps_goal
Goal Summary_goal
Created_goal
Created By_goal
Modified_goal
Modified By_goal
Client_goal
Completion Date_goal
Discontinuation Date_goal
Content Type_goal
Folder Child Count_goal
How do you know this goal has been met?_goal
ID_goal
Item Child Count_goal
Outcome Area ID_goal
Projected Completion Date_goal
Reason for Discontinuation_goal
Review Schedule_goal
Title_goal
Item Type_goal
Path_goal
Date_area
Client_area
Service_area
Objecti

In [36]:
outcome3.to_excel(r'S:\Rehab\Apricot\Migration\exports\outcome area goal activity.xlsx')

#### Goals alone

In [563]:
goalserv = pd.read_excel(r'S:\Rehab\Apricot\Migration\exports\appricot services export.xlsx')

In [543]:
# split out the contents of the client column on the ;# and add the split values to the newly created individual column
goal[['Client_goal','Individual']] = goal['Client_goal'].str.split(';#', expand=True)

# sort the individual column values alphabetically
goal = goal.sort_values('Individual',ascending=True)

# the key value had a space in it, so needed to remove that space
goal['Individual'] = goal['Individual'].str.replace(', ',',')

# remove test case rows
goal = goal[~goal['Individual'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
goal['Individual'] = goal['Individual'].replace(r'\(.*?\)','',regex=True).str.strip()

goal['Individual'] = goal['Individual'].replace(r'\".*?\"','',regex=True).str.strip()

In [559]:
peepsmerge = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'people_Full Name',
]].copy()

peepsmerge['people_Full Name'] = peepsmerge['people_Full Name'].str.replace(', ',',')
peepsmerge['demo_Date of Birth'] = peepsmerge['demo_Date of Birth'].fillna('1/1/1800')

In [564]:
goalserv['full'] = goalserv['Last']+','+goalserv['First']
goalserv

,First,Middle,Last,Date of Birth,Status,Location,Date Services Starting,full
0,Aaron,NaN,Erickson,03/10/1963,Services Started,Auburn,08/04/2023,"Erickson,Aaron"
1,Aaron,NaN,Williams,04/29/1983,Services Started,Mukilteo,09/26/2022,"Williams,Aaron"
2,Adan,NaN,Mendez-Lopez,11/03/2002,Services Started,Mukilteo,10/10/2022,"Mendez-Lopez,Adan"
3,Aiyana,NaN,Rightious,10/13/1997,Services Started,Mukilteo,09/11/2018,"Rightious,Aiyana"
4,Alan,NaN,Masta,04/05/1982,Services Started,Mukilteo,03/12/2024,"Masta,Alan"
...,...,...,...,...,...,...,...,...
456,Yonathan,NaN,Worku,03/21/1964,Services Started,Auburn,02/24/2023,"Worku,Yonathan"
457,Yvonne,NaN,Frazier,01/24/1962,Services Started,Auburn,01/13/2023,"Frazier,Yvonne"
458,Zachary,NaN,Smith,05/24/2000,Services Started,Mukilteo,02/13/2024,"Smith,Zachary"
459,Zania,NaN,Roberts,12/29/1992,Services Started,Auburn,08/21/2023,"Roberts,Zania"


In [565]:
goal2 = goal.merge(peepsmerge,how='right',left_on='Individual',right_on='people_Full Name')
goal3 = goal2.merge(goalserv,how='right',left_on='Individual',right_on='full')
goal3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2807 entries, 0 to 2806
Data columns (total 36 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   Start Date_goal                               2744 non-null   datetime64[ns]
 1   Goal_goal                                     2750 non-null   object        
 2   Action Steps_goal                             0 non-null      float64       
 3   Goal Summary_goal                             507 non-null    object        
 4   Created_goal                                  2752 non-null   datetime64[ns]
 5   Created By_goal                               2752 non-null   object        
 6   Modified_goal                                 2752 non-null   datetime64[ns]
 7   Modified By_goal                              2752 non-null   object        
 8   Client_goal                                   2752 non-null   object

In [568]:
goal3['associated service'] = goal3['Status']+' - '+goal3['Location']+' - Enrolled: '+goal3['Date Services Starting']
goal3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2807 entries, 0 to 2806
Data columns (total 37 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   Start Date_goal                               2744 non-null   datetime64[ns]
 1   Goal_goal                                     2750 non-null   object        
 2   Action Steps_goal                             0 non-null      float64       
 3   Goal Summary_goal                             507 non-null    object        
 4   Created_goal                                  2752 non-null   datetime64[ns]
 5   Created By_goal                               2752 non-null   object        
 6   Modified_goal                                 2752 non-null   datetime64[ns]
 7   Modified By_goal                              2752 non-null   object        
 8   Client_goal                                   2752 non-null   object

In [573]:
goal3['Actual goal completion'] = goal3['Completion Date_goal'].combine_first(goal3['Discontinuation Date_goal'])

In [661]:
goal3['goal end reason'] = np.where(goal3['Completion Date_goal'].notna(),'Goal Completed',
                            np.where(goal3['Discontinuation Date_goal'].notna(),'Goal closed - Incomplete',
                                    None))
goal3['goal category'] = None
goal3['goal category'] = goal3['goal category'].fillna('Other Goal Not Listed')
goal3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2807 entries, 0 to 2806
Data columns (total 40 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   Start Date_goal                               2744 non-null   datetime64[ns]
 1   Goal_goal                                     2750 non-null   object        
 2   Action Steps_goal                             0 non-null      float64       
 3   Goal Summary_goal                             507 non-null    object        
 4   Created_goal                                  2752 non-null   datetime64[ns]
 5   Created By_goal                               2752 non-null   object        
 6   Modified_goal                                 2752 non-null   datetime64[ns]
 7   Modified By_goal                              2752 non-null   object        
 8   Client_goal                                   2752 non-null   object

In [662]:
goal4 = goal3[[
    'First',
    'Middle',
    'Last',
    'Date of Birth',
    'associated service',
    'Start Date_goal',
    'Projected Completion Date_goal',
    'Actual goal completion',
    'goal end reason',
    'goal category',
    'Goal_goal',
    'Location'
]].copy()

goal4['Location'] = goal4['Location'].replace('Auburn',1174)
goal4['Location'] = goal4['Location'].replace('Mukilteo',2870)
goal4.head(5)

,First,Middle,Last,Date of Birth,associated service,Start Date_goal,Projected Completion Date_goal,Actual goal completion,goal end reason,goal category,Goal_goal,Location
0,Aaron,NaN,Erickson,03/10/1963,Services Started - Auburn - Enrolled: 08/04/2023,2023-08-04,2023-10-05,2023-10-16,Goal Completed,Other Goal Not Listed,"to explore a job path based on abilities, inte...",1174
1,Aaron,NaN,Erickson,03/10/1963,Services Started - Auburn - Enrolled: 08/04/2023,2023-08-04,2023-10-05,NaT,None,Other Goal Not Listed,secure independent housing,1174
2,Aaron,NaN,Erickson,03/10/1963,Services Started - Auburn - Enrolled: 08/04/2023,2023-08-04,2023-10-05,NaT,None,Other Goal Not Listed,exhibit ability to learn and execute tasks,1174
3,Aaron,NaN,Erickson,03/10/1963,Services Started - Auburn - Enrolled: 08/04/2023,2023-10-16,2023-10-31,2023-12-15,Goal closed - Incomplete,Other Goal Not Listed,Update resume,1174
4,Aaron,NaN,Erickson,03/10/1963,Services Started - Auburn - Enrolled: 08/04/2023,2023-10-16,2023-11-16,2023-12-15,Goal closed - Incomplete,Other Goal Not Listed,The client will complete interview preparation,1174


In [674]:
goal4['goal category'] = np.where(goal4['Goal_goal'].str.contains('assess|exhibit|work readiness|increase ability|report income|housing|demonstrate|evaluate|explore|mental health|sobriety|explore a job path|client interests|ptsd|adhd|anxiety|bi-polar|depression|epilepsy|schizo|IOP|legal|relapse|car|driv|license|math',case=False),'Assessment',
                            np.where(goal4['Goal_goal'].str.contains('JD|interview|resume|obtain a job|find a job|obtain employment|obtain competitive employment|full time|full-time|part time|part-time|job search|elevator|develop|intern',case=False),'Job Development',
                            np.where(goal4['Goal_goal'].str.contains('maintain employment|maintain competitive employment|90 day|retain|retention|90+',case=False),'Job Retention',
                            np.where(goal4['Goal_goal'].str.contains('CIS| CI|CI |community inclusion',case=False),'Community Inclusion',
                            np.where(goal4['Goal_goal'].str.contains('EC | EC|independent',case=False),'Individual Employment',
                            np.where(goal4['Goal_goal'].str.contains('CM',case=False),'Case Management',
                            'Other Goal Not Listed'))))))
goal4

,First,Middle,Last,Date of Birth,associated service,Start Date_goal,Projected Completion Date_goal,Actual goal completion,goal end reason,goal category,Goal_goal,Location
0,Aaron,NaN,Erickson,03/10/1963,Services Started - Auburn - Enrolled: 08/04/2023,2023-08-04,2023-10-05,2023-10-16,Goal Completed,Assessment,"to explore a job path based on abilities, inte...",1174
1,Aaron,NaN,Erickson,03/10/1963,Services Started - Auburn - Enrolled: 08/04/2023,2023-08-04,2023-10-05,NaT,None,Assessment,secure independent housing,1174
2,Aaron,NaN,Erickson,03/10/1963,Services Started - Auburn - Enrolled: 08/04/2023,2023-08-04,2023-10-05,NaT,None,Assessment,exhibit ability to learn and execute tasks,1174
3,Aaron,NaN,Erickson,03/10/1963,Services Started - Auburn - Enrolled: 08/04/2023,2023-10-16,2023-10-31,2023-12-15,Goal closed - Incomplete,Job Development,Update resume,1174
4,Aaron,NaN,Erickson,03/10/1963,Services Started - Auburn - Enrolled: 08/04/2023,2023-10-16,2023-11-16,2023-12-15,Goal closed - Incomplete,Job Development,The client will complete interview preparation,1174
...,...,...,...,...,...,...,...,...,...,...,...,...
2802,Zania,NaN,Roberts,12/29/1992,Services Started - Auburn - Enrolled: 08/21/2023,2023-08-21,2023-10-20,NaT,None,Other Goal Not Listed,understand how SS benefits will impact future ...,1174
2803,Zania,NaN,Roberts,12/29/1992,Services Started - Auburn - Enrolled: 08/21/2023,2023-08-21,2023-10-20,NaT,None,Assessment,"to explore a job path based on abilities, inte...",1174
2804,Zania,NaN,Roberts,12/29/1992,Services Started - Auburn - Enrolled: 08/21/2023,2023-08-21,2023-10-20,NaT,None,Assessment,"exhibit work readiness through attendance, eff...",1174
2805,Zania,NaN,Roberts,12/29/1992,Services Started - Auburn - Enrolled: 08/21/2023,2023-08-21,2023-10-20,NaT,None,Assessment,exhibit ability to learn and execute tasks,1174


In [675]:
goal4['goal category'].value_counts()

goal category
Assessment               1941
Job Development           464
Other Goal Not Listed     179
Job Retention             121
Individual Employment      74
Case Management            16
Community Inclusion        12
Name: count, dtype: int64

In [676]:
goal5 = goal4[goal4['Goal_goal'].notna()].copy()
goal5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2750 entries, 0 to 2806
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   First                           2750 non-null   object        
 1   Middle                          0 non-null      float64       
 2   Last                            2750 non-null   object        
 3   Date of Birth                   2750 non-null   object        
 4   associated service              2750 non-null   object        
 5   Start Date_goal                 2744 non-null   datetime64[ns]
 6   Projected Completion Date_goal  1999 non-null   datetime64[ns]
 7   Actual goal completion          1300 non-null   datetime64[ns]
 8   goal end reason                 1300 non-null   object        
 9   goal category                   2750 non-null   object        
 10  Goal_goal                       2750 non-null   object        
 11  Location 

In [677]:
goal5.to_excel(r'S:\Rehab\Apricot\Migration\exports\real goals2.xlsx')

#### Service Plan

In [596]:
peepsmerge = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'people_Full Name',
]].copy()

peepsmerge['people_Full Name'] = peepsmerge['people_Full Name'].str.replace(', ',',')
peepsmerge['demo_Date of Birth'] = peepsmerge['demo_Date of Birth'].fillna('1/1/1800')

In [ ]:
goalserv = pd.read_excel(r'S:\Rehab\Apricot\Migration\exports\appricot services export.xlsx')

In [ ]:
serve = pd.read_excel(r'S:\Rehab\Apricot\Migration\exports\real enrollment to service import2.xlsx')

In [590]:
subserve = pd.read_excel(r'S:\Rehab\Apricot\Migration\exports\appricot subservice funder export.xlsx')

In [594]:
service = pd.read_excel(r'S:\Rehab\Apricot\Migration\Services - unformatted.xlsx')

# split out the contents of the client column on the ;# and add the split values to the newly created individual column
service[['Individual','Individual2']] = service['Individual'].str.split(';#', expand=True)

# sort the individual column values alphabetically
service = service.sort_values('Individual2',ascending=True)

# the key value had a space in it, so needed to remove that space
service['Individual2'] = service['Individual2'].str.replace(', ',',')

# remove test case rows
service = service[~service['Individual2'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
service['Individual2'] = service['Individual2'].replace(r'\(.*?\)','',regex=True).str.strip()

service['Individual2'] = service['Individual2'].replace(r'\".*?\"','',regex=True).str.strip()

# service['Individual'] = service['Individual2']
# service[['first','last']] = None
service[['last','first','other']] = service['Individual2'].str.split(',', expand=True)
service

,Start Date,Individual,Venue,End Date,Service,Recommendations,Hours Allocated Per Month (DDA Only),Modified By,Content Type,Created,Created By,Folder Child Count,ID,Item Child Count,Modified,Notes,Projected End Date,Title,Item Type,Path,Individual2,last,first,other
313,2019-02-04,658,Auburn - Office Skills & Customer Service,2019-06-13 00:00:00,Assessment,NaN,NaN,Jason Person,Services,2019-02-01 08:57:44,System Account,0,367,0,2019-07-24 15:29:12,Completed school program. Plans to go to colle...,2019-06-14,View,Item,Lists/Services,"Abduwahab,Besharo",Abduwahab,Besharo,None
337,2018-12-17,512,Auburn - Community Based Services,2019-03-17 00:00:00,Job Retention,NaN,NaN,Lavada Kindle,Services,2019-02-06 08:01:03,Lavada Kindle,0,394,0,2019-03-29 10:29:10,Clt has rehabbed- did not return phone or emai...,2019-03-17,View,Item,Lists/Services,"Abraham,Noah",Abraham,Noah,None
168,2018-11-07,512,Auburn - Community Based Services,2018-12-17 00:00:00,Job Development,NaN,NaN,Lavada Kindle,Services,2018-12-05 10:16:42,Jason Person,0,214,0,2019-02-06 08:10:18,NaN,NaT,View,Item,Lists/Services,"Abraham,Noah",Abraham,Noah,None
25,2018-08-17,512,Auburn - Manufacturing Program,2018-11-02 00:00:00,Community Based Assessment,NaN,NaN,Kathy Powers,Services,2018-09-13 10:20:48,System Account,0,70,0,2018-12-27 15:41:36,NaN,NaT,View,Item,Lists/Services,"Abraham,Noah",Abraham,Noah,None
23,2018-08-20,270,Auburn - Custodial,2018-05-09 00:00:00,Assessment,NaN,NaN,Kathy Powers,Services,2018-08-20 16:51:20,Kathy Powers,0,68,0,2018-08-20 16:51:20,NaN,NaT,View,Item,Lists/Services,"Acosta,Rebecca",Acosta,Rebecca,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1554,2022-04-15,1263,Auburn - Manufacturing Program,2022-08-04 00:00:00,Assessment,NaN,NaN,Tricia Cleavelin,Services,2022-04-15 10:53:17,System Account,0,1668,0,2022-08-22 08:49:52,start of job development,2022-06-17,View,Item,Lists/Services,"Zepeda,Manuel",Zepeda,Manuel,None
2087,2023-08-28,1565,Mukilteo - Manufacturing Program,2023-08-31 00:00:00,NaN,Recommend a routine that ensures sleep and wor...,NaN,Gabby Duran,Services,2023-08-29 10:39:36,System Account,0,2215,0,2023-09-06 13:34:30,NaN,NaT,View,Item,Lists/Services,"Zhang,Edwin",Zhang,Edwin,None
2003,2023-06-26,1519,Auburn - Office Skills & Customer Service,2023-11-02 00:00:00,Training,NaN,NaN,Lew Humiston,Services,2023-06-23 15:09:38,System Account,0,2128,0,2023-11-02 16:39:51,NaN,2023-09-29,View,Item,Lists/Services,"Zhang,Yuqing",Zhang,Yuqing,None
2179,2023-10-18,1519,Auburn - Community Based Services,2023-11-02 00:00:00,Job Development,NaN,NaN,Lew Humiston,Services,2023-10-18 09:58:21,Lew Humiston,0,2308,0,2024-03-07 08:00:30,Obtained competitive employment,2024-01-18,View,Item,Lists/Services,"Zhang,Yuqing",Zhang,Yuqing,None


In [732]:
servicemergex = service2.merge(subserve,how='left',left_on=['first','last'],right_on=['First','Last'])
servicemergex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2061 entries, 0 to 2060
Data columns (total 42 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Start Date_x                          2056 non-null   datetime64[ns]
 1   Individual                            2056 non-null   object        
 2   Venue                                 2056 non-null   object        
 3   End Date                              1684 non-null   object        
 4   Service_x                             2026 non-null   object        
 5   Recommendations                       235 non-null    object        
 6   Hours Allocated Per Month (DDA Only)  177 non-null    float64       
 7   Modified By                           2056 non-null   object        
 8   Content Type                          2056 non-null   object        
 9   Created                               2056 non-null   datetime64[ns]
 10  

In [734]:
servicemergex.to_excel(r'S:\Rehab\Apricot\Migration\exports\service plan miss test.xlsx')

In [601]:
servicemerge = service2.merge(subserve,how='right',left_on=['first','last'],right_on=['First','Last'])
servicemerge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1826 entries, 0 to 1825
Data columns (total 42 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Start Date_x                          1812 non-null   datetime64[ns]
 1   Individual                            1812 non-null   object        
 2   Venue                                 1812 non-null   object        
 3   End Date                              1509 non-null   object        
 4   Service_x                             1789 non-null   object        
 5   Recommendations                       213 non-null    object        
 6   Hours Allocated Per Month (DDA Only)  151 non-null    float64       
 7   Modified By                           1812 non-null   object        
 8   Content Type                          1812 non-null   object        
 9   Created                               1812 non-null   datetime64[ns]
 10  

In [599]:
service2 = service.merge(peepsmerge,how='right',left_on='Individual2',right_on='people_Full Name')

In [600]:
service2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1069 entries, 0 to 1068
Data columns (total 28 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Start Date                            1064 non-null   datetime64[ns]
 1   Individual                            1064 non-null   object        
 2   Venue                                 1064 non-null   object        
 3   End Date                              844 non-null    object        
 4   Service                               1046 non-null   object        
 5   Recommendations                       133 non-null    object        
 6   Hours Allocated Per Month (DDA Only)  89 non-null     float64       
 7   Modified By                           1064 non-null   object        
 8   Content Type                          1064 non-null   object        
 9   Created                               1064 non-null   datetime64[ns]
 10  

In [601]:
servicemerge = service2.merge(subserve,how='right',left_on=['first','last'],right_on=['First','Last'])
servicemerge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1826 entries, 0 to 1825
Data columns (total 42 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Start Date_x                          1812 non-null   datetime64[ns]
 1   Individual                            1812 non-null   object        
 2   Venue                                 1812 non-null   object        
 3   End Date                              1509 non-null   object        
 4   Service_x                             1789 non-null   object        
 5   Recommendations                       213 non-null    object        
 6   Hours Allocated Per Month (DDA Only)  151 non-null    float64       
 7   Modified By                           1812 non-null   object        
 8   Content Type                          1812 non-null   object        
 9   Created                               1812 non-null   datetime64[ns]
 10  

In [603]:
# servicemerge.to_excel(r'S:\Rehab\Apricot\Migration\exports\service plan export.xlsx')

In [614]:
servicemerge2 = servicemerge[[
    'First',
    'Middle',
    'Last',
    'Date of Birth',
    'associated service',
    'sub service',
    'Service_y',
    'Start Date_y',
    'Projected End Date',
    'End Date',
    'Location'
]].copy()

servicemerge2['enrollmentID'] = range(1,len(servicemerge2)+1)

In [615]:
servicemerge2['Service_y'].value_counts()

Service_y
Assessment            671
Job Development       513
Job Retention         313
DDA Services          179
Case Management        82
Training               64
Independent Living      3
Staffing                1
Name: count, dtype: int64

In [617]:
servicemerge2.to_excel(r'S:\Rehab\Apricot\Migration\exports\service plan export.xlsx')

### Employment placement record

In [618]:
peepsmerge = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'people_Full Name',
]].copy()

peepsmerge['people_Full Name'] = peepsmerge['people_Full Name'].str.replace(', ',',')
peepsmerge['demo_Date of Birth'] = peepsmerge['demo_Date of Birth'].fillna('1/1/1800')

In [619]:
placement = pd.read_excel(r'S:\Rehab\Apricot\Migration\Employer placement record - unformatted.xlsx')

placement[['Individal','Individual2']] = placement['Individual'].str.split(';#', expand=True)

# sort the individual column values alphabetically
placement = placement.sort_values('Individual2',ascending=True)

# the key value had a space in it, so needed to remove that space
placement['Individual2'] = placement['Individual2'].str.replace(', ',',')

# remove test case rows
placement = placement[~placement['Individual2'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
placement['Individual2'] = placement['Individual2'].replace(r'\(.*?\)','',regex=True).str.strip()

placement['Individual2'] = placement['Individual2'].replace(r'\".*?\"','',regex=True).str.strip()

# remove unnecessary columns
placement = placement.drop(columns=['Content Type','Created By','Item Type','Path'])

In [621]:
placement2 = placement.merge(peepsmerge,how='right',left_on='Individual2',right_on='people_Full Name')
placement2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 658 entries, 0 to 657
Data columns (total 29 columns):
 #   Column                                                       Non-Null Count  Dtype         
---  ------                                                       --------------  -----         
 0   Title                                                        347 non-null    object        
 1   Placement Date                                               347 non-null    datetime64[ns]
 2   Orion Staff                                                  347 non-null    object        
 3   Individual                                                   347 non-null    object        
 4   Placement Type                                               344 non-null    object        
 5   Employer                                                     345 non-null    object        
 6   Employer:Company Name                                        345 non-null    object        
 7   Employer Contact 

In [623]:
placement3 = placement2[placement2['Individual'].notna()].copy()

In [624]:
placement3['Placement Type'].value_counts()

Placement Type
Manufacturing - Assembly                         55
Food Service / Restaurant                        48
Customer Service / Retail                        44
Manufacturing - Production                       41
Custodial / Janitorial / Maintenance             29
Manufacturing - Machine Operation / Machining    25
Warehouse                                        22
Administrative / Clerical                        16
Hospitality                                      11
Driver / Shipping & Recieving                    10
Manufacturing - Finish / Paint                    8
Medical (PCA, CNA, Dietary Aide)                  7
Construction / Trade                              7
Other (try to fit in other category)              7
Manufacturing - Composite                         6
Education                                         4
Manufacturing - Quality                           3
Management                                        1
Name: count, dtype: int64

In [628]:
placement3['Placement Type'] = placement3['Placement Type'].str.replace('Warehouse','Warehouse / Shipping & Receiving')
placement3['Placement Type'] = placement3['Placement Type'].str.replace('Customer contact / Retail','Retail / Retail Sales')
placement3['Placement Type'] = placement3['Placement Type'].str.replace('Administrative / Clerical','Administrative Support Occupations')
placement3['Placement Type'] = placement3['Placement Type'].str.replace('Driver / Shipping & Recieving','Driver')
placement3['Placement Type'] = placement3['Placement Type'].str.replace('Other (try to fit in other category)','Other Occupations (try to fit in other category)')
placement3['Placement Type'] = placement3['Placement Type'].str.replace('Hospitality','Customer contact (e.g. greeter, call center)')
placement3['Placement Type'] = placement3['Placement Type'].str.replace('Medical (PCA, CNA, Dietary Aide','Medical/Health Care Occupations (e.g. lab tech)')
placement3['Placement Type'] = placement3['Placement Type'].str.replace('Education','Education (e.g. paraeducator, daycare)')


In [640]:
placement4 = placement3[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'Placement Date',
    'End Date',
    'Employer:Company Name',
    'Competitive Employment: Made 90 days?',
    'Customized Employment:',
    'Customized Employment: Wage',
    'Customized Employment: Wage End Date',
    'Customized Employment: Stability Date',
    'Customized Employment: Check if update of current placement'
]].copy()
placement4['customized details'] = placement['Customized Employment: Wage'].astype(str)+'|'+placement4['Customized Employment: Wage End Date'].astype(str)+'|'+placement4['Customized Employment: Stability Date'].astype(str)

In [627]:
# placementmerge = placement.merge(peeps3,how='left',left_on='Individual2',right_on='Individual')
# placementmerge

In [679]:
placement4.to_excel(r'S:\Rehab\Apricot\Migration\exports\real employment placement2.xlsx')

In [680]:
employment = placement3[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'Placement Date',
    'Placement Type',
    'Job Title',
    'Customized Employment: Wage',
    'Competitive Employment: Wage at Placement',
    'Hours worked per week?',
    'Benefits at closure?',
    'Employer:Company Name'
]].copy()

In [681]:
employment.to_excel(r'S:\Rehab\Apricot\Migration\exports\real employment details3.xlsx')

In [ ]:
ddabill = [[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
]]

### Contacts

In [472]:
contact = pd.read_excel(r'S:\Rehab\Apricot\Migration\Relationships (contacts) - unformatted.xlsx')

# sort the individual column values alphabetically
contact = contact.sort_values('Individual',ascending=True).reset_index()

# the key value had a space in it, so needed to remove that space
contact['Individual'] = contact['Individual'].str.replace(' ','')

contact = contact.drop(columns=['View','Item Type','Path','index','Content Type','Folder Child Count','Item Child Count','Modified By','Modified'])

# remove test case rows
contact = contact[~contact['Individual'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
contact['Individual'] = contact['Individual'].replace(r'\(.*?\)','',regex=True).str.strip()

contact['Individual'] = contact['Individual'].replace(r'\".*?\"','',regex=True).str.strip()
contact

,Individual,Contact,Relationship to Individual,Created,Created By,ID
0,"Adams,Melissa","Woods, Kevin",Emergency Contact;#Family Member,2023-07-19 12:17:49,Traci Dosch,181
1,"Adams,Melissa","Habtyimer, Mulu",Emergency Contact;#Residential Provider,2023-07-19 12:08:24,Traci Dosch,179
2,"Adams,Melissa","Parsons, Tammy",Counselor,2023-07-19 12:06:51,Traci Dosch,178
3,"Adams,Melissa","Woods, Jane",Emergency Contact;#Family Member,2023-07-19 12:09:23,Traci Dosch,180
4,"Adamson,Kyle","Ricker, Stacy",Counselor,2020-09-18 12:24:14,Kathy Powers,63
...,...,...,...,...,...,...
152,"Waltjen,Melissa","Wolderufael, Endrias",Residential Provider,2023-02-25 18:45:42,Traci Dosch,169
153,"Waltjen,Melissa","Yip, Dr. Virginia",Advocate,2023-02-25 18:50:52,Traci Dosch,170
154,"Wicken-Arnot,Joshua","Helminak, Lara",Emergency Contact;#Family Member,2021-09-03 14:24:31,Traci Dosch,127
155,"Wicken-Arnot,Joshua","Arnot, Heather",Emergency Contact;#Family Member,2021-09-03 14:23:29,Traci Dosch,126


In [473]:
contact[['contact last','contact first']] = contact['Contact'].str.split(', ', expand=True)

In [520]:
peepsmerge = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'people_Full Name',
    'intake_Orion Location'
]].copy()

peepsmerge['people_Full Name'] = peepsmerge['people_Full Name'].str.replace(', ',',')

peepsmerge['intake_Orion Location'] = peepsmerge['intake_Orion Location'].replace('Auburn',1174)
peepsmerge['intake_Orion Location'] = peepsmerge['intake_Orion Location'].replace('Mukilteo',2870)

In [521]:
contactmerge = contact.merge(peepsmerge,how='right',left_on='Individual',right_on='people_Full Name')
contactmerge

,Individual,Contact,Relationship to Individual,Created,Created By,ID,contact last,contact first,people_First Name,people_Last Name,demo_Date of Birth,people_Full Name,intake_Orion Location
0,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,Keith,Shuman,1970-05-22 00:00:00,"Shuman,Keith",NaN
1,"Waltjen,Melissa","Abonyo, Stephen",Case Manager,2024-06-24 08:23:11,Jen Tabiando,215.0,Abonyo,Stephen,Melissa,Waltjen,1977-10-06 00:00:00,"Waltjen,Melissa",NaN
2,"Waltjen,Melissa","Gugel, Sarah",Counselor,2023-02-25 18:38:31,Traci Dosch,166.0,Gugel,Sarah,Melissa,Waltjen,1977-10-06 00:00:00,"Waltjen,Melissa",NaN
3,"Waltjen,Melissa","Carmen, Jill",Emergency Contact,2023-02-25 18:44:10,Traci Dosch,168.0,Carmen,Jill,Melissa,Waltjen,1977-10-06 00:00:00,"Waltjen,Melissa",NaN
4,"Waltjen,Melissa","Welch, Lisa",Counselor,2023-02-25 18:39:15,Traci Dosch,167.0,Welch,Lisa,Melissa,Waltjen,1977-10-06 00:00:00,"Waltjen,Melissa",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,Maxine,Ghansah,1989-09-02 00:00:00,"Ghansah,Maxine",2870.0
580,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,Steven,Mason,1966-04-19 00:00:00,"Mason,Steven",2870.0
581,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,Savannah,Phillips,1997-05-19 00:00:00,"Phillips,Savannah",1174.0
582,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,George,Johnston,1975-08-14 00:00:00,"Johnston,George",2870.0


In [522]:
contactmerge2 = contactmerge[contactmerge['Individual'].notna()].copy()
contactmerge2

,Individual,Contact,Relationship to Individual,Created,Created By,ID,contact last,contact first,people_First Name,people_Last Name,demo_Date of Birth,people_Full Name,intake_Orion Location
1,"Waltjen,Melissa","Abonyo, Stephen",Case Manager,2024-06-24 08:23:11,Jen Tabiando,215.0,Abonyo,Stephen,Melissa,Waltjen,1977-10-06 00:00:00,"Waltjen,Melissa",NaN
2,"Waltjen,Melissa","Gugel, Sarah",Counselor,2023-02-25 18:38:31,Traci Dosch,166.0,Gugel,Sarah,Melissa,Waltjen,1977-10-06 00:00:00,"Waltjen,Melissa",NaN
3,"Waltjen,Melissa","Carmen, Jill",Emergency Contact,2023-02-25 18:44:10,Traci Dosch,168.0,Carmen,Jill,Melissa,Waltjen,1977-10-06 00:00:00,"Waltjen,Melissa",NaN
4,"Waltjen,Melissa","Welch, Lisa",Counselor,2023-02-25 18:39:15,Traci Dosch,167.0,Welch,Lisa,Melissa,Waltjen,1977-10-06 00:00:00,"Waltjen,Melissa",NaN
5,"Waltjen,Melissa","Wolderufael, Endrias",Residential Provider,2023-02-25 18:45:42,Traci Dosch,169.0,Wolderufael,Endrias,Melissa,Waltjen,1977-10-06 00:00:00,"Waltjen,Melissa",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,"Teal,Radthmony","Teal, Tammy",Emergency Contact;#Family Member,2021-11-17 08:27:46,Traci Dosch,137.0,Teal,Tammy,Radthmony,Teal,1997-11-24 00:00:00,"Teal,Radthmony",2870.0
508,"Teal,Radthmony","Laura, Langerstedt",Counselor,2021-11-17 08:22:57,Traci Dosch,133.0,Laura,Langerstedt,Radthmony,Teal,1997-11-24 00:00:00,"Teal,Radthmony",2870.0
509,"Teal,Radthmony","Weikum, Jennifer",Counselor,2021-11-17 08:24:11,Traci Dosch,134.0,Weikum,Jennifer,Radthmony,Teal,1997-11-24 00:00:00,"Teal,Radthmony",2870.0
539,"Shim,Justin",NaN,Guardian,2022-07-06 09:19:27,System Account,146.0,NaN,NaN,Justin,Shim,2003-07-16 00:00:00,"Shim,Justin",1174.0


In [523]:
profile = pd.read_excel(r'S:\Rehab\Apricot\Migration\profiles - unformatted.xlsx')

In [524]:
conpro = contactmerge.merge(profile,how='left',left_on='Contact',right_on='Person')

conpro2 = conpro[conpro['Individual'].notna()].copy()
conpro2

,Individual,Contact,Relationship to Individual,Created_x,Created By_x,ID_x,contact last,contact first,people_First Name,people_Last Name,demo_Date of Birth,people_Full Name,intake_Orion Location,ID_y,Person,Content Type,Programs,Organization,State,City,ZIP Code,Active,Address,Anonymous,Became Donor On,Board Position,Case Worker,Charitable Interest,Checkbox Column 1,Checkbox Column 2,Choice Column 1,Choice Column 2,Choice Column 3,Choice Column 4,Choice Column 5,Comments,Comments Column 1,Comments Column 2,Country,Created_y,Created By_y,Date Column 1,Date Column 2,Donor Motivation,E-Mail,Email Lists,Folder Child Count,Item Child Count,Job Title,Link Column 1,Link Column 2,Mailing Address,Mailing City,Mailing Country,Mailing Lists,Mailing State,Mailing ZIP Code,Membership Type,Mobile Phone,Modified,Modified By,Number Column 1,Number Column 2,Permissions Set,Personal Interest,Phone,Phone 2,Photo,Professional Type,Residence,Set Show Programs and Residences - disabled,Show Programs,Show Residence,Text Column 1,Text Column 3,Text Column 4,Text Column 5,View,Volunteer Position,Web Page,Item Type,Path
3,"Waltjen,Melissa","Abonyo, Stephen",Case Manager,2024-06-24 08:23:11,Jen Tabiando,215.0,Abonyo,Stephen,Melissa,Waltjen,1977-10-06 00:00:00,"Waltjen,Melissa",NaN,1725.0,"Abonyo, Stephen",Professional,NaN,DDA-Snohomish,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-24 08:23:07,Jen Tabiando,NaN,NaN,NaN,stephen.abonyo@dshs.wa.gov,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-24 08:23:07,Jen Tabiando,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,View,NaN,NaN,Item,Lists/Profiles
4,"Waltjen,Melissa","Gugel, Sarah",Counselor,2023-02-25 18:38:31,Traci Dosch,166.0,Gugel,Sarah,Melissa,Waltjen,1977-10-06 00:00:00,"Waltjen,Melissa",NaN,1360.0,"Gugel, Sarah",Professional,NaN,DVR-Arlington,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-25 18:38:07,Traci Dosch,NaN,NaN,NaN,sarah.grugel@dshs.wa.gov,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,360-506-1791,2023-02-25 18:38:07,Traci Dosch,NaN,NaN,NaN,NaN,360-651-6120,NaN,NaN,Vocational Rehabilitation Counselor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,View,NaN,NaN,Item,Lists/Profiles
5,"Waltjen,Melissa","Carmen, Jill",Emergency Contact,2023-02-25 18:44:10,Traci Dosch,168.0,Carmen,Jill,Melissa,Waltjen,1977-10-06 00:00:00,"Waltjen,Melissa",NaN,1361.0,"Carmen, Jill",General Contact,NaN,NaN,CA,Chula Vista,91911-5569,True,1454 Melrose Ave. #1156\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USA,2023-02-25 18:43:59,Traci Dosch,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,1454 Melrose Ave #1156,Chula Vista,USA,NaN,CA,91911-5569,NaN,NaN,2023-02-25 18:43:59,Traci Dosch,NaN,NaN,NaN,NaN,619-488-4027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,View,NaN,NaN,Item,Lists/Profiles
6,"Waltjen,Melissa","Welch, Lisa",Counselor,2023-02-25 18:39:15,Traci Dosch,167.0,Welch,Lisa,Melissa,Waltjen,1977-10-06 00:00:00,"Waltjen,Melissa",NaN,1243.0,"Welch, Lisa",Professional,NaN,DDA-Snohomish,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-13 10:11:38,Diane Shepard,NaN,NaN,NaN,lisa.welch@dshs.wa.gov,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-25 18:41:48,Traci Dosch,NaN,NaN,NaN,NaN,(206) 402-2742,NaN,NaN,Referral Contact,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,View,NaN,NaN,Item,Lists/Profiles
7,"Waltjen,Melissa","Wolderufael, Endrias",Residential Provider,2023-02-25 18:45:42,Traci Dosch,169.0,Wolderufael,Endrias,Melissa,Waltjen,1977-10-06 00:00:00,"Waltjen,Melissa",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [525]:
conpro2.columns

Index(['Individual', 'Contact', 'Relationship to Individual', 'Created_x',
       'Created By_x', 'ID_x', 'contact last', 'contact first',
       'people_First Name', 'people_Last Name', 'demo_Date of Birth',
       'people_Full Name', 'intake_Orion Location', 'ID_y', 'Person',
       'Content Type', 'Programs', 'Organization', 'State', 'City', 'ZIP Code',
       'Active', 'Address', 'Anonymous', 'Became Donor On', 'Board Position',
       'Case Worker', 'Charitable Interest', 'Checkbox Column 1',
       'Checkbox Column 2', 'Choice Column 1', 'Choice Column 2',
       'Choice Column 3', 'Choice Column 4', 'Choice Column 5', 'Comments',
       'Comments Column 1', 'Comments Column 2', 'Country', 'Created_y',
       'Created By_y', 'Date Column 1', 'Date Column 2', 'Donor Motivation',
       'E-Mail', 'Email Lists', 'Folder Child Count', 'Item Child Count',
       'Job Title', 'Link Column 1', 'Link Column 2', 'Mailing Address',
       'Mailing City', 'Mailing Country', 'Mailing Lists',

In [526]:
conpro3 = conpro2[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'contact first',
    'contact last',
    'Relationship to Individual',
    'E-Mail',
    'Phone',
    'Address',
    'City',
    'State',
    'ZIP Code',
    'Professional Type',
    'intake_Orion Location'
]].copy()

In [527]:
conpro3['Active'] = None
conpro3['Active'] = conpro3['Active'].fillna('Active')

In [528]:
conpro3['Emergency'] = np.where(conpro3['Relationship to Individual'].str.contains('Emergency'),'Yes','No')
conpro3['Guardian'] = np.where(conpro3['Relationship to Individual'].str.contains('Guardian'),'Yes','No')
conpro3['Preferred contact'] = np.where(conpro3['Phone'].notna(),'Primary Phone','Address')

In [529]:
conpro3['Relationship to Individual'].value_counts()

Relationship to Individual
Guardian                                                20
Emergency Contact;#Family Member                        19
Counselor                                               19
Case Manager                                            13
Emergency Contact;#Family Member;#Guardian              12
Advocate                                                 8
Family Member                                            4
Residential Provider                                     4
Emergency Contact                                        3
Counselor;#Advocate                                      3
Family Member;#Guardian                                  2
Emergency Contact;#Guardian                              2
Emergency Contact;#Residential Provider                  1
Emergency Contact;#Family Member;#Former Guardian        1
Advocate;#Residential Provider                           1
Former Counselor                                         1
Emergency Contact;#Advocate;#

In [530]:
conpro3['Relationship to Individual'] = conpro3['Relationship to Individual'].str.replace('Emergency Contact','',regex=True)
conpro3['Relationship to Individual'] = conpro3['Relationship to Individual'].str.replace(';#','|',regex=True)
conpro3['Relationship to Individual'] = conpro3['Relationship to Individual'].str.replace('Guardian','Advocate',regex=True)
conpro3['Relationship to Individual'] = conpro3['Relationship to Individual'].str.strip('|')
conpro3['Relationship to Individual'] = conpro3['Relationship to Individual'].str.replace(r'\|.*','',regex=True)
conpro3['Relationship to Individual'] = conpro3['Relationship to Individual'].str.replace('Former Counselor','Advocate',regex=True)
conpro3['Relationship to Individual'] = conpro3['Relationship to Individual'].fillna('Advocate')

In [531]:
conpro3.to_excel(r'S:\Rehab\Apricot\Migration\exports\real contacts2.xlsx')

#### Emergency Contacts

In [82]:
cont = pd.read_excel(r'S:\Rehab\Apricot\Migration\Client information (demographics and assessment) - unformatted.xlsx')

# split out the contents of the client column on the ;# and add the split values to the newly created individual column
# cont[['Individual','Individual2']] = cont['Individual'].str.split(';#', expand=True)

# sort the individual column values alphabetically
cont = cont.sort_values('Individual',ascending=True)

# the key value had a space in it, so needed to remove that space
cont['Individual'] = cont['Individual'].str.replace(', ',',')

# remove test case rows
cont = cont[~cont['Individual'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
cont['Individual'] = cont['Individual'].replace(r'\(.*?\)','',regex=True).str.strip()

cont['Individual'] = cont['Individual'].replace(r'\".*?\"','',regex=True).str.strip()
cont

,ID,Individual,Preferred Names,Date of Birth,Gender,Disability - Primary,Disabilities - Additional,Disability / Barrier Notes:,Date of Last Employment,Smoker?,...,Telephone Number,Transportation,Veteran Status,View,Visual/Hearing Aids,Weight,Work Restrictions,Work Restrictions Details,Item Type,Path
321,365,"Abduwahab,Besharo",Besharo,2000-08-30 00:00:00,Female,NaN,NaN,No information provided.,NaT,No,...,NaN,School Bus,NaN,View,NaN,NaN,NaN,No physical restrictions.,Item,Lists/Individuals Information
246,288,"Abraham,Noah",NaN,1972-04-02 00:00:00,Male,Ortho - Leg / Foot,Med - Other;#18,NaN,2014-08-01,No,...,NaN,Car,NaN,View,NaN,NaN,NaN,Left leg prosthesis - please indicate difficul...,Item,Lists/Individuals Information
3,43,"Acosta,Rebecca",Rebecca,1979-05-01 00:00:00,Female,Psych - PTSD,Neuro - Other;#24;#Substance Abuse;#38;#Neuro ...,NaN,NaT,NaN,...,NaN,Public Transit,NaN,View,NaN,NaN,Lift over 25 lbs;#2;#English Reading Proficien...,NaN,Item,Lists/Individuals Information
918,962,"Acteson,Jnean",NaN,1980-05-22 00:00:00,Female,Psych - Bi-Polar,Neuro - Other;#24;#Substance Abuse;#38;#Psych ...,"clt. reports bi-polar (and depression), anxiet...",2022-04-15,No,...,NaN,Public Transit,NaN,View,NaN,NaN,NaN,No physical restrictions reported. Please note...,Item,Lists/Individuals Information
245,286,"Adair,Chris",Chris,1995-11-26 00:00:00,Male,Dev - Autism,NaN,NaN,NaT,NaN,...,NaN,Other,NaN,View,NaN,NaN,NaN,NaN,Item,Lists/Individuals Information
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1361,1405,NaN,Alan Masta,1982-04-05 00:00:00,Male,Med - Other,NaN,Heat Intolerance - can not be in direct sunlig...,2020-01-01,No,...,NaN,Car,"Current Active Duty, Guard, Reserves or Veteran",View,NaN,NaN,NaN,"no repeated heavy lifting, needs to be indoors...",Item,Lists/Individuals Information
1362,1406,NaN,Alan Masta,1982-04-05 00:00:00,Male,Med - Other,NaN,Heat Sensitivity,2020-05-01,No,...,NaN,Car,"Current Active Duty, Guard, Reserves or Veteran",View,NaN,NaN,Lift over 25 lbs;#2,"No repeated heavy lifting, no outdoor work, mu...",Item,Lists/Individuals Information
1375,1419,NaN,Isabela,1992-04-07 00:00:00,Female,NaN,NaN,NaN,2024-02-01,NaN,...,NaN,Car,NaN,View,NaN,NaN,NaN,NaN,Item,Lists/Individuals Information
1401,1445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,View,NaN,NaN,NaN,NaN,Item,Lists/Individuals Information


In [78]:
peepsmerge = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'people_Full Name',
]].copy()

peepsmerge['people_Full Name'] = peepsmerge['people_Full Name'].str.replace(', ',',')
peepsmerge['demo_Date of Birth'] = peepsmerge['demo_Date of Birth'].fillna('1/1/1800')

In [83]:
contmerge = cont.merge(peepsmerge,how='right',left_on='Individual',right_on='people_Full Name')
contmerge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535 entries, 0 to 534
Data columns (total 92 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   ID                                        520 non-null    float64       
 1   Individual                                520 non-null    object        
 2   Preferred Names                           144 non-null    object        
 3   Date of Birth                             486 non-null    object        
 4   Gender                                    512 non-null    object        
 5   Disability - Primary                      414 non-null    object        
 6   Disabilities - Additional                 317 non-null    object        
 7   Disability / Barrier Notes:               357 non-null    object        
 8   Date of Last Employment                   362 non-null    datetime64[ns]
 9   Smoker?                         

In [84]:
emergency = contmerge[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'Emergency Contact Name',
    'Emergency Contact Number',
    'Emergency Contact Relationship'
]]

In [85]:
missing = pd.read_excel(r'S:\Rehab\Apricot\Migration\exports\apricot missing contacts.xlsx')

In [87]:
missingemergency = missing.merge(emergency,how='left', left_on=['First','Last'],right_on=['people_First Name','people_Last Name'])
missingemergency.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   First                           480 non-null    object 
 1   Middle                          17 non-null     object 
 2   Last                            480 non-null    object 
 3   Date of Birth                   480 non-null    object 
 4   Record ID                       0 non-null      float64
 5   people_First Name               432 non-null    object 
 6   people_Last Name                432 non-null    object 
 7   demo_Date of Birth              432 non-null    object 
 8   Emergency Contact Name          301 non-null    object 
 9   Emergency Contact Number        297 non-null    object 
 10  Emergency Contact Relationship  297 non-null    object 
dtypes: float64(1), object(10)
memory usage: 41.4+ KB


In [88]:
missingemergency.to_excel(r'S:\Rehab\Apricot\Migration\exports\missing emergency contacts.xlsx')

### DDA sheets

In [4]:
hours = pd.read_excel(r'S:\Rehab\Apricot\Migration\Employer placement hours - unformatted.xlsx')

# split out the contents of the client column on the ;# and add the split values to the newly created individual column
hours[['Individual','Individual2']] = hours['Individual'].str.split(';#', expand=True)

# sort the individual column values alphabetically
hours = hours.sort_values('Individual2',ascending=True)

# the key value had a space in it, so needed to remove that space
hours['Individual2'] = hours['Individual2'].str.replace(', ',',')

# remove test case rows
hours = hours[~hours['Individual'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
hours['Individual'] = hours['Individual'].replace(r'\(.*?\)','',regex=True).str.strip()

hours['Individual'] = hours['Individual'].replace(r'\".*?\"','',regex=True).str.strip()
hours

,Individual,Billing Date,Hours worked this MONTH,EmployerPlacementID,Content Type,Created,Created By,Folder Child Count,ID,Item Child Count,Modified,Modified By,Title,Item Type,Path,Individual2
612,1012,2023-07-31,0.00,545.0,Item,2023-08-26 07:34:52,Traci Dosch,0,635,0,2023-08-26 07:34:52,Traci Dosch,View,Item,Lists/EmployerPlacementHours,"Akers,Laura"
173,1012,2022-02-28,48.00,347.0,Item,2022-03-01 09:56:51,Traci Dosch,0,179,0,2022-03-01 09:56:51,Traci Dosch,View,Item,Lists/EmployerPlacementHours,"Akers,Laura"
174,1012,2022-02-28,16.00,327.0,Item,2022-03-01 09:57:11,Traci Dosch,0,180,0,2022-03-01 09:57:11,Traci Dosch,View,Item,Lists/EmployerPlacementHours,"Akers,Laura"
175,1012,2022-02-28,4.00,356.0,Item,2022-03-01 09:57:36,Traci Dosch,0,181,0,2022-03-01 09:57:36,Traci Dosch,View,Item,Lists/EmployerPlacementHours,"Akers,Laura"
562,1012,2023-06-30,51.06,477.0,Item,2023-06-27 14:48:56,Traci Dosch,0,585,0,2023-06-27 14:48:56,Traci Dosch,View,Item,Lists/EmployerPlacementHours,"Akers,Laura"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,442,2022-09-27,128.00,416.0,Item,2022-09-27 08:15:50,Traci Dosch,0,337,0,2022-09-27 08:15:50,Traci Dosch,View,Item,Lists/EmployerPlacementHours,"Yao,Shaun"
391,442,2022-11-30,160.00,455.0,Item,2022-12-07 09:18:28,Traci Dosch,0,408,0,2022-12-07 09:18:28,Traci Dosch,View,Item,Lists/EmployerPlacementHours,"Yao,Shaun"
320,442,2022-08-31,184.00,416.0,Item,2022-09-06 06:35:42,Traci Dosch,0,334,0,2022-09-06 06:35:42,Traci Dosch,View,Item,Lists/EmployerPlacementHours,"Yao,Shaun"
463,442,2023-03-31,184.00,455.0,Item,2023-03-24 08:52:08,Traci Dosch,0,485,0,2023-03-24 08:52:08,Traci Dosch,View,Item,Lists/EmployerPlacementHours,"Yao,Shaun"


In [5]:
peepsmerge = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'people_Full Name',
]].copy()

peepsmerge['people_Full Name'] = peepsmerge['people_Full Name'].str.replace(', ',',')
peepsmerge['demo_Date of Birth'] = peepsmerge['demo_Date of Birth'].fillna('1/1/1800')

In [26]:
place = pd.read_excel(r'S:\Rehab\Apricot\Migration\Employer placement record2 - unformatted.xlsx')

In [27]:
hourplace = hours.merge(place,how='left',left_on='EmployerPlacementID',right_on='ID')
hourplace.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 48 columns):
 #   Column                                                       Non-Null Count  Dtype         
---  ------                                                       --------------  -----         
 0   Individual_x                                                 920 non-null    object        
 1   Billing Date                                                 912 non-null    datetime64[ns]
 2   Hours worked this MONTH                                      912 non-null    float64       
 3   EmployerPlacementID                                          919 non-null    float64       
 4   Content Type_x                                               920 non-null    object        
 5   Created_x                                                    920 non-null    datetime64[ns]
 6   Created By_x                                                 920 non-null    object        
 7   Folder Child Coun

In [29]:
hourplace2 = hourplace.merge(peepsmerge,how='left',left_on='Individual2',right_on='people_Full Name')
hourplace2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 52 columns):
 #   Column                                                       Non-Null Count  Dtype         
---  ------                                                       --------------  -----         
 0   Individual_x                                                 920 non-null    object        
 1   Billing Date                                                 912 non-null    datetime64[ns]
 2   Hours worked this MONTH                                      912 non-null    float64       
 3   EmployerPlacementID                                          919 non-null    float64       
 4   Content Type_x                                               920 non-null    object        
 5   Created_x                                                    920 non-null    datetime64[ns]
 6   Created By_x                                                 920 non-null    object        
 7   Folder Child Coun

In [38]:
hourplace2.to_excel(r'S:\Rehab\Apricot\Migration\exports\dda month hours1.xlsx')

In [31]:
ddahourexport = pd.read_excel(r'S:\Rehab\Apricot\Migration\exports\external linking\employment for dda hours export report.xlsx')

In [ ]:
emprep = {
    'Walmart':'Walmart Kelsey St',
    'Safeway':'Safeway Bothell Everett',
    'Home Depot':'Home Depot Hwy 99' 
}

In [34]:
hourplace3 = hourplace2.merge(ddahourexport,how='left',left_on=['people_First Name','people_Last Name','Employer:Company Name'],right_on=['First','Last','Employer Organization Name'])
hourplace3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1166 entries, 0 to 1165
Data columns (total 64 columns):
 #   Column                                                       Non-Null Count  Dtype         
---  ------                                                       --------------  -----         
 0   Individual_x                                                 1166 non-null   object        
 1   Billing Date                                                 1157 non-null   datetime64[ns]
 2   Hours worked this MONTH                                      1157 non-null   float64       
 3   EmployerPlacementID                                          1165 non-null   float64       
 4   Content Type_x                                               1166 non-null   object        
 5   Created_x                                                    1166 non-null   datetime64[ns]
 6   Created By_x                                                 1166 non-null   object        
 7   Folder Child Co

In [45]:
hourplace4 = hourplace3[[
    'Individual2',
    'people_First Name',
    'Middle',
    'people_Last Name',
    'demo_Date of Birth',
    'Billing Date',
    'Employer Organization Name',
    'Placement Date',
    'employment Record ID',
    'Hours worked this MONTH',
    'Customized Employment: Wage'
]].copy()

hourplace4['associated employment'] = hourplace4['Employer Organization Name']+' Started: '+hourplace4['Placement Date'].astype(str)
hourplace5 = hourplace4[hourplace4['people_First Name'].notna()].copy()

In [46]:
hourplace6 = hourplace5.drop_duplicates(subset=['people_First Name','people_Last Name','Billing Date','employment Record ID',]).copy()

In [47]:
hourplace6.to_excel(r'S:\Rehab\Apricot\Migration\exports\dda month reporting3.xlsx')

#### DDA Hours Assignment

In [48]:
service = pd.read_excel(r'S:\Rehab\Apricot\Migration\Services - unformatted.xlsx')

# split out the contents of the client column on the ;# and add the split values to the newly created individual column
service[['Individual','Individual2']] = service['Individual'].str.split(';#', expand=True)

# sort the individual column values alphabetically
service = service.sort_values('Individual2',ascending=True)

# the key value had a space in it, so needed to remove that space
service['Individual2'] = service['Individual2'].str.replace(', ',',')

# remove test case rows
service = service[~service['Individual'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
service['Individual'] = service['Individual'].replace(r'\(.*?\)','',regex=True).str.strip()

service['Individual'] = service['Individual'].replace(r'\".*?\"','',regex=True).str.strip()

In [49]:
peepsmerge = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'people_Full Name',
]].copy()

peepsmerge['people_Full Name'] = peepsmerge['people_Full Name'].str.replace(', ',',')
peepsmerge['demo_Date of Birth'] = peepsmerge['demo_Date of Birth'].fillna('1/1/1800')

In [63]:
servicemerge = service.merge(peepsmerge,how='left',left_on='Individual2',right_on='people_Full Name')

In [64]:
servicemerge['Service'] = servicemerge['Service'].str.replace('BFET - SL (Life Skills)','Life Skills')
servicemerge['Service'] = servicemerge['Service'].str.replace('DDA - Individual Employment Services','Individual Employment Services')
servicemerge['Service'] = servicemerge['Service'].str.replace('BFET - BR (Job Retention)','Job Retention')
servicemerge['Service'] = servicemerge['Service'].str.replace('DDA - Community Inclusion','Community Inclusion')
servicemerge['Service'] = servicemerge['Service'].str.replace('BFET - JS (Supervised Job Search)','Supervised Job Search')
servicemerge['Service'] = servicemerge['Service'].str.replace('BFET - JT (Job Search Training)','Job Search Training')
servicemerge['Service'] = servicemerge['Service'].str.replace('DDA - School to Work','School to Work')
servicemerge['Service'] = servicemerge['Service'].str.replace('DDA - Job Foundations','Job Foundations')
servicemerge['Service'] = servicemerge['Service'].str.replace('Staffing','Orion Staffing Assignment')
servicemerge['Service'].value_counts()

Service
Assessment                          629
Job Development                     580
Job Retention                       480
Community Based Assessment          211
Case Management                     143
Life Skills                         109
Training                            108
Individual Employment Services       89
Community Inclusion                  35
Intensive Training Services          34
Job Search Training                  26
Supervised Job Search                22
School to Work                       12
IL Services                           5
Job Foundations                       4
Internship - Other                    3
Trial Work Experience                 2
Pre-ETS IL Skills Training            1
Internship - CNC Mills                1
Internship - Mechanical Assembly      1
Name: count, dtype: int64

In [65]:
servicemerge['Service Category'] = np.where(servicemerge['Service'].str.contains('Assessment|Community Based Assessment|Trial Work Experience|Life Skills'), 'Assessment',
                             np.where(servicemerge['Service'].str.contains('Job Retention|Intensive Training Services|Off Site Psycho Social Supports - Non-Supported Employment|Off Site Psycho Social Supports - Supported Employment'), 'Job Retention',
                             np.where(servicemerge['Service'].str.contains('Supervised Job Search|Job Search Training|Job Development'), 'Job Development',
                             np.where(servicemerge['Service'].str.contains('Case Management'), 'Case Management',
                             np.where(servicemerge['Service'].str.contains('Community Inclusion|Individual Employment Services|Job Foundations|School to Work'), 'DDA',
                             np.where(servicemerge['Service'].str.contains('IL Services'), 'Independent Living',
                             np.where(servicemerge['Service'].str.contains('Internship = CNC Mills|Internship - Mechanical Assembly|Internship - Other|Training'), 'Training',
                             np.where(servicemerge['Service'].str.contains('Orion Staffing Assignment'), 'Staffing',
                             np.where(servicemerge['Service'].str.contains('Pre-ETS IL Peer Mentoring|Pre-ETS IL Self Advocacy|Pre-ETS IL Skills Training|Pre-ETS IL Work Related Systems Access|Pre-ETS Information Interview|Pre-ETS Job Exploration|Pre-ETS Job Shadow|Pre-ETS Social Skills|Pre-ETS WBL|Pre-ETS Workplace Readiness Training'), 'Pre ETS',
                             'Other')))))))))
servicemerge['Service Category'].value_counts()

Service Category
Assessment            1005
Job Development        628
Job Retention          514
Case Management        143
DDA                    140
Training               113
Independent Living       5
Other                    1
Name: count, dtype: int64

In [69]:
servicemerge['subservice'] = np.where(servicemerge['Service'].str.contains('Community Inclusion'), 'Community Inclusion',
                            np.where(servicemerge['Service'].str.contains('Individual Employment Services'), 'Individual Employment Services',
                            np.where(servicemerge['Service'].str.contains('School to Work'), 'School to Work',
                            np.where(servicemerge['Service'].str.contains('Job Development'), 'Job Development',
                            np.where(servicemerge['Service'].str.contains('Intensive Training Services'), 'Job Retention',
                            'Other')))))
servicemerge['subservice'].value_counts()

subservice
Other                             1745
Job Development                    580
Individual Employment Services      89
Community Inclusion                 89
Job Retention                       34
School to Work                      12
Name: count, dtype: int64

In [57]:
hours = pd.read_excel(r'S:\Rehab\Apricot\Migration\exports\apricot dda hours assignment export.xlsx')

In [72]:
servicehourmerge = servicemerge.merge(hours,how='right',left_on=['people_First Name','people_Last Name','subservice'],right_on=['First','Last','Sub-Service Category'])
servicehourmerge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 37 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Start Date                            119 non-null    datetime64[ns]
 1   Individual                            119 non-null    object        
 2   Venue                                 119 non-null    object        
 3   End Date                              29 non-null     object        
 4   Service                               119 non-null    object        
 5   Recommendations                       0 non-null      object        
 6   Hours Allocated Per Month (DDA Only)  99 non-null     float64       
 7   Modified By                           119 non-null    object        
 8   Content Type                          119 non-null    object        
 9   Created                               119 non-null    datetime64[ns]
 10  Cr

In [73]:
servicehourmerge2 = servicehourmerge[[
    'First',
    'Middle',
    'Last',
    'Date of Birth',
    'Associated Service',
    'service Record ID',
    'Associated Sub Service',
    'Sub-Service Category',
    'Start Date',
    'End Date',
    'Hours Allocated Per Month (DDA Only)',
    'Assigned Programs'  
]].copy()

In [74]:
servicehourmerge2.to_excel(r'S:\Rehab\Apricot\Migration\exports\real dda hours2.xlsx')

In [24]:
place = pd.read_excel(r'S:\Rehab\Apricot\Migration\Employer placement hours - unformatted.xlsx')

# split out the contents of the client column on the ;# and add the split values to the newly created individual column
place[['Individual','Individual2']] = place['Individual'].str.split(';#', expand=True)

# sort the individual column values alphabetically
place = place.sort_values('Individual2',ascending=True)

# the key value had a space in it, so needed to remove that space
place['Individual2'] = place['Individual2'].str.replace(', ',',')

In [21]:
indprof = pd.read_csv(r'S:\Rehab\Apricot\Migration\for import\IndividualProfile import1 unreviewed.csv')
indprof = indprof.iloc[1:]
indprof

,System Header Row\nDO NOT DELETE THIS ROW\nOR THIS COLUMN!,field_3436_first,field_3436_middle,field_3436_last,field_3195,field_3848,field_2893_first,field_2893_middle,field_2893_last,field_3194,...,field_3212_country,field_3212_geolocation,field_3215,field_3216,field_3433,field_3434,field_3908,field_3909,field_3910,field_2899
1,NaN,Aaron,NaN,Erickson,3/10/1963,NaN,Aaron,NaN,Erickson,NaN,...,NaN,NaN,No,NaN,No,NaN,NaN,NaN,NaN,1174
2,NaN,Aaron,NaN,Mariano,5/15/2000,NaN,Aaron,NaN,Mariano,NaN,...,NaN,NaN,No,NaN,No,NaN,NaN,NaN,NaN,1174
3,NaN,Aaron,NaN,Sittauer,5/18/1982,NaN,Aaron,NaN,Sittauer,NaN,...,NaN,NaN,No,NaN,No,NaN,NaN,NaN,NaN,2870
4,NaN,Aaron,NaN,Williams,4/29/1983,NaN,Aaron,NaN,Williams,NaN,...,NaN,NaN,No,NaN,No,NaN,NaN,NaN,NaN,2870
5,NaN,Abdifata,NaN,Guyo,1/1/1983,NaN,Abdifata,NaN,Guyo,NaN,...,NaN,NaN,No,NaN,No,NaN,NaN,NaN,NaN,1174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,No,NaN,No,NaN,639133,Basic Plus,Low,2870
1367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,No,NaN,No,NaN,248974,Basic Plus,High,2870
1368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,No,NaN,No,NaN,770945,Core,Medium,2870
1369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,No,NaN,No,NaN,391709,Basic Plus,Medium,2870


In [22]:
indprof['Individual2'] = indprof['field_3436_last']+','+indprof['field_3436_first']

In [25]:
placemerge = place.merge(indprof,how='left',on='Individual2')
placemerge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 842 entries, 0 to 841
Data columns (total 41 columns):
 #   Column                                                    Non-Null Count  Dtype         
---  ------                                                    --------------  -----         
 0   Title                                                     842 non-null    object        
 1   Individual                                                842 non-null    object        
 2   Billing Date                                              834 non-null    datetime64[ns]
 3   Hours worked this MONTH                                   834 non-null    float64       
 4   EmployerPlacementID                                       841 non-null    float64       
 5   Item Type                                                 842 non-null    object        
 6   Path                                                      842 non-null    object        
 7   Individual2                                 

In [26]:
placemerge.to_excel(r'S:\Rehab\Apricot\Migration\exports\end month report1.xlsx')

In [ ]:
comm = pd.read_excel(r'S:\Rehab\Apricot\Migration\Client Community Activity - unformatted.xlsx')

### Hide

Merge dda case info and individual profile data together

this was very messy so I just did it manually, hence the commented code

In [10]:
ddacase = pd.read_excel(r'S:\Rehab\Apricot\Migration\DDA Case info - unformatted.xlsx')

# split out the contents of the client column on the ;# and add the split values to the newly created individual column
ddacase[['Individual','Individual2']] = ddacase['Individual'].str.split(';#', expand=True)

# sort the individual column values alphabetically
ddacase = ddacase.sort_values('Individual2',ascending=True)

# the key value had a space in it, so needed to remove that space
ddacase['Individual2'] = ddacase['Individual2'].str.replace(', ',',')

In [11]:
# people2 = pd.read_excel(r'S:\Rehab\Apricot\Migration\exports\people merge2.xlsx')
# people3 = people2[['Last Name_x','First Name_x','Date of Birth','Full Name']].copy()

In [12]:
# ddacasemerge = ddacase.merge(people3,how='left',left_on='Individual2',right_on='Full Name')
# ddacasemerge.head(3)

,Title,Individual,Date,DDA Case File Number,Funding Code,Acuity Level,End Date,Date of Annual Plan,6 Month Review Due Date,Item Type,Path,Individual2,Last Name_x,First Name_x,Date of Birth,Full Name
0,View,1533,2023-10-26,773156.0,Basic Plus,Medium,NaT,NaT,1900-06-30,Item,Lists/DDACaseInfo,"Adams,Melissa",Adams,Melissa,1987-08-31 00:00:00,"Adams,Melissa"
1,View,1012,2020-05-11,772403.0,Basic Plus,Low,NaT,2023-06-01,2023-11-30,Item,Lists/DDACaseInfo,"Akers,Laura",Akers,Laura,1978-09-25 00:00:00,"Akers,Laura"
2,View,307,2016-09-28,872605.0,Basic Plus,High,NaT,2023-01-07,2023-07-08,Item,Lists/DDACaseInfo,"Almgren,Anders",Almgren,Anders,1981-10-31 00:00:00,"Almgren,Anders"


In [13]:
# ddacasemerge2 = ddacasemerge[['Last Name_x','First Name_x','Date of Birth','DDA Case File Number','Funding Code','Acuity Level']].copy()

In [14]:
# indprof = pd.read_csv(r'S:\Rehab\Apricot\Migration\for import\IndividualProfile import1 unreviewed.csv')
# indprof = indprof.iloc[1:]
# indprof

,System Header Row\nDO NOT DELETE THIS ROW\nOR THIS COLUMN!,field_3436_first,field_3436_middle,field_3436_last,field_3195,field_3848,field_2893_first,field_2893_middle,field_2893_last,field_3194,...,field_3212_country,field_3212_geolocation,field_3215,field_3216,field_3433,field_3434,field_3908,field_3909,field_3910,field_2899
1,NaN,Besharo,NaN,Abduwahab,8/30/2000,NaN,Besharo,NaN,Abduwahab,NaN,...,NaN,NaN,No,NaN,No,NaN,NaN,NaN,NaN,Auburn
2,NaN,Noah,NaN,Abraham,4/2/1972,NaN,Noah,NaN,Abraham,NaN,...,NaN,NaN,No,NaN,No,NaN,NaN,NaN,NaN,Auburn
3,NaN,Rebecca,NaN,Acosta,5/1/1979,NaN,Rebecca,NaN,Acosta,NaN,...,NaN,NaN,No,NaN,No,NaN,NaN,NaN,NaN,NaN
4,NaN,Jnean,NaN,Acteson,5/22/1980,NaN,Jnean,NaN,Acteson,NaN,...,NaN,NaN,No,NaN,No,NaN,NaN,NaN,NaN,Auburn
5,NaN,Chris,NaN,Adair,11/26/1995,NaN,Chris,NaN,Adair,NaN,...,NaN,NaN,No,NaN,No,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407,NaN,Janet,NaN,Zello,3/29/1993,NaN,Janet,NaN,Zello,NaN,...,NaN,NaN,No,NaN,No,NaN,NaN,NaN,NaN,NaN
1408,NaN,Tevin,NaN,Zeno,2/21/1991,NaN,Tevin,NaN,Zeno,NaN,...,NaN,NaN,No,NaN,No,NaN,NaN,NaN,NaN,Auburn
1409,NaN,Manuel,NaN,Zepeda,9/6/1980,NaN,Manuel,NaN,Zepeda,NaN,...,NaN,NaN,No,NaN,No,NaN,NaN,NaN,NaN,Auburn
1410,NaN,Edwin,NaN,Zhang,10/30/1998,NaN,Edwin,NaN,Zhang,NaN,...,NaN,NaN,No,NaN,No,NaN,NaN,NaN,NaN,Mukilteo


In [15]:
# indprofmerge = indprof.merge(ddacasemerge2,how='left',
#                              left_on=['field_3436_first','field_3436_last','field_3195'],
#                             right_on=['First Name_x','Last Name_x','Date of Birth'])
# indprofmerge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1411 entries, 0 to 1410
Data columns (total 39 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   System Header Row
DO NOT DELETE THIS ROW
OR THIS COLUMN!  0 non-null      object 
 1   field_3436_first                                          1411 non-null   object 
 2   field_3436_middle                                         0 non-null      object 
 3   field_3436_last                                           1411 non-null   object 
 4   field_3195                                                1215 non-null   object 
 5   field_3848                                                0 non-null      object 
 6   field_2893_first                                          1411 non-null   object 
 7   field_2893_middle                                         0 non-null      object 
 8   field_2893_last   

In [16]:
# remove = indprofmerge[indprofmerge['Last Name_x'].notna()]
# remove

,System Header Row\nDO NOT DELETE THIS ROW\nOR THIS COLUMN!,field_3436_first,field_3436_middle,field_3436_last,field_3195,field_3848,field_2893_first,field_2893_middle,field_2893_last,field_3194,...,field_3908,field_3909,field_3910,field_2899,Last Name_x,First Name_x,Date of Birth,DDA Case File Number,Funding Code,Acuity Level
78,NaN,Dominick,NaN,Bailey,NaN,NaN,Dominick,NaN,Bailey,NaN,...,NaN,NaN,NaN,Mukilteo,Bailey,Dominick,NaN,445246.0,Basic Plus,Medium
79,NaN,Dominick,NaN,Bailey,NaN,NaN,Dominick,NaN,Bailey,NaN,...,NaN,NaN,NaN,Mukilteo,Bailey,Dominick,NaN,445246.0,Basic Plus,Medium
171,NaN,Michael,NaN,Bryant,NaN,NaN,Michael,NaN,Bryant,NaN,...,NaN,NaN,NaN,Mukilteo,Bryant,Michael,NaN,750285.0,Core,NaN
485,NaN,Annika,NaN,Hagemeier,NaN,NaN,Annika,NaN,Hagemeier,NaN,...,NaN,NaN,NaN,Mukilteo,Hagemeier,Annika,NaN,876318.0,School to Work,NaN
1017,NaN,Tatum,NaN,Rehm,NaN,NaN,Tatum,NaN,Rehm,NaN,...,NaN,NaN,NaN,Mukilteo,Rehm,Tatum,NaN,NaN,School to Work,NaN
1040,NaN,Dennis,NaN,Rockwell,NaN,NaN,Dennis,NaN,Rockwell,NaN,...,NaN,NaN,NaN,Mukilteo,Rockwell,Dennis,NaN,884390.0,Fircrest,Medium
1199,NaN,Desiree,NaN,Stuard,NaN,NaN,Desiree,NaN,Stuard,NaN,...,NaN,NaN,NaN,Mukilteo,Stuard,Desiree,NaN,864046.0,Basic Plus,Medium
1200,NaN,Desiree,NaN,Stuard,NaN,NaN,Desiree,NaN,Stuard,NaN,...,NaN,NaN,NaN,Mukilteo,Stuard,Desiree,NaN,864046.0,Basic Plus,Medium


In [17]:
# ddacasemerge3 = pd.merge(ddacasemerge2,remove,how='outer',
#                          left_on=['First Name_x','Last Name_x','Date of Birth'],
#                          right_on=['field_3436_first','field_3436_last','field_3195'])

In [18]:
# indprofmerge.to_excel(r'S:\Rehab\Apricot\Migration\exports\ind.xlsx')
# remove.to_excel(r'S:\Rehab\Apricot\Migration\exports\dda remove.xlsx')
# ddacasemerge2.to_excel(r'S:\Rehab\Apricot\Migration\exports\dda ind.xlsx')

In [51]:
indprof['field_3848']

,Client Alerts
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
1406,NaN
1407,NaN
1408,NaN
1409,NaN


### Service funding staff

In [453]:
fund = pd.read_excel(r'S:\Rehab\Apricot\Migration\Funding - unformatted.xlsx')

# split out the contents of the client column on the ;# and add the split values to the newly created individual column
fund[['Individual','Individual2']] = fund['Individual'].str.split(';#', expand=True)

# sort the individual column values alphabetically
fund = fund.sort_values('Individual2',ascending=True)

# the key value had a space in it, so needed to remove that space
fund['Individual2'] = fund['Individual2'].str.replace(', ',',')

# remove test case rows
fund = fund[~fund['Individual'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
fund['Individual'] = fund['Individual'].replace(r'\(.*?\)','',regex=True).str.strip()

fund['Individual'] = fund['Individual'].replace(r'\".*?\"','',regex=True).str.strip()
fund

,Individual,Venue,Service,Funding Source,Start Date,End Date,Final Funding Amount,Projected End / Billing Date,Projected Funding Amount,Staff,Modified,Date Billed - Intake,Amount Billed - Intake,Date Billed - Activity,Amount Billed - Activity,Date Billed - Outcome,Amount Billed - Outcome,Billing - Bonus,Optional DVR Billing Tracking!!,Content Type,Created,Created By,Folder Child Count,ID,Item Child Count,Modified By,Title,Type of Funding,Item Type,Path,Individual2
1,376,Auburn - Contact Center,NaN,NaN,2018-04-19,NaT,NaN,NaT,NaN,NaN,2018-04-19 15:57:09,NaT,NaN,NaT,NaN,NaT,NaN,NaN,NaN,Funding,2018-04-19 15:57:09,System Account,0,43,0,System Account,View,NaN,Item,Lists/Funding,
2,377,Auburn - Contact Center,NaN,NaN,2018-04-19,NaT,NaN,NaT,NaN,NaN,2018-04-19 16:21:46,NaT,NaN,NaT,NaN,NaT,NaN,NaN,NaN,Funding,2018-04-19 16:21:46,System Account,0,44,0,System Account,View,NaN,Item,Lists/Funding,
107,739,Mukilteo - Community Based Services,Job Development,DVR-Lynnwood,2019-02-25,NaT,NaN,2019-08-26,3715.0,Lauren Campbell,2019-05-10 12:19:21,NaT,NaN,NaT,NaN,NaT,NaN,NaN,NaN,Funding,2019-05-10 12:19:21,Lauren Campbell,0,159,0,Lauren Campbell,View,Contract,Item,Lists/Funding,
101,512,Auburn - Community Based Services,Job Development,DVR-Seatac,2018-12-17,2019-03-17,4070.0,2018-12-19,4070.0,Lavada Kindle,2019-03-20 11:15:50,NaT,NaN,NaT,NaN,NaT,NaN,NaN,NaN,Funding,2019-03-20 11:15:50,Lavada Kindle,0,153,0,Lavada Kindle,View,Contract,Item,Lists/Funding,"Abraham,Noah"
10,512,Auburn - Manufacturing Program,Community Based Assessment,DVR-Seatac,2018-08-17,2018-10-17,2450.0,2018-10-17,2450.0,Jason Person,2019-03-20 11:17:09,NaT,NaN,NaT,NaN,NaT,NaN,NaN,NaN,Funding,2018-09-13 10:20:48,System Account,0,55,0,Lavada Kindle,View,Contract,Item,Lists/Funding,"Abraham,Noah"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296,1778,Auburn - Manufacturing Program,BFET - SL (Life Skills),BFET,2024-05-17,NaT,NaN,2024-07-12,NaN,Jason Person,2024-06-17 13:36:40,NaT,NaN,NaT,NaN,NaT,NaN,NaN,Do not enter data - just to draw a line!,Funding,2024-06-17 13:36:40,Jason Person,0,1388,0,Jason Person,View,NaN,Item,Lists/Funding,"Zeno,Tevin"
987,1565,Mukilteo - Manufacturing Program,Community Based Assessment,DVR-Lynnwood,2023-08-28,2023-08-31,NaN,NaT,NaN,NaN,2023-09-06 13:32:41,NaT,NaN,NaT,NaN,NaT,NaN,NaN,NaN,Funding,2023-08-29 10:39:36,System Account,0,1072,0,Gabby Duran,View,NaN,Item,Lists/Funding,"Zhang,Edwin"
1078,1519,Auburn - Community Based Services,Job Retention,DVR-Seatac,2023-11-02,2024-03-05,NaN,2024-02-02,NaN,Lew Humiston,2024-03-07 07:55:51,NaT,NaN,NaT,NaN,NaT,NaN,NaN,Do not enter data - just to draw a line!,Funding,2023-11-02 16:44:53,Lew Humiston,0,1163,0,Lew Humiston,View,Contract,Item,Lists/Funding,"Zhang,Yuqing"
1060,1519,Auburn - Community Based Services,Job Development,DVR-Seatac,2023-10-18,2023-12-05,NaN,2024-01-18,3959.0,Lew Humiston,2024-03-18 09:22:36,2023-10-18,746.0,2023-11-02,997.0,2024-03-07,2181.0,Healthcare;#Rapid Placement;#High Wage,Do not enter data - just to draw a line!,Funding,2023-10-18 09:59:32,Lew Humiston,0,1145,0,Lew Humiston,View,Contract,Item,Lists/Funding,"Zhang,Yuqing"


In [454]:
peepsmerge = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'people_Full Name'
]].copy()

peepsmerge['people_Full Name'] = peepsmerge['people_Full Name'].str.replace(', ',',')

In [455]:
fund2 = fund.merge(peepsmerge, how='left',left_on='Individual2',right_on='people_Full Name')
fund2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1318 entries, 0 to 1317
Data columns (total 35 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   Individual                       1318 non-null   object        
 1   Venue                            1308 non-null   object        
 2   Service                          1152 non-null   object        
 3   Funding Source                   1295 non-null   object        
 4   Start Date                       1316 non-null   datetime64[ns]
 5   End Date                         1073 non-null   datetime64[ns]
 6   Final Funding Amount             466 non-null    float64       
 7   Projected End / Billing Date     742 non-null    datetime64[ns]
 8   Projected Funding Amount         473 non-null    float64       
 9   Staff                            1180 non-null   object        
 10  Modified                         1318 non-null   datetime64[

In [456]:
fund2 = fund2.drop(columns=['Title','Item Type','Path','Optional DVR Billing Tracking!!'])

In [457]:
fund2['Service'].value_counts()

Service
Job Development                                               208
Assessment                                                    197
Community Based Assessment                                    158
BFET - SL (Life Skills)                                       118
Job Retention                                                  89
Case Management                                                84
DDA - Individual Employment Services                           77
Training                                                       57
Intensive Training Services                                    36
DDA - Community Inclusion                                      35
BFET - BR (Job Retention)                                      32
BFET - JS (Supervised Job Search)                              25
BFET - JT (Job Search Training)                                16
DDA - School to Work                                           11
IL Services                                                     2
Of

In [458]:
# edit service names

fund2['Service'] = fund2['Service'].str.replace('BFET - SL (Life Skills)','Life Skills')
fund2['Service'] = fund2['Service'].str.replace('DDA - Individual Employment Services','Individual Employment Services')
fund2['Service'] = fund2['Service'].str.replace('BFET - BR (Job Retention)','Job Retention')
fund2['Service'] = fund2['Service'].str.replace('DDA - Community Inclusion','Community Inclusion')
fund2['Service'] = fund2['Service'].str.replace('BFET - JS (Supervised Job Search)','Supervised Job Search')
fund2['Service'] = fund2['Service'].str.replace('BFET - JT (Job Search Training)','Job Search Training')
fund2['Service'] = fund2['Service'].str.replace('DDA - School to Work','School to Work')
fund2['Service'] = fund2['Service'].str.replace('DDA - Job Foundations','Job Foundations')
fund2['Service'] = fund2['Service'].str.replace('Staffing','Orion Staffing Assignment')

In [459]:
# add a service category to the df

fund2['Service Category'] = np.where(fund2['Service'].str.contains('Assessment|Community Based Assessment|Trial Work Experience|Life Skills'), 'Assessment',
                             np.where(fund2['Service'].str.contains('Job Retention|Intensive Training Services|Off Site Psycho Social Supports - Non-Supported Employment|Off Site Psycho Social Supports - Supported Employment'), 'Job Retention',
                             np.where(fund2['Service'].str.contains('Supervised Job Search|Job Search Training|Job Development'), 'Job Development',
                             np.where(fund2['Service'].str.contains('Case Management'), 'Case Management',
                             np.where(fund2['Service'].str.contains('Community Inclusion|Individual Employment Services|Job Foundations|School to Work'), 'DDA',
                             np.where(fund2['Service'].str.contains('IL Services'), 'Independent Living',
                             np.where(fund2['Service'].str.contains('Internship = CNC Mills|Internship - Mechanical Assembly|Internship - Other|Training'), 'Training',
                             np.where(fund2['Service'].str.contains('Orion Staffing Assignment'), 'Staffing',
                             np.where(fund2['Service'].str.contains('Pre-ETS IL Peer Mentoring|Pre-ETS IL Self Advocacy|Pre-ETS IL Skills Training|Pre-ETS IL Work Related Systems Access|Pre-ETS Information Interview|Pre-ETS Job Exploration|Pre-ETS Job Shadow|Pre-ETS Social Skills|Pre-ETS WBL|Pre-ETS Workplace Readiness Training'), 'Pre ETS',
                             'Other')))))))))

In [460]:
fund2['location code'] = np.where(fund2['Venue'].str.contains('Auburn'),1174,2870)

In [461]:
fund2.columns

Index(['Individual', 'Venue', 'Service', 'Funding Source', 'Start Date',
       'End Date', 'Final Funding Amount', 'Projected End / Billing Date',
       'Projected Funding Amount', 'Staff', 'Modified', 'Date Billed - Intake',
       'Amount Billed - Intake', 'Date Billed - Activity',
       'Amount Billed - Activity', 'Date Billed - Outcome',
       'Amount Billed - Outcome', 'Billing - Bonus', 'Content Type', 'Created',
       'Created By', 'Folder Child Count', 'ID', 'Item Child Count',
       'Modified By', 'Type of Funding', 'Individual2', 'people_First Name',
       'people_Last Name', 'demo_Date of Birth', 'people_Full Name',
       'Service Category', 'location code'],
      dtype='object')

In [462]:
fund3 =  fund2[fund2['people_Full Name'].notna()].copy()
fund3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 775 entries, 5 to 1317
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Individual                    775 non-null    object        
 1   Venue                         771 non-null    object        
 2   Service                       694 non-null    object        
 3   Funding Source                765 non-null    object        
 4   Start Date                    775 non-null    datetime64[ns]
 5   End Date                      582 non-null    datetime64[ns]
 6   Final Funding Amount          181 non-null    float64       
 7   Projected End / Billing Date  411 non-null    datetime64[ns]
 8   Projected Funding Amount      206 non-null    float64       
 9   Staff                         689 non-null    object        
 10  Modified                      775 non-null    datetime64[ns]
 11  Date Billed - Intake          35 non

In [463]:
fund3 = fund3[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'Start Date',
    'End Date',
    'Service Category',
    'Service',
    'Funding Source',
    'Staff',
    'location code'
]].copy()

In [464]:
# create a migration id column

fund3['migrationID'] = range(1,len(fund3)+1)
fund3

,people_First Name,people_Last Name,demo_Date of Birth,Start Date,End Date,Service Category,Service,Funding Source,Staff,location code,migrationID
5,Jnean,Acteson,1980-05-22 00:00:00,2022-08-15,2022-10-07,Job Development,Job Search Training,BFET,Tamie Clark,1174,1
6,Jnean,Acteson,1980-05-22 00:00:00,2022-05-20,2023-01-09,Assessment,Life Skills,BFET,Tricia Cleavelin,1174,2
7,Jnean,Acteson,1980-05-22 00:00:00,2022-10-10,2023-01-10,Job Retention,Job Retention,BFET,Tamie Clark,1174,3
13,Melissa,Adams,1987-08-31 00:00:00,2023-07-01,NaT,DDA,Individual Employment Services,DDA-Snohomish,Traci Dosch,2870,4
20,Diana,Aguilar,1974-08-18 00:00:00,2024-01-29,2024-04-05,Assessment,Assessment,DVR-Kent,Jason Person,1174,5
...,...,...,...,...,...,...,...,...,...,...,...
1313,Tevin,Zeno,1991-02-21 00:00:00,2024-05-17,NaT,Assessment,Life Skills,BFET,Jason Person,1174,771
1314,Edwin,Zhang,1998-10-30 00:00:00,2023-08-28,2023-08-31,Assessment,Community Based Assessment,DVR-Lynnwood,NaN,2870,772
1315,Yuqing,Zhang,1978-02-12 00:00:00,2023-11-02,2024-03-05,Job Retention,Job Retention,DVR-Seatac,Lew Humiston,1174,773
1316,Yuqing,Zhang,1978-02-12 00:00:00,2023-10-18,2023-12-05,Job Development,Job Development,DVR-Seatac,Lew Humiston,1174,774


In [465]:
fund3['Funding Source'] = fund3['Funding Source'].fillna('Unfunded')
# fund3['DVR Office'] = fund3[fund3['Funding Source'].str.contains('DVR')] 
# fund3

# Create a new column with values containing "DVR"
fund3['DVR Office'] = fund3['Funding Source'].where(fund3['Funding Source'].str.contains('DVR'), np.nan)

# Remove text following "DVR" in the original column
fund3['Funding Source'] = fund3['Funding Source'].str.replace(r'DVR.*', 'DVR', regex=True)

# Do same thing that was done with DVR but with DDA
fund3['DDA Office'] = fund3['Funding Source'].where(fund3['Funding Source'].str.contains('DDA'), np.nan)

fund3['Funding Source'] = fund3['Funding Source'].str.replace(r'DDA.*', 'DDA', regex=True)
fund3

,people_First Name,people_Last Name,demo_Date of Birth,Start Date,End Date,Service Category,Service,Funding Source,Staff,location code,migrationID,DVR Office,DDA Office
5,Jnean,Acteson,1980-05-22 00:00:00,2022-08-15,2022-10-07,Job Development,Job Search Training,BFET,Tamie Clark,1174,1,NaN,NaN
6,Jnean,Acteson,1980-05-22 00:00:00,2022-05-20,2023-01-09,Assessment,Life Skills,BFET,Tricia Cleavelin,1174,2,NaN,NaN
7,Jnean,Acteson,1980-05-22 00:00:00,2022-10-10,2023-01-10,Job Retention,Job Retention,BFET,Tamie Clark,1174,3,NaN,NaN
13,Melissa,Adams,1987-08-31 00:00:00,2023-07-01,NaT,DDA,Individual Employment Services,DDA,Traci Dosch,2870,4,NaN,DDA-Snohomish
20,Diana,Aguilar,1974-08-18 00:00:00,2024-01-29,2024-04-05,Assessment,Assessment,DVR,Jason Person,1174,5,DVR-Kent,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1313,Tevin,Zeno,1991-02-21 00:00:00,2024-05-17,NaT,Assessment,Life Skills,BFET,Jason Person,1174,771,NaN,NaN
1314,Edwin,Zhang,1998-10-30 00:00:00,2023-08-28,2023-08-31,Assessment,Community Based Assessment,DVR,NaN,2870,772,DVR-Lynnwood,NaN
1315,Yuqing,Zhang,1978-02-12 00:00:00,2023-11-02,2024-03-05,Job Retention,Job Retention,DVR,Lew Humiston,1174,773,DVR-Seatac,NaN
1316,Yuqing,Zhang,1978-02-12 00:00:00,2023-10-18,2023-12-05,Job Development,Job Development,DVR,Lew Humiston,1174,774,DVR-Seatac,NaN


In [466]:
fund3['Funding Source'].value_counts()

Funding Source
DVR                                                                 290
Snohomish County Human Services | Housing and Community Services    130
DDA                                                                 110
BFET                                                                 94
King County Jobs and Housing                                         69
City of Auburn                                                       29
City of Federal Way                                                  19
WRT - State Board for Community and Technical Colleges               12
Unfunded                                                             10
City of Renton                                                        4
City of Kent                                                          3
SJI (Seattle Jobs Initiative)                                         2
DSB-Tacoma                                                            1
Labor & Industries                               

In [467]:
fundrep = {
    'Olsen & Associates':'Labor & Industries',
    'Strategic Consulting Services':'Labor & Industries',
    'Pioneer Industries':'Labor & Industries',
    'Aerospace Joint Apprenticeship Committee (AJAC)':'Labor & Industries',
    'Workforce Training and Education Coordinating Board':'WRT',
    'WRT - State Board for Community and Technical Colleges':'WRT',
    'City of Renton':'City Funding - Human Services (Auburn, Renton)',
    'City of Kent':'City Funding - Human Services (Auburn, Renton)',
    'City of Auburn':'City Funding - Human Services (Auburn, Renton)',
    'City of Federal Way':'City Funding - CDBG (Federal Way)',
    'Snohomish County Human Services | Housing and Community Services':'Snohomish County - ERSS',
    'DDA-King County':'DDA',
    'DDA-Snohomish':'DDA',
    'Veterans Administration-Seattle':'Veterans Administration',
    'King County Jobs and Housing':'King County Jobs & Housing',
    'SJI (Seattle Jobs Initiative)':'Labor & Industries',
    'DSB-Tacoma':'DVR'
}

fund3['Funding Source'] = fund3['Funding Source'].replace(fundrep)

In [469]:
fund3.to_excel(r'S:\Rehab\Apricot\Migration\exports\real service funding staff.xlsx')

#### Program

In [422]:
service = pd.read_excel(r'S:\Rehab\Apricot\Migration\Services - unformatted.xlsx')

# remove repetitive columns
service = service.drop(columns=['Title','Item Type','Path','Folder Child Count','Created By','Content Type','Item Child Count'])

# remove the (###) from the individual name
service['Individual'] = service['Individual'].replace(r'\b\d{1,4};#','',regex=True)

# the key value had a space in it, so needed to remove that space
service['Individual'] = service['Individual'].str.replace(', ',',')

# remove test case rows
service = service[~service['Individual'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
service['Individual'] = service['Individual'].replace(r'\(.*?\)','',regex=True).str.strip()

service['Individual'] = service['Individual'].replace(r'\".*?\"','',regex=True).str.strip()

In [425]:
venue = pd.read_excel(r'S:\Rehab\Apricot\Migration\venue history - unformatted.xlsx')

# remove repetitive columns
venue = venue.drop(columns=['Item Type','Path','Folder Child Count','Created By','Content Type','Item Child Count'])

# the key value had a space in it, so needed to remove that space
venue['Individual'] = venue['Individual'].str.replace(', ',',')

# remove test case rows
venue = venue[~venue['Individual'].str.contains('test', case=False, na=False)].reset_index().copy()

# remove any text (within parethesis) to standardize names
venue['Individual'] = venue['Individual'].replace(r'\(.*?\)','',regex=True).str.strip()

venue['Individual'] = venue['Individual'].replace(r'\".*?\"','',regex=True).str.strip()

venue2 = venue.add_prefix('venue_')

In [426]:
venue2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2208 entries, 0 to 2207
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   venue_index                2208 non-null   int64         
 1   venue_Individual           2208 non-null   object        
 2   venue_Venue                2208 non-null   object        
 3   venue_Created              2208 non-null   datetime64[ns]
 4   venue_Start Date           2208 non-null   datetime64[ns]
 5   venue_Exit Date            1995 non-null   datetime64[ns]
 6   venue_Status               2208 non-null   object        
 7   venue_Referred By          832 non-null    object        
 8   venue_Shift                1165 non-null   object        
 9   venue_Shift 2 Change Date  33 non-null     datetime64[ns]
 10  venue_Shift Notes          351 non-null    object        
 11  venue_Orion Staff          0 non-null      float64       
 12  venue_

In [427]:
peepsmerge = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'people_Full Name'
]].copy()

peepsmerge['people_Full Name'] = peepsmerge['people_Full Name'].str.replace(', ',',')

In [428]:
peepsmerge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 528 entries, 0 to 527
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   people_First Name   528 non-null    object
 1   people_Last Name    528 non-null    object
 2   demo_Date of Birth  489 non-null    object
 3   people_Full Name    528 non-null    object
dtypes: object(4)
memory usage: 16.6+ KB


In [430]:
# service2 = service.merge(peepsmerge, how='left',left_on='Individual',right_on='people_Full Name')
# service2.info()

In [431]:
venue3 = venue2.merge(peepsmerge, how='left',left_on='venue_Individual',right_on='people_Full Name')

venue3['location code'] = np.where(venue3['venue_Venue'].str.contains('Auburn'),1174,2870)
venue3['location'] = np.where(venue3['venue_Venue'].str.contains('Auburn'),'Auburn','Mukilteo')

venue3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2216 entries, 0 to 2215
Data columns (total 34 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   venue_index                2216 non-null   int64         
 1   venue_Individual           2216 non-null   object        
 2   venue_Venue                2216 non-null   object        
 3   venue_Created              2216 non-null   datetime64[ns]
 4   venue_Start Date           2216 non-null   datetime64[ns]
 5   venue_Exit Date            2002 non-null   datetime64[ns]
 6   venue_Status               2216 non-null   object        
 7   venue_Referred By          835 non-null    object        
 8   venue_Shift                1169 non-null   object        
 9   venue_Shift 2 Change Date  33 non-null     datetime64[ns]
 10  venue_Shift Notes          353 non-null    object        
 11  venue_Orion Staff          0 non-null      float64       
 12  venue_

In [432]:
# service2['location code'] = np.where(service2['Venue'].str.contains('Auburn'),1174,2870)
# service2['location'] = np.where(service2['Venue'].str.contains('Auburn'),'Auburn','Mukilteo')

In [433]:
# service2['Venue'].value_counts()

In [434]:
# change the venue to match the training program

# service2['Venue'] = service2['Venue'].str.replace('Auburn - Office Skills & Customer Service','Clerical Customer Service Training')
# service2['Venue'] = service2['Venue'].str.replace('Auburn - Contact Center','Clerical Customer Service Training')
# service2['Venue'] = service2['Venue'].str.replace('Auburn - Manufacturing Program','Transitional Training - Manufacturing')
# service2['Venue'] = service2['Venue'].str.replace('Mukilteo - Manufacturing Program','Transitional Training - Manufacturing')
# service2['Venue'] = service2['Venue'].str.replace('Auburn - Custodial','Transitional Training - Custodial')
# service2['Venue'] = service2['Venue'].str.replace('Mukilteo-Light Assembly','Transitional Training - Manufacturing')
# ### service2['Venue'] = service2['Venue'].str.replace('Auburn - Community Based Services','')
# ### service2['Venue'] = service2['Venue'].str.replace('Mukilteo - Community Based Services','')


In [435]:
# change the venue to match the training program

venue3['venue_Venue'] = venue3['venue_Venue'].str.replace('Auburn - Office Skills & Customer Service','Clerical Customer Service Training')
venue3['venue_Venue'] = venue3['venue_Venue'].str.replace('Auburn - Contact Center','Clerical Customer Service Training')
venue3['venue_Venue'] = venue3['venue_Venue'].str.replace('Auburn - Manufacturing Program','Transitional Training - Manufacturing')
venue3['venue_Venue'] = venue3['venue_Venue'].str.replace('Mukilteo - Manufacturing Program','Transitional Training - Manufacturing')
venue3['venue_Venue'] = venue3['venue_Venue'].str.replace('Auburn - Custodial','Transitional Training - Custodial')
venue3['venue_Venue'] = venue3['venue_Venue'].str.replace('Mukilteo-Light Assembly','Transitional Training - Manufacturing')
### venue3['Venue'] = venue3['Venue'].str.replace('Auburn - Community Based Services','')
### venue3['Venue'] = venue3['Venue'].str.replace('Mukilteo - Community Based Services','')

In [436]:
# service2

In [437]:
# service2.columns

In [438]:
venue3.columns

Index(['venue_index', 'venue_Individual', 'venue_Venue', 'venue_Created',
       'venue_Start Date', 'venue_Exit Date', 'venue_Status',
       'venue_Referred By', 'venue_Shift', 'venue_Shift 2 Change Date',
       'venue_Shift Notes', 'venue_Orion Staff', 'venue_Assigned Staff',
       'venue_Modified', 'venue_Eligible?', 'venue_Exit Reason',
       'venue_Header -Shift2', 'venue_Header-Shift', 'venue_ID',
       'venue_Modified By', 'venue_Notes', 'venue_Referral Date',
       'venue_Shift 2', 'venue_Shift 2 Notes', 'venue_Shift 3',
       'venue_Shift 3 Change Date', 'venue_Shift 3 Notes', 'venue_View',
       'people_First Name', 'people_Last Name', 'demo_Date of Birth',
       'people_Full Name', 'location code', 'location'],
      dtype='object')

In [445]:
venue4 =  venue3[venue3['people_Full Name'].notna()].copy()
venue4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 882 entries, 20 to 2215
Data columns (total 34 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   venue_index                882 non-null    int64         
 1   venue_Individual           882 non-null    object        
 2   venue_Venue                882 non-null    object        
 3   venue_Created              882 non-null    datetime64[ns]
 4   venue_Start Date           882 non-null    datetime64[ns]
 5   venue_Exit Date            671 non-null    datetime64[ns]
 6   venue_Status               882 non-null    object        
 7   venue_Referred By          353 non-null    object        
 8   venue_Shift                456 non-null    object        
 9   venue_Shift 2 Change Date  8 non-null      datetime64[ns]
 10  venue_Shift Notes          242 non-null    object        
 11  venue_Orion Staff          0 non-null      float64       
 12  venue_Assig

In [446]:
venue4 = venue4[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth', 
    'venue_Venue',
    'location',
    'venue_Start Date',
    'venue_Exit Date',
    'venue_Exit Reason',
    'venue_Notes',
    'venue_Shift',
    'venue_Shift Notes',
    'location code'
]].copy()

venue4['Status'] = np.where(venue4['venue_Exit Date'].isna(),'Program Started','Closed No Program Services')

In [447]:
venue5 = venue4[~venue4['venue_Venue'].str.contains('Community Based Services')].reset_index().copy()

In [448]:
venue5['venue_Venue'].value_counts()

venue_Venue
Transitional Training - Manufacturing    361
Clerical Customer Service Training        41
Transitional Training - Custodial          8
Name: count, dtype: int64

In [449]:
venue6 = venue5[~venue5['venue_Venue'].str.contains('test', case=False, na=False)].copy()

venue6['venue_Venue'].value_counts()

venue_Venue
Transitional Training - Manufacturing    361
Clerical Customer Service Training        41
Transitional Training - Custodial          8
Name: count, dtype: int64

In [451]:
venue6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410 entries, 0 to 409
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   index               410 non-null    int64         
 1   people_First Name   410 non-null    object        
 2   people_Last Name    410 non-null    object        
 3   demo_Date of Birth  397 non-null    object        
 4   venue_Venue         410 non-null    object        
 5   location            410 non-null    object        
 6   venue_Start Date    410 non-null    datetime64[ns]
 7   venue_Exit Date     349 non-null    datetime64[ns]
 8   venue_Exit Reason   331 non-null    object        
 9   venue_Notes         191 non-null    object        
 10  venue_Shift         392 non-null    object        
 11  venue_Shift Notes   238 non-null    object        
 12  location code       410 non-null    int32         
 13  Status              410 non-null    object        

In [452]:
venue6.to_excel(r'S:\Rehab\Apricot\Migration\exports\real enrollment to program.xlsx')

In [242]:
# service3 = service2[[
#     'people_First Name',
#     'people_Last Name',
#     'demo_Date of Birth',
#     'Venue',
#     'location',
#     'Start Date',
#     'End Date',
#     'Recommendations',
#     'Notes',
#     'location code',
# ]].copy()
# service3

,people_First Name,people_Last Name,demo_Date of Birth,Venue,location,Start Date,End Date,Recommendations,Notes,location code
0,NaN,NaN,NaN,Mukilteo - Community Based Services,Mukilteo,2018-05-03,2023-01-31 00:00:00,NaN,NaN,2870
1,Crystal,Bibbee,1981-06-19 00:00:00,Mukilteo - Community Based Services,Mukilteo,2018-05-30,2021-10-20 00:00:00,NaN,Crystal moved from IE to CI and chose to work ...,2870
2,Erica,Bricklin,NaN,Clerical Customer Service Training,Auburn,2018-08-13,2018-11-15 00:00:00,NaN,NaN,1174
3,Glen,Tillotson,1946-12-06 00:00:00,Transitional Training - Manufacturing,Auburn,2018-05-11,2018-08-21 00:00:00,NaN,NaN,1174
4,Son,Quach,1973-01-09 00:00:00,Transitional Training - Manufacturing,Auburn,2018-08-17,2018-10-11 00:00:00,NaN,NaN,1174
...,...,...,...,...,...,...,...,...,...,...
2566,NaN,NaN,NaN,Mukilteo - Community Based Services,Mukilteo,2024-05-14,NaN,NaN,NaN,2870
2567,NaN,NaN,NaN,Mukilteo - Community Based Services,Mukilteo,2024-05-01,NaN,NaN,NaN,2870
2568,NaN,NaN,NaN,Mukilteo - Community Based Services,Mukilteo,2024-05-16,NaN,NaN,NaN,2870
2569,NaN,NaN,NaN,Mukilteo - Community Based Services,Mukilteo,2024-05-20,NaN,NaN,NaN,2870


In [243]:
# service3 = service3[~service3['Venue'].str.contains('Community Based Services')].reset_index().copy()
# service3 = service3.drop(columns='index')

In [244]:
# service3.to_excel(r'S:\Rehab\Apricot\Migration\exports\enrollment to program import1.xlsx')

#### Service

In [410]:
enroll = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'intake_Orion Location',
    'intake_Date of Intake',
    'intake_Intake Admin Notes',
    'intake_Status of Intake',
    'intake_Start Date',
    'intake_Processing details: Funding and Project Eligibility',
    'intake_Assigned to Venue',
    'intake_Processing details: Criminal History Check',
    'intake_Processing details: Date Closed',
    'intake_Processing details: Closed'
]].copy()

enroll['location code'] = enroll['intake_Orion Location']
enroll['location code'] = enroll['location code'].replace('Auburn',1174)
enroll['location code'] = enroll['location code'].replace('Mukilteo',2870)

enrollcolname = [
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'Location',
    'Date intake started',
    'Service intake processing notes',
    'Status',
    'Date services starting',
    'Eligible funders',
    'Is client also participating in a training program?',
    'Criminal History Check',
    'Date Closing',
    'Closing Note',
    'Assigned programs'
]

enroll.columns = enrollcolname

In [411]:
# replace values in status

statrep = {
    'Started':'Services Started',
    'Withdrawn':'Closed No Services',
    'Denied':'Closed No Services',
    'Approved':'On Hold',
    'Tour Only':'Intake Processing',
    
}
enroll['Status'] = enroll['Status'].replace(statrep)

In [412]:
import re

enroll.loc[:,'Eligible funders'] = enroll['Eligible funders'].replace(r';#\d{1,2}','|',regex=True)
enroll.loc[:,'Eligible funders'] = enroll['Eligible funders'].str.rstrip('|')
enroll.loc[:,'Eligible funders'] = enroll['Eligible funders'].str.replace(';#','')
enroll['Eligible funders'].value_counts()

Eligible funders
Snohomish County CSBG                                          68
BFET                                                           43
King County Jobs & Housing                                     37
Auburn Human Services Grant                                    15
Federal Way CDBG Program                                       10
BFET|Auburn Human Services Grant                                8
Renton Human Services Grant                                     6
BFET|King County Jobs & Housing                                 5
Auburn Human Services Grant|BFET                                5
Auburn Human Services Grant|BFET|King County Jobs & Housing     5
SBCTC - WRT Program                                             4
King County Jobs & Housing|Auburn Human Services Grant          3
Auburn Human Services Grant|King County Jobs & Housing|BFET     3
REDF                                                            3
Auburn Human Services Grant|King County Jobs & Housing     

In [413]:
# find unique values in the eligible funders column

unique_funders = enroll['Eligible funders'].unique()

split_items = [item for sublist in unique_funders if isinstance(sublist, str) for item in sublist.split('|')]
uniquedf = pd.DataFrame(split_items)
unique_items = uniquedf[0].unique()

In [414]:
for i in unique_items:
    print(i)

Snohomish County CSBG
Federal Way CDBG Program
King County Jobs & Housing
Auburn Human Services Grant
BFET
SBCTC - WRT Program
Renton Human Services Grant
REDF
RTI Study
Auburn COVID Grant
CareerReady (SJI)
Kent Human Services Grant
Kent COVID Grant


In [415]:
elrep = {
    'Renton Human Services Grant':'City of Renton',
    'Auburn Human Services Grant':'City of Auburn',
    'Kent Human Services Grant':'City of Kent',
    'Auburn COVID Grant':'City of Auburn',
    'Kent COVID Grant':'City of Kent',
    'Federal Way CDBG Program':'City of Federal Way',
    'King County Jobs & Housing':'King County Jobs and Housing',
    'Snohomish County CSBG':'Snohomish County Human Services Housing and Community Services',
    'SBCTC - WRT Program':'WRT - State Board for Community and Technical Colleges',
    'REDF':'',
    'CareerReady \(SJI\)':'',
    'RTI Study':'',
    'WDC - Youth Internship':''
}

enroll['Eligible funders'] = enroll['Eligible funders'].replace(elrep,regex=True)
enroll['Eligible funders'] = enroll['Eligible funders'].str.replace('||','|')
enroll['Eligible funders'] = enroll['Eligible funders'].str.replace('||','|')
enroll['Eligible funders'] = enroll['Eligible funders'].str.strip('|')

In [416]:
enroll['Eligible funders'].value_counts()

Eligible funders
Snohomish County Human Services Housing and Community Services                68
BFET                                                                          43
King County Jobs and Housing                                                  37
City of Auburn                                                                19
City of Federal Way                                                           11
BFET|City of Auburn                                                            8
City of Auburn|BFET                                                            6
City of Renton                                                                 6
City of Auburn|BFET|King County Jobs and Housing                               5
BFET|King County Jobs and Housing                                              5
WRT - State Board for Community and Technical Colleges                         5
                                                                               4
City of Aub

In [417]:
# set training program to binary response

enroll['Is client also participating in a training program?'] = np.where(enroll['Is client also participating in a training program?'].str.contains('Manufacturing Program|Office Skills & Customer Service|Custodial|Contact Center'),'Yes','No')

In [418]:
enroll['Is client also participating in a training program?'].value_counts()

Is client also participating in a training program?
Yes    396
No     132
Name: count, dtype: int64

In [419]:
enroll['National Criminal History Check'] = np.where(enroll['Criminal History Check'] == 'National - pending','Pending','')
enroll['Criminal History Check'] = enroll['Criminal History Check'].str.replace('National - pending','')

In [420]:
enroll

,people_First Name,people_Last Name,demo_Date of Birth,Location,Date intake started,Service intake processing notes,Status,Date services starting,Eligible funders,Is client also participating in a training program?,Criminal History Check,Date Closing,Closing Note,Assigned programs,National Criminal History Check
0,Keith,Shuman,1970-05-22 00:00:00,NaN,NaT,NaN,NaN,NaT,NaN,Yes,NaN,NaT,NaN,NaN,
1,Melissa,Waltjen,1977-10-06 00:00:00,NaN,NaT,NaN,NaN,NaT,NaN,Yes,NaN,NaT,NaN,NaN,
2,Rebekah,Lockhart,2002-03-26 00:00:00,Auburn,2022-11-07,On vacation end of November. Have her coming i...,Services Started,2023-01-06,NaN,Yes,Approved,NaT,NaN,1174.0,
3,Jerry,Gainey,1985-02-16 00:00:00,Auburn,2022-12-19,"Met with the client post-tour, discussed progr...",Services Started,2023-01-06,NaN,Yes,Approved,NaT,NaN,1174.0,
4,Dez'Rae,Oster,1995-03-28 00:00:00,Mukilteo,2022-12-20,NaN,Services Started,2022-12-20,NaN,No,Pending,NaT,NaN,2870.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,Maxine,Ghansah,1989-09-02 00:00:00,Mukilteo,2022-10-19,NaN,Services Started,2022-10-19,NaN,No,Pending,NaT,NaN,2870.0,
524,Steven,Mason,1966-04-19 00:00:00,Mukilteo,2022-12-02,NaN,Services Started,2022-12-02,NaN,No,Pending,NaT,NaN,2870.0,
525,Savannah,Phillips,1997-05-19 00:00:00,Auburn,2023-01-23,"Referred by Step Mom. In a AAHAA house, but do...",Services Started,2023-02-03,City of Auburn|BFET|King County Jobs and Housing,Yes,Approved,NaT,NaN,1174.0,
526,George,Johnston,1975-08-14 00:00:00,Mukilteo,2023-03-24,NaN,Services Started,2023-04-10,NaN,Yes,Approved,NaT,NaN,2870.0,


In [421]:
enroll.to_excel(r'S:\Rehab\Apricot\Migration\exports\real enrollment to service import2.xlsx')

### Intake

In [357]:
intake = pd.read_excel(r'S:\Rehab\Apricot\Migration\Intake Wizard (intake application) - unformatted.xlsx')

# remove repetitive columns
intake = intake.drop(columns=['Title','Item Type','Path','Folder Child Count','Content Type','Item Child Count'])

# the key value had a space in it, so needed to remove that space
intake['Full Name'] = intake['Full Name'].str.replace(', ',',')

# remove test case rows
intake = intake[~intake['Full Name'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
intake['Full Name'] = intake['Full Name'].replace(r'\(.*?\)','',regex=True).str.strip()

intake['Full Name'] = intake['Full Name'].replace(r'\".*?\"','',regex=True).str.strip()

In [358]:
intake2 = intake[[
    'Created By',
    'Created',
    'Processing details: Closed',
    'Processing details: Date Closed',
    'Processing details: Closed',
    'Processing details: notes',
    'Created',
    'Orion Location',
    'Assigned to Venue',
    'Assigned to Venue',
    'Referral details: Referral Source',
    'Date of Tour',
    'Date of Tour',
    'Referral details: Referral Source',
    'Referral Contact:First Name',
    'Referral Contact:Last Name',
    'First Name',
    'Last Name',
    'Nickname',
    'Phone',
    'Phone',
    'Email',
    'Address',
    'City',
    'State',
    'Zip Code:County',
    'Zip Code',
    'Guardian: Status',
    'Guardian: email',
    'Guardian: First Name',
    'Guardian: Last Name',
    'Guardian: Phone',
    'Status of Intake',
    'Orion Location'
]]
colname = [
    'Assigned Staff',
    'Date Assigned',
    'Closing Referral without Moving Forward?',
    'Date Closed',
    'Referral Closed Reason',
    'Processing Notes',
    'Date',
    'Site',
    'Auburn Programs/Services Referring To',
    'Mukilteo Programs/Services Referring To',
    'Referral Source',
    'Is this Application being completed on site at Orion as part of a Site Tour?',
    'Date of Tour',
    'Agency Name',
    'Referral Contact:First Name',
    'Referral Contact:Last Name',
    'First Name',
    'Last Name',
    'Nickname',
    'Phone availabile',
    'Phone',
    'Email',
    'Address',
    'City',
    'State',
    'Zip Code:County',
    'Zip Code',
    'Guardian: Status',
    'Guardian: email',
    'Guardian: First Name',
    'Guardian: Last Name',
    'Guardian: Phone',
    'Status',
    'Orion Location'
]
intake2.columns = colname

In [359]:
# edit the duplicated columns with static values
intake2 = intake2.copy()
intake2.loc[intake2['Closing Referral without Moving Forward?'].notna(), 'Closing Referral without Moving Forward?'] = 'Yes'

#set the date value as a str
intake2['Is this Application being completed on site at Orion as part of a Site Tour?'] = intake2['Is this Application being completed on site at Orion as part of a Site Tour?'].astype(str)
intake2.loc[intake2['Is this Application being completed on site at Orion as part of a Site Tour?'].notna(), 'Is this Application being completed on site at Orion as part of a Site Tour?'] = 'Yes'

intake2.loc[intake2['Phone availabile'].notna(), 'Phone availabile'] = 'Yes'

In [360]:

intake2.loc[:,'Guardian info'] = intake2['Guardian: email']+' '+intake2['Guardian: First Name']+' '+intake2['Guardian: Last Name']+' '+intake2['Guardian: Phone']

intake3 = intake2.drop(columns=['Guardian: email','Guardian: First Name','Guardian: Last Name','Guardian: Phone']).copy()
intake3['Orion Location'] = intake3['Orion Location'].replace('Auburn',1174)
intake3['Orion Location'] = intake3['Orion Location'].replace('Mukilteo',2870)
intake3

,Assigned Staff,Date Assigned,Closing Referral without Moving Forward?,Date Closed,Referral Closed Reason,Processing Notes,Date,Site,Auburn Programs/Services Referring To,Mukilteo Programs/Services Referring To,Referral Source,Is this Application being completed on site at Orion as part of a Site Tour?,Date of Tour,Agency Name,Referral Contact:First Name,Referral Contact:Last Name,First Name,Last Name,Nickname,Phone availabile,Phone,Email,Address,City,State,Zip Code:County,Zip Code,Guardian: Status,Status,Orion Location,Guardian info
0,Cynthia Kim,2018-04-17 14:21:57,NaN,NaT,NaN,NaN,2018-04-17 14:21:57,Auburn,Auburn - Manufacturing Program,Auburn - Manufacturing Program,NaN,Yes,2018-04-09,NaN,NaN,NaN,Glen,Tillotson,NaN,Yes,253-269-2367,gatmail@lycos.com,1804 4th Street NE,Auburn,WA,NaN,NaN,No,Started,1174.0,NaN
1,Kristina Pressley,2018-08-06 11:47:26,NaN,NaT,NaN,Appt set up with DVR on 8/6/2018. On hold unt...,2018-08-06 11:47:26,Auburn,NaN,NaN,NaN,Yes,2018-08-06,NaN,NaN,NaN,Gary,Wang,NaN,Yes,425-525-8272,NaN,17314 108th Ave SE,Renton,WA,King,98055.0,Yes,Withdrawn,1174.0,gnawyaj@uw.edu Jay Wang 206-371-3676
2,Kristina Pressley,2018-08-07 13:59:54,NaN,NaT,NaN,Referred at Fry Apts event (Low Income Housing...,2018-08-07 13:59:54,Auburn,Auburn - Office Skills & Customer Service,Auburn - Office Skills & Customer Service,NaN,Yes,2018-08-07,NaN,NaN,NaN,Erica,Bricklin,NaN,Yes,206-501-6537,NaN,223 Yesler Way,Seattle,WA,King,98104.0,No,Started,1174.0,NaN
3,Kristina Pressley,2018-08-07 15:04:49,Yes,2018-08-13,Denied,Could not produce enough for UA. Chose not to...,2018-08-07 15:04:49,Auburn,NaN,NaN,47;#DVR-Puyallup,Yes,2018-08-07,47;#DVR-Puyallup,Stacy,Ricker,Carrie,Johnson,NaN,Yes,253-973-3182,carr1erunt@gmail.com,"1201 S. 336th St, #Q208",Federal Way,WA,King,98003.0,NaN,Denied,1174.0,NaN
4,Kristina Pressley,2018-08-08 09:39:37,NaN,NaT,NaN,Doug Stauffer is Noah's DVR Counselor,2018-08-08 09:39:37,Auburn,Auburn - Manufacturing Program,Auburn - Manufacturing Program,35;#DVR-Seatac,Yes,2018-07-23,35;#DVR-Seatac,NaN,NaN,Noah,Abraham,NaN,Yes,509-579-9700,noah.abraham@hotmail.com,295 Lind Ave SW,Renton,WA,King,98057.0,NaN,Started,1174.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2216,Mars Flannery,2024-07-05 13:56:33,Yes,2024-07-05,Requested to Close,"6/26/2024 - Flannery, Mars - Oliveira came for...",2024-07-05 13:56:33,Auburn,NaN,NaN,NaN,Yes,2024-06-24,NaN,NaN,NaN,Oliveira,Antonio,NaN,Yes,(470) 643-6026,firenzensakanda@gmail.com,3118 South 140th Street,Tukwila,WA,King,98168.0,NaN,Withdrawn,1174.0,NaN
2217,Mars Flannery,2024-07-05 14:20:18,Yes,2024-07-05,Requested to Close,"6/26/2024 - Flannery, Mars - Lema came for the...",2024-07-05 14:20:18,Auburn,NaN,NaN,NaN,Yes,2024-06-24,NaN,NaN,NaN,Patricio (Lema),Lema Joao,Lema,Yes,(206) 643-7095,patriciolema278@gmail.com,8700 Aurora Avenue North,Seattle,WA,King,98103.0,NaN,Withdrawn,1174.0,NaN
2218,Mars Flannery,2024-07-05 14:38:39,Yes,2024-07-05,Requested to Close,"7/5/2024 - Flannery, Mars - Erick came for the...",2024-07-05 14:38:39,Auburn,NaN,NaN,NaN,Yes,2024-06-24,NaN,NaN,NaN,Erick,Salomao,NaN,Yes,(206) 538-1755,rickbybe.4@gmail.com,8700 Aurora Avenue North,Seattle,WA,King,98103.0,NaN,Withdrawn,1174.0,NaN
2219,Mars Flannery,2024-07-05 14:52:06,Yes,2024-07-05,Requested to Close,"7/5/2024 - Flannery, Mars - Barry came for the...",2024-07-05 14:52:06,Auburn,NaN,NaN,NaN,Yes,2024-06-24,NaN,NaN,NaN,Barry,Amadou,NaN,Yes,(206) 582-9658,amadoubarilobarry177@gmail.com,3118 South 140th Street,Tukwila,WA,King,98168.0,NaN,Withdrawn,1174.0,NaN


In [361]:
# remove values from the site specific service

auburn_mask = intake3['Site'] == 'Auburn'
mukilteo_mask = intake3['Site'] == 'Mukilteo'

# Apply masks to set values to NaN where they don't match the site
intake3.loc[auburn_mask, 'Mukilteo Programs/Services Referring To'] = np.nan
intake3.loc[mukilteo_mask, 'Auburn Programs/Services Referring To'] = np.nan

intake3

,Assigned Staff,Date Assigned,Closing Referral without Moving Forward?,Date Closed,Referral Closed Reason,Processing Notes,Date,Site,Auburn Programs/Services Referring To,Mukilteo Programs/Services Referring To,Referral Source,Is this Application being completed on site at Orion as part of a Site Tour?,Date of Tour,Agency Name,Referral Contact:First Name,Referral Contact:Last Name,First Name,Last Name,Nickname,Phone availabile,Phone,Email,Address,City,State,Zip Code:County,Zip Code,Guardian: Status,Status,Orion Location,Guardian info
0,Cynthia Kim,2018-04-17 14:21:57,NaN,NaT,NaN,NaN,2018-04-17 14:21:57,Auburn,Auburn - Manufacturing Program,NaN,NaN,Yes,2018-04-09,NaN,NaN,NaN,Glen,Tillotson,NaN,Yes,253-269-2367,gatmail@lycos.com,1804 4th Street NE,Auburn,WA,NaN,NaN,No,Started,1174.0,NaN
1,Kristina Pressley,2018-08-06 11:47:26,NaN,NaT,NaN,Appt set up with DVR on 8/6/2018. On hold unt...,2018-08-06 11:47:26,Auburn,NaN,NaN,NaN,Yes,2018-08-06,NaN,NaN,NaN,Gary,Wang,NaN,Yes,425-525-8272,NaN,17314 108th Ave SE,Renton,WA,King,98055.0,Yes,Withdrawn,1174.0,gnawyaj@uw.edu Jay Wang 206-371-3676
2,Kristina Pressley,2018-08-07 13:59:54,NaN,NaT,NaN,Referred at Fry Apts event (Low Income Housing...,2018-08-07 13:59:54,Auburn,Auburn - Office Skills & Customer Service,NaN,NaN,Yes,2018-08-07,NaN,NaN,NaN,Erica,Bricklin,NaN,Yes,206-501-6537,NaN,223 Yesler Way,Seattle,WA,King,98104.0,No,Started,1174.0,NaN
3,Kristina Pressley,2018-08-07 15:04:49,Yes,2018-08-13,Denied,Could not produce enough for UA. Chose not to...,2018-08-07 15:04:49,Auburn,NaN,NaN,47;#DVR-Puyallup,Yes,2018-08-07,47;#DVR-Puyallup,Stacy,Ricker,Carrie,Johnson,NaN,Yes,253-973-3182,carr1erunt@gmail.com,"1201 S. 336th St, #Q208",Federal Way,WA,King,98003.0,NaN,Denied,1174.0,NaN
4,Kristina Pressley,2018-08-08 09:39:37,NaN,NaT,NaN,Doug Stauffer is Noah's DVR Counselor,2018-08-08 09:39:37,Auburn,Auburn - Manufacturing Program,NaN,35;#DVR-Seatac,Yes,2018-07-23,35;#DVR-Seatac,NaN,NaN,Noah,Abraham,NaN,Yes,509-579-9700,noah.abraham@hotmail.com,295 Lind Ave SW,Renton,WA,King,98057.0,NaN,Started,1174.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2216,Mars Flannery,2024-07-05 13:56:33,Yes,2024-07-05,Requested to Close,"6/26/2024 - Flannery, Mars - Oliveira came for...",2024-07-05 13:56:33,Auburn,NaN,NaN,NaN,Yes,2024-06-24,NaN,NaN,NaN,Oliveira,Antonio,NaN,Yes,(470) 643-6026,firenzensakanda@gmail.com,3118 South 140th Street,Tukwila,WA,King,98168.0,NaN,Withdrawn,1174.0,NaN
2217,Mars Flannery,2024-07-05 14:20:18,Yes,2024-07-05,Requested to Close,"6/26/2024 - Flannery, Mars - Lema came for the...",2024-07-05 14:20:18,Auburn,NaN,NaN,NaN,Yes,2024-06-24,NaN,NaN,NaN,Patricio (Lema),Lema Joao,Lema,Yes,(206) 643-7095,patriciolema278@gmail.com,8700 Aurora Avenue North,Seattle,WA,King,98103.0,NaN,Withdrawn,1174.0,NaN
2218,Mars Flannery,2024-07-05 14:38:39,Yes,2024-07-05,Requested to Close,"7/5/2024 - Flannery, Mars - Erick came for the...",2024-07-05 14:38:39,Auburn,NaN,NaN,NaN,Yes,2024-06-24,NaN,NaN,NaN,Erick,Salomao,NaN,Yes,(206) 538-1755,rickbybe.4@gmail.com,8700 Aurora Avenue North,Seattle,WA,King,98103.0,NaN,Withdrawn,1174.0,NaN
2219,Mars Flannery,2024-07-05 14:52:06,Yes,2024-07-05,Requested to Close,"7/5/2024 - Flannery, Mars - Barry came for the...",2024-07-05 14:52:06,Auburn,NaN,NaN,NaN,Yes,2024-06-24,NaN,NaN,NaN,Barry,Amadou,NaN,Yes,(206) 582-9658,amadoubarilobarry177@gmail.com,3118 South 140th Street,Tukwila,WA,King,98168.0,NaN,Withdrawn,1174.0,NaN


In [362]:
intake3['Mukilteo Programs/Services Referring To'].value_counts()

Mukilteo Programs/Services Referring To
Mukilteo - Community Based Services    188
Mukilteo - Manufacturing Program        94
Mukilteo-Light Assembly                  1
Name: count, dtype: int64

In [363]:
intake3['Auburn Programs/Services Referring To'].value_counts()

Auburn Programs/Services Referring To
Auburn - Manufacturing Program               716
Auburn - Office Skills & Customer Service    103
Auburn - Community Based Services             32
Auburn - Custodial                            22
Auburn - Contact Center                        2
Mukilteo - Manufacturing Program               1
Name: count, dtype: int64

In [364]:
# Services referring to edit

mukdict = {
    'Mukilteo - Community Based Services':'Community Based Services',
    'Mukilteo - Manufacturing Program':'Transitional Training - Manufacturing',
    'Mukilteo-Light Assembly':'Transitional Training - Manufacturing'
}
aubdict = {
    'Auburn - Manufacturing Program':'Transitional Training - Manufacturing',
    'Auburn - Office Skills & Customer Service':'Clerical Customer Service Training',
    'Auburn - Community Based Services':'Community Based Services',
    'Auburn - Custodial':'Transitional Training - Custodial',
    'Auburn - Contact Center':'Clerical Customer Service Training',
    'Mukilteo - Manufacturing Program':'Transitional Training - Manufacturing'
}

intake3['Mukilteo Programs/Services Referring To'] = intake3['Mukilteo Programs/Services Referring To'].replace(mukdict)
intake3['Auburn Programs/Services Referring To'] = intake3['Auburn Programs/Services Referring To'].replace(aubdict)

In [365]:
# processing notes edit

refrep = {
    'Denied':'Client not a good fit',
    'Requested to Close':'Client refused services',
    'Timed Out':'Client unable to be reached',
}

intake3['Referral Closed Reason'] = intake3['Referral Closed Reason'].replace(refrep)

In [366]:
intake3['Referral Source'].value_counts()

Referral Source
75;#Previous Program Participant or Word of Mouth    237
73;#WorkSource                                       100
76;#AAHAA                                             96
45;#DVR-Lynnwood                                      90
35;#DVR-Seatac                                        67
                                                    ... 
86;#Olsen & Associates                                 1
77;#Goodwill                                           1
87;#Strategic Consulting Services                      1
63;#Tacoma Rescue Mission                              1
136;#Senior Housing Stability Program                  1
Name: count, Length: 65, dtype: int64

In [367]:
# referral source edit values

pattern = r'\b\d{1,3};#'
intake3['Referral Source'] = intake3['Referral Source'].str.replace(pattern,'',regex=True)

# Remove text following "DVR" in the original column
intake3['Referral Source'] = intake3['Referral Source'].str.replace(r'DVR.*', 'DVR', regex=True)

intake3['Referral Source'] = intake3['Referral Source'].str.replace(r'DDA.*', 'DDA', regex=True)

ref2rep = {
    'Previous Program Participant or Word of Mouth':'Family/Friend',
    'WorkSource':'Private Referral Partner (example, L&I)',
    'AAHAA':'Private Referral Partner (example, L&I)',
    'Senior Housing Stability Program':'Private Referral Partner (example, L&I)',
    'Tacoma Rescue Mission':'Private Referral Partner (example, L&I)',
    'Strategic Consulting Services':'Private Referral Partner (example, L&I)',
    'Goodwill':'Private Referral Partner (example, L&I)',
    'Olsen & Associates':'Private Referral Partner (example, L&I)',
    'Fusion Family Center':'Private Referral Partner (example, L&I)',
    'A Walk to Freedom Counseling':'Private Referral Partner (example, L&I)',
    'Peer Kent':'Private Referral Partner (example, L&I)',
    'Renton Community Court Resource Center':'Other Government Organization',
    'YWCA':'Private Referral Partner (example, L&I)',
    'Asian Counseling and Referral Service (ACRS)':'Private Referral Partner (example, L&I)',
    'Union Gospel Mission':'Private Referral Partner (example, L&I)',
    'Freedom Project':'Private Referral Partner (example, L&I)',
    'Compass Career Solutions':'Private Referral Partner (example, L&I)',
    'Aerospace Joint Apprenticeship Committee (AJAC)':'Private Referral Partner (example, L&I)',
    'Auburn Food Bank':'Private Referral Partner (example, L&I)',
    'Labor & Industries':'Private Referral Partner (example, L&I)',
    'Valley Cities':'Private Referral Partner (example, L&I)',
    'Sound Mental Health':'Private Referral Partner (example, L&I)',
    'Multi-Service Center':'Private Referral Partner (example, L&I)',
    'Tacoma Rescue Mission':'Private Referral Partner (example, L&I)',
    'Neighborhood House':'Private Referral Partner (example, L&I)',
    'Ray of Hope':'Private Referral Partner (example, L&I)',
    'Thunderbird Treatment Center':'Private Referral Partner (example, L&I)',
    'Catholic Community Services':'Private Referral Partner (example, L&I)',
    'WorkStep':'Private Referral Partner (example, L&I)',
    'SJI (Seattle Jobs Initiative)':'Private Referral Partner (example, L&I)',
    'Goodwill':'Private Referral Partner (example, L&I)',
    'TRAC Associates':'Private Referral Partner (example, L&I)',
    'Vine Maple Place':'Private Referral Partner (example, L&I)',
    'Partners in Employment':'Private Referral Partner (example, L&I)',
    'Pioneer Industries':'Private Referral Partner (example, L&I)',
    'Auburn School District':'School/Education Program',
    'Snohomish County Human Services | Housing and Community Services':'Other Government Organization',
    'LCSNW - Lutheran Services of the Northwest':'Housing/Shelter Program',
    'We Care Daily Clinics':'Sobriety Program',
    'King County Sheriff Office':'Other Government Organization',
    'Alcoholics Anonymous (AA)':'Sobriety Program',
    'Auburn Community Resource Center':'Other Government Organization',
    'King County Jobs and Housing':'Housing/Shelter Program',
    'Everett Recovery Cafe':'Sobriety Program',
    'Federal Way Courts':'Other Government Organization',
    'Recovery Navigator Program':'Sobriety Program',
    'Take The Next Step':'Sobriety Program',
    'Senior Housing Stability Program':'Housing/Shelter Program',
    'Veterans Administration-Seattle':'Other Government Organization',
    'Veterans Administration-Lakewood':'Other Government Organization',
    'Department of Corrections':'Other Government Organization',
    'Low Income Housing Institute (LIHI)':'Housing/Shelter Program',
    'Battlefield Addiction':'Sobriety Program',
    'Social Media (ex. Facebook, website, etc.)':'Other',
    'Cocoon House':'Housing/Shelter Program'
}

intake3['Referral Source'] = intake3['Referral Source'].replace(ref2rep)

In [368]:
intake3['Guardian: Status'] = intake3['Guardian: Status'].fillna('No')
intake3['Site'] = intake3['Site'].fillna('Auburn')
intake3['Zip Code:County'] = intake3['Zip Code:County'].fillna('King')

In [369]:
intake3.head(2)

,Assigned Staff,Date Assigned,Closing Referral without Moving Forward?,Date Closed,Referral Closed Reason,Processing Notes,Date,Site,Auburn Programs/Services Referring To,Mukilteo Programs/Services Referring To,Referral Source,Is this Application being completed on site at Orion as part of a Site Tour?,Date of Tour,Agency Name,Referral Contact:First Name,Referral Contact:Last Name,First Name,Last Name,Nickname,Phone availabile,Phone,Email,Address,City,State,Zip Code:County,Zip Code,Guardian: Status,Status,Orion Location,Guardian info
0,Cynthia Kim,2018-04-17 14:21:57,NaN,NaT,NaN,NaN,2018-04-17 14:21:57,Auburn,Transitional Training - Manufacturing,NaN,NaN,Yes,2018-04-09,NaN,NaN,NaN,Glen,Tillotson,NaN,Yes,253-269-2367,gatmail@lycos.com,1804 4th Street NE,Auburn,WA,King,NaN,No,Started,1174.0,NaN
1,Kristina Pressley,2018-08-06 11:47:26,NaN,NaT,NaN,Appt set up with DVR on 8/6/2018. On hold unt...,2018-08-06 11:47:26,Auburn,NaN,NaN,NaN,Yes,2018-08-06,NaN,NaN,NaN,Gary,Wang,NaN,Yes,425-525-8272,NaN,17314 108th Ave SE,Renton,WA,King,98055.0,Yes,Withdrawn,1174.0,gnawyaj@uw.edu Jay Wang 206-371-3676


In [370]:
intake3['Status'].value_counts()

Status
Started                 1114
Withdrawn                512
Tour Only                236
Denied                   189
Pending / In Process     112
Approved                  44
On Hold                    1
Name: count, dtype: int64

In [371]:
intake4 = intake3[intake3['Status'] != 'Started'].reset_index().copy()

status1list = ['Tour Only','Denied','Withdrawn']
status1 = intake4[(intake4['Status'].isin(status1list)) & (intake4['Date'].dt.year == 2024)]

status2list = ['Approved','On Hold','Pending / In Process']
status2 = intake4[(intake4['Status'].isin(status2list))]

In [372]:
intake5 = pd.concat([status1,status2], ignore_index=True)
intake5 = intake5.copy()
intake5 = intake5.drop(columns='index')
intake5['migrationID'] = range(1,len(intake5) + 1)
intake5['Assigned Staff'] = intake5['Assigned Staff'].str.replace('Jeri Lucas Audit','Jeri Lucas')
intake5

,Assigned Staff,Date Assigned,Closing Referral without Moving Forward?,Date Closed,Referral Closed Reason,Processing Notes,Date,Site,Auburn Programs/Services Referring To,Mukilteo Programs/Services Referring To,Referral Source,Is this Application being completed on site at Orion as part of a Site Tour?,Date of Tour,Agency Name,Referral Contact:First Name,Referral Contact:Last Name,First Name,Last Name,Nickname,Phone availabile,Phone,Email,Address,City,State,Zip Code:County,Zip Code,Guardian: Status,Status,Orion Location,Guardian info,migrationID
0,Jeri Lucas,2024-01-08 13:43:24,Yes,2024-02-15,Client not a good fit,2/15/24 - appeal denied. Reasons include: curr...,2024-01-08 13:43:24,Auburn,NaN,NaN,NaN,Yes,2024-01-08,NaN,NaN,NaN,Billy,Jones,NaN,Yes,2069156152,NaN,7737 16th Ave SW,Seattle,WA,King,98106.0,No,Denied,1174.0,NaN,1
1,Jeri Lucas,2024-01-08 13:56:20,NaN,2024-01-30,NaN,Attended tour and completed intake paperwork o...,2024-01-08 13:56:20,Auburn,NaN,NaN,Family/Friend,Yes,2024-01-08,75;#Previous Program Participant or Word of Mouth,NaN,NaN,Ismael,Pozos Lopez,NaN,Yes,(253) 882-7054,ismael.pozosll@gmail.com,2654 SW 330th St.,Federal Way,WA,King,98023.0,No,Withdrawn,1174.0,NaN,2
2,Jeri Lucas,2024-01-08 14:03:23,NaN,2024-02-09,NaN,NaN,2024-01-08 14:03:23,Auburn,NaN,NaN,NaN,Yes,2024-01-08,NaN,NaN,NaN,Jason,Keasal,NaN,Yes,(206) 423-9915,jasonkeasal2@gmail.com,7737 16th Ave SW,Seattle,WA,King,98106.0,No,Withdrawn,1174.0,NaN,3
3,Jeri Lucas,2024-01-08 14:14:18,Yes,2024-02-15,Client not a good fit,2/15/24 - appeal denied. Reasons include - nat...,2024-01-08 14:14:18,Auburn,NaN,NaN,NaN,Yes,2024-01-08,NaN,NaN,NaN,Marvin,Caal,Marv,Yes,(425) 205-7896,marvcaal1024@gmail.com,7737 16th Ave SW,Seattle,WA,King,98106.0,No,Denied,1174.0,NaN,4
4,Jeri Lucas,2024-01-08 14:48:00,NaN,2024-01-30,NaN,01/30/2024--Closed,2024-01-08 14:48:00,Auburn,NaN,NaN,NaN,Yes,2024-01-08,NaN,NaN,NaN,Abdirisaq,Hassan,Zak,Yes,(206) 841-0492 or (206) 532-8521,fobioso2244@gmail.com,7737 16th Ave SW,Seattle,WA,King,98106.0,No,Withdrawn,1174.0,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,Mars Flannery,2024-07-02 15:32:43,NaN,NaT,NaN,"7/1/2024 - Flannery, Mars - Dina came for the ...",2024-07-02 15:32:43,Auburn,NaN,NaN,Other Government Organization,Yes,2024-07-01,119;#Auburn Community Resource Center,NaN,NaN,Dina,Buyak,NaN,Yes,(253) 987-9799,dina.buyak@comcast.net,NaN,NaN,WA,King,NaN,No,Pending / In Process,1174.0,NaN,462
462,Mars Flannery,2024-07-03 13:38:29,NaN,NaT,NaN,"7/3/2024 - Flannery, Mars - Kalaki came for th...",2024-07-03 13:38:29,Auburn,NaN,NaN,NaN,Yes,2024-06-24,NaN,NaN,NaN,Kalaki,Francisco,NaN,Yes,(929) 549-6027,mubangagofer@gmail.com,3118 South 140th Street,Tukwila,WA,King,98168.0,No,Pending / In Process,1174.0,NaN,463
463,Mars Flannery,2024-07-03 15:12:09,NaN,NaT,NaN,"6/24/2024 - Flannery, Mars - Israel came for t...",2024-07-03 15:12:09,Auburn,NaN,NaN,NaN,Yes,2024-06-24,NaN,NaN,NaN,Massamba (Israel),Matuvange,Israel,Yes,(206) 786-6782,NaN,3118 South 140th Street,Tukwila,WA,King,98168.0,No,Pending / In Process,1174.0,NaN,464
464,Cynthia Kim,2024-07-05 11:47:54,NaN,NaT,NaN,Auburn Community Lab student. Wants manufactur...,2024-07-05 11:47:54,Auburn,NaN,NaN,School/Education Program,Yes,2024-05-30,83;#Auburn School District,NaN,NaN,Mercedes,Atwood,Sadie,NaN,NaN,mercedesatwood14@gmail.com,NaN,Des Moines,WA,King,NaN,No,Pending / In Process,1174.0,NaN,465


In [373]:
pattern = r'\b\d{1,3};#'
intake5['Agency Name'] = intake5['Agency Name'].str.replace(pattern,'',regex=True)

In [374]:
people = merge6[['people_First Name','people_Last Name','demo_Date of Birth']].copy()

In [375]:
intake6 = intake5.merge(people,how='left',left_on=['First Name','Last Name'],right_on=['people_First Name','people_Last Name'])
intake6

,Assigned Staff,Date Assigned,Closing Referral without Moving Forward?,Date Closed,Referral Closed Reason,Processing Notes,Date,Site,Auburn Programs/Services Referring To,Mukilteo Programs/Services Referring To,Referral Source,Is this Application being completed on site at Orion as part of a Site Tour?,Date of Tour,Agency Name,Referral Contact:First Name,Referral Contact:Last Name,First Name,Last Name,Nickname,Phone availabile,Phone,Email,Address,City,State,Zip Code:County,Zip Code,Guardian: Status,Status,Orion Location,Guardian info,migrationID,people_First Name,people_Last Name,demo_Date of Birth
0,Jeri Lucas,2024-01-08 13:43:24,Yes,2024-02-15,Client not a good fit,2/15/24 - appeal denied. Reasons include: curr...,2024-01-08 13:43:24,Auburn,NaN,NaN,NaN,Yes,2024-01-08,NaN,NaN,NaN,Billy,Jones,NaN,Yes,2069156152,NaN,7737 16th Ave SW,Seattle,WA,King,98106.0,No,Denied,1174.0,NaN,1,NaN,NaN,NaN
1,Jeri Lucas,2024-01-08 13:56:20,NaN,2024-01-30,NaN,Attended tour and completed intake paperwork o...,2024-01-08 13:56:20,Auburn,NaN,NaN,Family/Friend,Yes,2024-01-08,Previous Program Participant or Word of Mouth,NaN,NaN,Ismael,Pozos Lopez,NaN,Yes,(253) 882-7054,ismael.pozosll@gmail.com,2654 SW 330th St.,Federal Way,WA,King,98023.0,No,Withdrawn,1174.0,NaN,2,NaN,NaN,NaN
2,Jeri Lucas,2024-01-08 14:03:23,NaN,2024-02-09,NaN,NaN,2024-01-08 14:03:23,Auburn,NaN,NaN,NaN,Yes,2024-01-08,NaN,NaN,NaN,Jason,Keasal,NaN,Yes,(206) 423-9915,jasonkeasal2@gmail.com,7737 16th Ave SW,Seattle,WA,King,98106.0,No,Withdrawn,1174.0,NaN,3,NaN,NaN,NaN
3,Jeri Lucas,2024-01-08 14:14:18,Yes,2024-02-15,Client not a good fit,2/15/24 - appeal denied. Reasons include - nat...,2024-01-08 14:14:18,Auburn,NaN,NaN,NaN,Yes,2024-01-08,NaN,NaN,NaN,Marvin,Caal,Marv,Yes,(425) 205-7896,marvcaal1024@gmail.com,7737 16th Ave SW,Seattle,WA,King,98106.0,No,Denied,1174.0,NaN,4,NaN,NaN,NaN
4,Jeri Lucas,2024-01-08 14:48:00,NaN,2024-01-30,NaN,01/30/2024--Closed,2024-01-08 14:48:00,Auburn,NaN,NaN,NaN,Yes,2024-01-08,NaN,NaN,NaN,Abdirisaq,Hassan,Zak,Yes,(206) 841-0492 or (206) 532-8521,fobioso2244@gmail.com,7737 16th Ave SW,Seattle,WA,King,98106.0,No,Withdrawn,1174.0,NaN,5,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,Mars Flannery,2024-07-02 15:32:43,NaN,NaT,NaN,"7/1/2024 - Flannery, Mars - Dina came for the ...",2024-07-02 15:32:43,Auburn,NaN,NaN,Other Government Organization,Yes,2024-07-01,Auburn Community Resource Center,NaN,NaN,Dina,Buyak,NaN,Yes,(253) 987-9799,dina.buyak@comcast.net,NaN,NaN,WA,King,NaN,No,Pending / In Process,1174.0,NaN,462,NaN,NaN,NaN
462,Mars Flannery,2024-07-03 13:38:29,NaN,NaT,NaN,"7/3/2024 - Flannery, Mars - Kalaki came for th...",2024-07-03 13:38:29,Auburn,NaN,NaN,NaN,Yes,2024-06-24,NaN,NaN,NaN,Kalaki,Francisco,NaN,Yes,(929) 549-6027,mubangagofer@gmail.com,3118 South 140th Street,Tukwila,WA,King,98168.0,No,Pending / In Process,1174.0,NaN,463,NaN,NaN,NaN
463,Mars Flannery,2024-07-03 15:12:09,NaN,NaT,NaN,"6/24/2024 - Flannery, Mars - Israel came for t...",2024-07-03 15:12:09,Auburn,NaN,NaN,NaN,Yes,2024-06-24,NaN,NaN,NaN,Massamba (Israel),Matuvange,Israel,Yes,(206) 786-6782,NaN,3118 South 140th Street,Tukwila,WA,King,98168.0,No,Pending / In Process,1174.0,NaN,464,NaN,NaN,NaN
464,Cynthia Kim,2024-07-05 11:47:54,NaN,NaT,NaN,Auburn Community Lab student. Wants manufactur...,2024-07-05 11:47:54,Auburn,NaN,NaN,School/Education Program,Yes,2024-05-30,Auburn School District,NaN,NaN,Mercedes,Atwood,Sadie,NaN,NaN,mercedesatwood14@gmail.com,NaN,Des Moines,WA,King,NaN,No,Pending / In Process,1174.0,NaN,465,NaN,NaN,NaN


In [376]:
intake6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466 entries, 0 to 465
Data columns (total 35 columns):
 #   Column                                                                        Non-Null Count  Dtype         
---  ------                                                                        --------------  -----         
 0   Assigned Staff                                                                466 non-null    object        
 1   Date Assigned                                                                 466 non-null    datetime64[ns]
 2   Closing Referral without Moving Forward?                                      131 non-null    object        
 3   Date Closed                                                                   245 non-null    datetime64[ns]
 4   Referral Closed Reason                                                        131 non-null    object        
 5   Processing Notes                                                              360 non-null  

In [377]:
intake6.to_excel(r'S:\Rehab\Apricot\Migration\exports\real intake application.xlsx')

### Next

In [191]:
testing = ['testing','this is a test','a third thing']
for i in testing:
    i = i.replace('test','')
testing

['testing', 'this is a test', 'a third thing']

### linking IDs for employers

In [687]:
orgid = pd.read_excel(r'S:\Rehab\Apricot\Migration\exports\external linking\org id.xlsx')
placementid = pd.read_excel(r'S:\Rehab\Apricot\Migration\exports\external linking\placement id.xlsx')

placemerge = placementid.merge(orgid,how='left',left_on='Employer Organization Name',right_on='Organization Name')
placemerge

,Employer Organization Name,employer placement Record ID,Organization Name,partner org Record ID
0,1st. Security Bank Home Lending Office,14993,1st. Security Bank Home Lending Office,14058.0
1,3rd Street Book Exchange,15239,3rd Street Book Exchange,14059.0
2,A&G Machine,15065,A&G Machine,14060.0
3,AAHAA HOUSE,15164,AAHAA HOUSE,14061.0
4,Accurus,15028,Accurus,14062.0
...,...,...,...,...
277,Walmart Kelsey St,15149,Walmart Kelsey St,14313.0
278,Walmart Kelsey St,15176,Walmart Kelsey St,14313.0
279,Warpony,15122,Warpony,14213.0
280,Whiplash,15003,Whiplash,14214.0


In [683]:
placemerge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284 entries, 0 to 283
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Employer Organization Name    284 non-null    object 
 1   employer placement Record ID  284 non-null    int64  
 2   Organization Name             280 non-null    object 
 3   partner org Record ID         280 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 9.0+ KB


In [688]:
placemerge.to_excel(r'S:\Rehab\Apricot\Migration\exports\placement organization linking ids2.xlsx')

In [686]:
detail = pd.read_excel(r'S:\Rehab\Apricot\Migration\exports\external linking\placement org dedupe ids.xlsx')

In [689]:
detail2 = detail[detail['Employer Organization Name'] == detail['import employer name']]
detail2

,Record ID,placement Record ID,Employer Organization Name,import employer name,details Record ID
0,Melissa Waltjen (3495),15156,Papa John's (Lake Stevens),Papa John's (Lake Stevens),14543
1,Melissa Waltjen (3495),15156,Papa John's (Lake Stevens),Papa John's (Lake Stevens),14544
2,Melissa Waltjen (3495),15156,Papa John's (Lake Stevens),Papa John's (Lake Stevens),14545
3,Rebekah Lockhart (3496),15180,Zones,Zones,14569
6,Rebekah Lockhart (3496),15181,Northwest Center---SeaTac,Northwest Center---SeaTac,14570
...,...,...,...,...,...
550,Brian Jamiel (5579),14991,Romac Industries Inc.,Romac Industries Inc.,14342
551,Kevin Houghton (5582),15125,Elliot Bay Brewery and Pub,Elliot Bay Brewery and Pub,14503
552,Monica Rutledge (5583),15164,AAHAA HOUSE,AAHAA HOUSE,14553
555,Monica Rutledge (5583),15165,Thermal NW,Thermal NW,14554


In [690]:
detail2.to_excel(r'S:\Rehab\Apricot\Migration\exports\external linking\placement org dedupe ids DEDUPED.xlsx')

### Linking goals by id

In [691]:
goaldate = pd.read_excel(r'S:\Rehab\Apricot\Migration\exports\external linking\service plan to goal linking by date.xlsx')

In [706]:
goaldate = goaldate.sort_values(by=['Date Goal Opened','sub service date'],ascending=True)

In [707]:
goaldate

,Record ID,service plan Record ID,Service text calc,sub service date,Date Goal Opened,goal Record ID,Import service text calc
249,Jacob Griffin (3540),12870,Community Inclusion,08/01/2023,01/01/2024,16228,Community Inclusion
250,Jacob Griffin (3540),12870,Community Inclusion,08/01/2023,01/01/2024,16229,Community Inclusion
1466,Blair Buchanan (5471),12666,Job Retention,10/10/2023,01/01/2024,15426,Job Retention
296,Bryan Miller (3556),12683,Job Retention,01/02/2024,01/02/2024,15473,Job Retention
297,Bryan Miller (3556),12684,Job Development,09/07/2023,01/02/2024,15472,Job Development
...,...,...,...,...,...,...,...
981,Steven Kelleher (3741),13189,Case Management,12/21/2023,12/21/2023,17696,Case Management
2122,Steven Mason (5587),13194,Case Management,11/21/2023,12/23/2022,17702,Case Management
2120,Steven Mason (5587),13193,Case Management,12/02/2022,12/23/2022,17702,Case Management
1546,Malcolm Campbell (5491),13032,Job Retention,01/27/2021,12/28/2020,16879,Job Retention


In [716]:
goaldate_equal = goaldate[goaldate['Date Goal Opened'] == goaldate['sub service date']].copy()
goaldate_unequal = goaldate[goaldate['Date Goal Opened'] != goaldate['sub service date']].copy()

In [ ]:
# goaldate_equal.to_excel(r'S:\Rehab\Apricot\Migration\exports\external linking\goal service equal DEDUPED.xlsx')

In [718]:
goaldate_unequal.info()

<class 'pandas.core.frame.DataFrame'>
Index: 604 entries, 249 to 1544
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Record ID                 604 non-null    object
 1   service plan Record ID    604 non-null    int64 
 2   Service text calc         604 non-null    object
 3   sub service date          604 non-null    object
 4   Date Goal Opened          604 non-null    object
 5   goal Record ID            604 non-null    int64 
 6   Import service text calc  604 non-null    object
dtypes: int64(2), object(5)
memory usage: 37.8+ KB


In [719]:
# Filter to ensure 'Date Goal Opened' is on or after 'sub service date'
filtered_goaldate = goaldate_unequal[goaldate_unequal['Date Goal Opened'] >= goaldate_unequal['sub service date']]

# Remove duplicates based on 'goal Record ID' while keeping the first occurrence
goaldate2 = filtered_goaldate.drop_duplicates(subset='goal Record ID', keep='first')

In [728]:
goaldate3 = pd.concat([goaldate_equal,goaldate2])
goaldate3 = goaldate3.drop_duplicates(subset='goal Record ID', keep='first')

In [729]:
goaldate3 = goaldate3.sort_values(by='Record ID')

In [731]:
goaldate3.to_excel(r'S:\Rehab\Apricot\Migration\exports\external linking\goal service DEDUPED.xlsx')

In [724]:
# bad code

# goaldate2 = goaldate[goaldate['Date Goal Opened'] >= goaldate['sub service date'] and goaldate['goal Record ID'].is_unique()]